In [1]:
import os
import ast
import json
import ineqpy
import numpy as np
import pandas as pd
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go

from pathlib import Path
from collections import defaultdict

In [2]:
path_meta = Path("../meta")
path_data = Path("../data")
path_key = Path("../key")
font_size = 15

workflow_pwc = ['1']
display_format_pwc = ['Rankable Table', 'Scatter Plot']
display_format_rankable = ['Rankable Table', 'Rankable Bar Chart']
workflow_patterns_with_submission = ['1', '2', '3', '4', '5']
non_accepted_publication_type = ['blog', 'report', 'white paper']

leaderboard_system_without_evaluation_datasets = ['Ko Chatbot Arena']
leaderboard_system_without_pull_requests = ['SEAL', 'FSMBench', 'CompMix', 'DocVQA', 'GENIE', 'InfographicVQA', 'LMExamQA', 'Models Leaderboard', 'MP-DocVQA', 'OpenEval', 'Program Synthesis Models Leaderboard', 'ProtoQA', 'TextSynth Server']
leaderboard_system_without_contacts = ['OpenEval']
leaderboard_system_without_documentation = ['FSMBench']
leaderboard_system_without_evaluation_harness = ['LLM-Leaderboard']

macro_display_format_mapping = {
    'Table': ['Regular Table', 'Rankable Table', 'Table Screenshot'],
    'Figure': ['Bar Chart', 'Box Plot', 'Heat Map', 'Line Chart', 'Pie Chart', 'Radar Chart', 'Scatter Plot', 'Rankable Bar Chart'],
}

platform_abbreviation_mapping = {
    'gh': 'GitHub',
    'hf': 'Hugging Face Spaces',
    'pwc': 'Papers With Code',
    'ip': 'independent platform',
}

ranking_dataframe_organization_tactics_mapping = {
    'Algorithm': ['Algorithm'],
    'Evaluation Configuration': ['Dataset Slice', 'Evaluator', 'Metric'],
    'Leaderboard Aggregation': ['Aggregated Result'],
    'Leaderboard Release': ['Leaderboard Launch Date', 'Leaderboard Version'],
    'Model Capability': ['Benchmark Dataset', 'Service Load', 'Supported Functionality', 'Supported Language', 'Supported Modality', 'Task'],
    'Model Information': ['Model Accessibility', 'Model Size', 'Model Type'],
    'Prompt Engineering': ['#Prompt Example', 'Output Length', 'Prompt Design', 'Prompt Length', 'Relation Extractor', 'Tokenizer']
}

publisher_synonyms = {
    'Alibaba Group': ['DAMO Academy', 'Aliyun'],
    'Amazon': ['Alexa AI Lab', 'AWS AI Lab'],
    'Agency for Science Technology and Research': ['A*STAR I2R', 'CFAR A*STAR'],
    'Bauhaus University': ['Bauhaus University Weimar'],
    'ByteDance': ['ByteDance AI Lab'],
    'Chinese Academy of Sciences': ['University of Chinese Academy of Sciences'],
    'CSIRO': ['Data61'],
    'Fraunhofer Society': ['Fraunhofer Institute for Integrated Circuits'],
    'Frédéric Joliot Institute for Life Sciences': ['NeuroSpin'],
    'Google': ['Google Brain', 'Google DeepMind', 'Google Blueshift'],
    'Huawei': ["Huawei Noah's Ark Lab"],
    'Jingdong': ['Explore Academy'],
    'Kunlun Tech': ['Skywork'],
    'Max Planck Society': ['Max Planck Institute for Informatics', 'Max Planck Institute for Intelligent Systems', 'Max Planck Institute for Mathematics in the Sciences'],
    'Meta': [
        'Meta FAIR',
        'Meta GenAI'
    ],
    'Microsoft': ['Microsoft Research Asia'],
    'National Public School': ['National Public School HSR Layout'],
    'Naver': ['Naver Labs Europe'],
    'Queen\'s University Kingston': ['Queen\'s University'],
    'Tencent': [
        'Tencent AI Lab',
        'Tencent ARC Lab',
        'Tencent Youtu Lab'
    ],
    'Toyota Technological Institute': ['Toyota Technological Institute Chicago'],
    'University of California': [
        'University of California Berkeley',
        'University of California Davis',
        'University of California Irvine',
        'University of California Los Angeles',
        'University of California Santa Barbara',
        'University of California Santa Cruz',
        'University of California San Diego',
    ],
    'University of Michigan': ['University of Michigan Ann Arbor'],
    'University of North Carolina': ['University of North Carolina Chapel Hill'],
    'University of Illinois': ['University of Illinois Chicago', 'University of Illinois Urbana Champaign'],
    'University of Massachusetts': ['University of Massachusetts Amherst'],
    'University of Maryland': ['University of Maryland College Park'],
    'University of Tennessee': ['University of Tennessee Knoxville'],
    'University of Texas': ['University of Texas Austin'],
    'University of Wisconsin': ['University of Wisconsin Madison'],
}

metrics_synonyms = {
    'accuracy': [
        'acc',
        'accuarcy',
        'qa accuracy'
    ],
    'average': [
        'avg',
        '平均'
    ],
    'average accuracy': [
        'avg. accuracy'
    ],
    'average score': ['平均分'],
    'bleu': ['bleu score'],
    'bleu-1': [
        'narrativeqa bleu-1',
        'socialiqa bleu-1',
        'mcscript bleu-1',
        'cosmosqa bleu-1'
    ],
    'bleu-4': ['bleu4'],
    'bertscore': ['bert score'],
    'code': ['代码'],
    'elo rating': [
        'chatbot arena elo',
        'elo'
    ],
    'exact match': ['em', 'exact match accuracy'],
    'lerc': [
        'cosmosqa lerc',
        'mcscript lerc',
        'socialiqa lerc',
        'narrativeqa lerc'
    ],
    'link': ['url'],
    'mean rank': [
        'text-to-video mean rank',
        'video-to-text mean rank'
    ],
    'median rank': [
        'text-to-video median rank',
        'video-to-text median rank',
        'text-to-videomedian rank',
        'text-to-video medianr'
    ],
    'meteor': [
        'cosmosqa meteor',
        'narrativeqa meteor',
        'socialiqa meteor',
        'mcscript meteor'
    ],
    'neg mean rank': [
        'i->t neg mean rank',
        't->i neg mean rank'
    ],
    'organization': [
        '发布机构',
        '机构',
        'orgaisation',
    ],
    'others': ['其他'],
    'overall': ['xiezhi overall'],
    'overall score': [
        '总分',
        '总体分数'
    ],
    'pass@1': [
        'interview pass@1',
        'competition pass@1',
        'introductory pass@1'
    ],
    'pass@5': [
        'interview pass@5',
        'introductory pass@5',
        'competition pass@5'
    ],
    'pass@1000': [
        'interview pass@1000',
        'competition pass@1000',
        'introductory pass@1000'
    ],
    'pass@any': [
        'introductory pass@any',
        'competition pass@any',
        'interview pass@any'
    ],
    '#parameters':  [
        '#p',
        '#params',
        '# params',
        '#size',
        '参数量',
        'model size',
        'model size/b',
        'number of params',
        'param',
        'parameters',
        'params',
        'size'
    ],
    'perplexity': ['ppl'],
    'precision@1': ['i->t p@1'],
    'precision@20': ['p@20'],
    'recall@1': [
        'r@1',
        'text-to-videor@1',
        'video-to-text r@1',
        'text-to-video r@1',
        'text-to-image r@1',
        'image-to-text r@1'
    ],
    'recall@5': [
        'text-to-image r@5',
        'video-to-text r@5',
        'image-to-text r@5',
        'text-to-video r@5',
        'r@5',
    ],
    'recall@10': [
        'recall@10 on 1 rounds',
        'recall@10 on 2 rounds',
        'recall@10 on 3 rounds',
        'r@10',
        'video-to-text r@10',
        'text-to-image r@10',
        'text-to-video r@10',
        'image-to-text r@10'
    ],
    'recall@50': [
        'text-to-video r@50',
        'video-to-text r@50',
    ],
    'score': ['分数'],
    'submission date': ['提交时间'],
    'top-1 accuracy': ['top 1 accuracy'],
    'top-5 accuracy': ['top 5 accuracy'],
    'type': ['model type'],
    'win rate': ['胜率'],
    'word error rate': ['wer']
}

pwc_leaderboard_mapping = {
    'A-OKVQA': ['visual-question-answering-on-a-okvqa'],
    'ACE (2005)': ['relation-extraction-on-ace-2005'],
    'ADE20K': ['semantic-segmentation-on-ade20k'],
    'AI2D': ['visual-question-answering-vqa-on-ai2d'],
    'AISHELL-1': ['speech-recognition-on-aishell-1'],
    'ANLI': ['natural-language-inference-on-anli-test'],
    'APPS': ['code-generation-on-apps'],
    'ARC Challenge': ['common-sense-reasoning-on-arc-challenge', 'common-sense-reasoning-on-arc-easy'],
    'ASDiv': ['math-word-problem-solving-on-asdiv-a'],
    'AVA': ['action-recognition-on-ava-v2-2', 'spatio-temporal-action-localization-on-ava'],
    'ActivityNet Captions': ['video-captioning-on-activitynet-captions', 'video-captioning-on-activitynet-captions'],
    'ActivityNet': ['temporal-action-localization-on-activitynet', 'video-retrieval-on-activitynet', 'zero-shot-video-retrieval-on-activitynet'],
    'ActivityNet-QA': ['video-question-answering-on-activitynet-qa'],
    'AudioCaps': ['audio-to-text-retrieval-on-audiocaps', 'text-to-audio-retrieval-on-audiocaps', 'zero-shot-audio-captioning-on-audiocaps'],
    'BACE': ['molecular-property-prediction-on-bace-1'],
    'BANKING77': ['intent-detection-on-banking77'],
    'BBBP': ['molecular-property-prediction-on-bbbp-1'],
    'BBH': ['multi-task-language-understanding-on-bbh-nlp', 'multi-task-language-understanding-on-bbh-alg'],
    'BC5CDR': ['named-entity-recognition-ner-on-bc5cdr'],
    'BEIR': ['argument-retrieval-on-arguana-beir', 'argument-retrieval-on-touche-2020-beir', 'biomedical-information-retrieval-on-bioasq-1', 'biomedical-information-retrieval-on-nfcorpus-1', 'biomedical-information-retrieval-on-trec-1', 'citation-prediction-on-scidocs-beir', 'duplicate-question-retrieval-on-cqadupstack-1', 'duplicate-question-retrieval-on-quora-beir', 'entity-retrieval-on-dbpedia-beir', 'fact-checking-on-scifact-beir', 'fact-checking-on-fever-beir', 'fact-checking-on-climate-fever-beir', 'zero-shot-text-search-on-beir', 'news-retrieval-on-trec-news-beir', 'passage-retrieval-on-msmarco-beir', 'question-answering-on-fiqa-2018-beir', 'question-answering-on-hotpotqa-beir', 'tweet-retrieval-on-signal-1m-rt-beir'],
    'BIG-Bench': ['mathematical-induction-on-big-bench', 'physics-mc-on-big-bench', 'riddle-sense-on-big-bench', 'figure-of-speech-detection-on-big-bench', 'fantasy-reasoning-on-big-bench', 'common-sense-reasoning-on-big-bench-logical', 'logical-reasoning-on-big-bench-logical', 'analogical-similarity-on-big-bench', 'implicit-relations-on-big-bench', 'intent-recognition-on-big-bench', 'lambada-on-big-bench', 'movie-dialog-same-or-different-on-big-bench', 'nonsense-words-grammar-on-big-bench', 'phrase-relatedness-on-big-bench', 'question-selection-on-big-bench', 'misconceptions-on-big-bench', 'sentence-ambiguity-on-big-bench', 'general-knowledge-on-big-bench', 'analytic-entailment-on-big-bench', 'entailed-polarity-on-big-bench', 'epistemic-reasoning-on-big-bench', 'evaluating-information-essentiality-on-big', 'logical-args-on-big-bench', 'metaphor-boolean-on-big-bench', 'physical-intuition-on-big-bench', 'presuppositions-as-nli-on-big-bench', 'discourse-marker-prediction-on-big-bench', 'empirical-judgments-on-big-bench', 'irony-identification-on-big-bench', 'english-proverbs-on-big-bench', 'similarities-abstraction-on-big-bench', 'gre-reading-comprehension-on-big-bench', 'crash-blossom-on-big-bench', 'human-organs-senses-multiple-choice-on-big', 'odd-one-out-on-big-bench', 'identify-odd-metapor-on-big-bench', 'dark-humor-detection-on-big-bench', 'understanding-fables-on-big-bench', 'timedial-on-big-bench', 'common-sense-reasoning-on-big-bench-known', 'language-modelling-on-big-bench-lite', 'auto-debugging-on-big-bench-lite', 'memorization-on-big-bench-hindu-knowledge', 'logical-reasoning-on-big-bench-strategyqa', 'logical-reasoning-on-big-bench-logic-grid', 'crass-ai-on-big-bench', 'multiple-choice-question-answering-mcqa-on-31', 'word-sense-disambiguation-on-big-bench', 'multi-task-language-understanding-on-bbh-alg', 'sarcasm-detection-on-big-bench-snarks', 'common-sense-reasoning-on-big-bench-sports', 'multiple-choice-question-answering-mcqa-on-30', 'logical-reasoning-on-big-bench-temporal', 'common-sense-reasoning-on-big-bench', 'common-sense-reasoning-on-big-bench-causal', 'common-sense-reasoning-on-big-bench-date', 'logical-reasoning-on-big-bench-formal', 'multiple-choice-question-answering-mcqa-on-27', 'multiple-choice-question-answering-mcqa-on-28', 'multiple-choice-question-answering-mcqa-on-29', 'logical-reasoning-on-big-bench-penguins-in-a', 'logical-reasoning-on-big-bench-reasoning', 'multi-task-language-understanding-on-bbh-nlp'],
    'BenchLMM': ['visual-question-answering-on-benchlmm'],
    'BioASQ': ['question-answering-on-bioasq'],
    'BioLAMA': ['knowledge-probing-on-biolama'],
    'BoolQ': ['question-answering-on-boolq'],
    'C4': ['language-modelling-on-c4'],
    'CB': ['natural-language-inference-on-commitmentbank'],
    'CFQ': ['semantic-parsing-on-cfq'],
    'CIFAR-10': ['image-classification-on-cifar-10'],
    'CLUE': ['language-modelling-on-clue-afqmc', 'language-modelling-on-clue-ocnli-50k', 'language-modelling-on-clue-drcd', 'language-modelling-on-clue-cmnli', 'language-modelling-on-clue-wsc1-1', 'language-modelling-on-clue-c3', 'language-modelling-on-clue-cmrc2018'],
    'CNN DM': ['document-summarization-on-cnn-daily-mail', 'abstractive-text-summarization-on-cnn-daily'],
    'COCO Captions': ['image-captioning-on-coco-captions'],
    'COPA': ['question-answering-on-copa'],
    'ChEBI-20': ['cross-modal-retrieval-on-chebi-20', 'text-based-de-novo-molecule-generation-on', 'molecule-captioning-on-chebi-20'],
    'Charades': ['zero-shot-action-recognition-on-charades-1'],
    'Charades-STA': ['moment-retrieval-on-charades-sta'],
    'ChartQA': ['chart-question-answering-on-chartqa'],
    'Civil Comments': ['toxic-comment-classification-on-civil'],
    'Clotho': ['text-to-audio-retrieval-on-clotho', 'audio-captioning-on-clotho', 'audio-to-text-retrieval-on-clotho', 'zero-shot-text-to-audio-retrieval-on-clotho'],
    'CoLA': ['linguistic-acceptability-on-cola'],
    'CoNLL': ['coreference-resolution-on-conll-2012', 'coreference-resolution-on-conll12', 'joint-entity-and-relation-extraction-on-2', 'named-entity-recognition-ner-on-conll-2003', 'named-entity-recognition-on-conll03', 'relation-extraction-on-conll04', 'semantic-role-labeling-on-conll-2005', 'semantic-role-labeling-on-conll05-wsj', 'semantic-role-labeling-on-conll12'],
    'CoNaLa': ['code-generation-on-conala'],
    'CoQA': ['question-answering-on-coqa'],
    'CodeContests': ['code-generation-on-codecontests'],
    'Common Voice': ['speech-recognition-on-common-voice-8-0-13', 'speech-recognition-on-common-voice-8-0-14', 'speech-recognition-on-common-voice-8-0-35', 'speech-recognition-on-common-voice-8-0-21', 'speech-recognition-on-common-voice-8-0-19', 'speech-recognition-on-common-voice-8-0-15', 'speech-recognition-on-common-voice-7-0-irish', 'speech-recognition-on-common-voice-8-0-irish', 'speech-recognition-on-common-voice-2', 'speech-recognition-on-common-voice-7-0-3', 'speech-recognition-on-common-voice-8-0-german', 'speech-recognition-on-common-voice-8-0-6', 'speech-recognition-on-common-voice-7-0-4', 'speech-recognition-on-common-voice-7-0-13', 'speech-recognition-on-common-voice-8-0-37', 'speech-recognition-on-common-voice-8-0-22', 'speech-recognition-on-common-voice-8-0', 'speech-recognition-on-common-voice-frisian', 'speech-recognition-on-common-voice-8-0-4', 'speech-recognition-on-common-voice-italian', 'speech-recognition-on-common-voice-8-0-french', 'speech-recognition-on-common-voice-english', 'speech-recognition-on-common-voice-8-0-hindi', 'speech-recognition-on-common-voice-7-0-5', 'speech-recognition-on-common-voice-7-0-hindi', 'speech-recognition-on-common-voice-8-0-dutch', 'speech-recognition-on-common-voice-spanish', 'speech-recognition-on-common-voice-french', 'speech-recognition-on-common-voice-german'],
    'ComplexWebQuestions': ['knowledge-base-question-answering-on'],
    'CrossNER': ['zero-shot-named-entity-recognition-ner-on-1'],
    'CrowS-Pairs': ['stereotypical-bias-analysis-on-crows-pairs'],
    'DROP': ['question-answering-on-drop-test', 'question-answering-on-drop'],
    'DiDeMo': ['video-retrieval-on-didemo', 'zero-shot-video-retrieval-on-didemo'],
    'DocVQA': ['visual-question-answering-on-docvqa-test'],
    'FLEURS': ['automatic-speech-recognition-on-fleurs-1'],
    'FSD50K': ['audio-classification-on-fsd50k'],
    'FewCLUE': ['language-modelling-on-fewclue-eprstmt', 'language-modelling-on-fewclue-ocnli-fc', 'language-modelling-on-fewclue-bustm', 'language-modelling-on-fewclue-chid-fc', 'language-modelling-on-fewclue-cluewsc-fc'],
    'FinQA': ['question-answering-on-finqa'],
    'FineAction': ['temporal-action-localization-on-fineaction'],
    'Flickr30K': ['zero-shot-cross-modal-retrieval-on-flickr30k', 'image-to-text-retrieval-on-flickr30k'],
    'Food-101': ['zero-shot-transfer-image-classification-on-17'],
    'GEM': ['extreme-summarization-on-gem-xsum'],
    'GENIA': ['named-entity-recognition-on-genia'],
    'GQA': ['visual-question-answering-on-gqa-test-dev'],
    'GSM8K': ['arithmetic-reasoning-on-gsm8k'],
    'HIV': ['molecular-property-prediction-on-hiv-dataset'],
    'HMDB51': ['action-recognition-in-videos-on-hmdb-51', 'zero-shot-action-recognition-on-hmdb51'],
    'HallusionBench': ['visual-question-answering-vqa-on-3'],
    'HellaSwag': ['sentence-completion-on-hellaswag'],
    'HumanEval': ['code-generation-on-humaneval'],
    'IFEval': ['instruction-following-on-ifeval'],
    'ImageNet': ['few-shot-image-classification-on-imagenet-10', 'few-shot-image-classification-on-imagenet-1-1', 'few-shot-image-classification-on-imagenet-5', 'image-classification-on-imagenet', 'self-supervised-image-classification-on', 'self-supervised-image-classification-on-1', 'zero-shot-transfer-image-classification-on-1', 'zero-shot-transfer-image-classification-on-3'],
    'iNaturalist': ['long-tail-learning-on-inaturalist-2018', 'image-classification-on-inaturalist-2018'],
    'InfographicVQA': ['visual-question-answering-vqa-on'],
    'InfoSeek': ['visual-question-answering-vqa-on-infoseek'],
    'IUPAC': ['iupac-name-prediction-on-iupac'],
    'iVQA': ['video-question-answering-on-ivqa'],
    'JFT-300M': ['image-classification-on-jft-300m'],
    'Kinetics': ['action-classification-on-kinetics-400', 'action-classification-on-kinetics-600', 'action-classification-on-kinetics-700', 'spatio-temporal-action-localization-on-ava', 'video-generation-on-kinetics-600-12-frames', 'zero-shot-action-recognition-on-kinetics'],
    'LAMBADA': ['language-modelling-on-lambada'],
    'LLaVA-Bench': ['visual-instruction-following-on-llava-bench'],
    'LibriSpeech': ['speech-recognition-on-librispeech-test-clean', 'speech-recognition-on-librispeech-test-other'],
    'MATH': ['math-word-problem-solving-on-math'],
    'MAWPS': ['math-word-problem-solving-on-mawps'],
    'MBPP': ['code-generation-on-mbpp'],
    'MGSM': ['multi-task-language-understanding-on-mgsm'],
    'MM-Vet': ['visual-question-answering-on-mm-vet'],
    'MMLU': ['multi-task-language-understanding-on-mmlu', 'mathematical-reasoning-on-mmlu-mathematics', 'multiple-choice-question-answering-mcqa-on-11', 'multiple-choice-question-answering-mcqa-on-8', 'multiple-choice-question-answering-mcqa-on-25', 'multiple-choice-question-answering-mcqa-on-7', 'multiple-choice-question-answering-mcqa-on-9', 'multiple-choice-question-answering-mcqa-on-10', 'multiple-choice-question-answering-mcqa-on-13', 'multiple-choice-question-answering-mcqa-on-12', 'multiple-choice-question-answering-mcqa-on-2', 'multiple-choice-question-answering-mcqa-on-3', 'multiple-choice-question-answering-mcqa-on-4', 'multiple-choice-question-answering-mcqa-on-5', 'multiple-choice-question-answering-mcqa-on-16', 'multiple-choice-question-answering-mcqa-on-17', 'multiple-choice-question-answering-mcqa-on-18', 'multiple-choice-question-answering-mcqa-on-20', 'multiple-choice-question-answering-mcqa-on-14', 'multiple-choice-question-answering-mcqa-on-19', 'multiple-choice-question-answering-mcqa-on-6', 'multiple-choice-question-answering-mcqa-on-15', 'multiple-choice-question-answering-mcqa-on-24', 'multiple-choice-question-answering-mcqa-on-26', 'multiple-choice-question-answering-mcqa-on-23'],
    'MRPC': ['semantic-textual-similarity-on-mrpc'],
    'MSCOCO': ['image-captioning-on-coco', 'image-retrieval-on-coco', 'image-to-text-retrieval-on-coco', 'text-to-image-generation-on-coco', 'zero-shot-cross-modal-retrieval-on-coco-2014'],
    'MSRVTT': ['text-to-video-generation-on-msr-vtt', 'video-captioning-on-msr-vtt-1', 'video-retrieval-on-msr-vtt', 'zero-shot-video-retrieval-on-msr-vtt', 'video-retrieval-on-msr-vtt-1ka'],
    'MSRVTT-QA': ['zeroshot-video-question-answer-on-msrvtt-qa', 'video-question-answering-on-msrvtt-qa', 'visual-question-answering-on-msrvtt-qa-1'],
    'MSVD': ['video-retrieval-on-msvd', 'video-captioning-on-msvd-1', 'zero-shot-video-retrieval-on-msvd'],
    'MSVD-QA': ['zeroshot-video-question-answer-on-msvd-qa', 'zeroshot-video-question-answer-on-msvd-qa'],
    'MathQA': ['math-word-problem-solving-on-mathqa'],
    'MedMCQA': ['multiple-choice-question-answering-mcqa-on-21'],
    'MedQA': ['question-answering-on-medqa-usmle'],
    'MiT': ['action-classification-on-moments-in-time'],
    'MoleculeNet': ['molecular-property-prediction-on-moleculenet'],
    'MultiRC': ['question-answering-on-multirc'],
    'MusicCaps': ['text-to-music-generation-on-musiccaps'],
    'MusicQA': ['music-question-answering-on-musicqa'],
    'NExT-QA': ['temporal-casual-qa-on-next-qa', 'question-answering-on-next-qa-open-ended', 'video-question-answering-on-next-qa', 'zero-shot-video-question-answer-on-next-qa', 'zero-shot-video-question-answer-on-next-gqa'],
    'NLVR': ['visual-reasoning-on-nlvr2-test', 'visual-reasoning-on-nlvr2-dev'],
    'NQ': ['passage-retrieval-on-natural-questions', 'question-answering-on-natural-questions'],
    'NoCaps': ['image-captioning-on-nocaps-entire', 'image-captioning-on-nocaps-near-domain', 'image-captioning-on-nocaps-out-of-domain', 'image-captioning-on-nocaps-in-domain', 'image-captioning-on-nocaps-val-in-domain', 'image-captioning-on-nocaps-val-overall', 'image-captioning-on-nocaps-val-out-domain', 'image-captioning-on-nocaps-val-near-domain'],
    'OK-VQA': ['visual-question-answering-on-ok-vqa'],
    'OVEN': ['fine-grained-image-recognition-on-oven'],
    'OmniBenchmark': ['image-classification-on-omnibenchmark'],
    'OpenbookQA': ['question-answering-on-openbookqa', 'question-answering-on-obqa'],
    'PIQA': ['question-answering-on-piqa'],
    'PMC-VQA': ['generative-visual-question-answering-on-pmc', 'visual-question-answering-vqa-on-pmc-vqa', 'medical-visual-question-answering-on-pmc-vqa'],
    'Pets37': ['fine-grained-image-classification-on-oxford-1'],
    'PubChemQA': ['question-answering-on-pubchemqa'],
    'PubMedQA': ['question-answering-on-pubmedqa'],
    'QNLI': ['natural-language-inference-on-qnli'],
    'RACE': ['reading-comprehension-on-race', 'question-answering-on-race'],
    'RAFT': ['few-shot-text-classification-on-raft'],
    'RTE': ['natural-language-inference-on-rte'],
    'RareAct': ['action-recognition-on-rareact'],
    'ReCoRD': ['common-sense-reasoning-on-record'],
    'RefCOCO': ['referring-expression-comprehension-on-refcoco-1', 'referring-expression-comprehension-on-refcoco-1', 'referring-expression-segmentation-on-refcocog', 'referring-expression-segmentation-on-refcoco', 'referring-expression-comprehension-on-1', 'referring-expression-comprehension-on', 'referring-expression-comprehension-on-refcoco'],
    'Robust (2004)': ['ad-hoc-information-retrieval-on-trec-robust04'],
    'SCROLLS': ['long-range-modeling-on-scrolls'],
    'SICK': ['semantic-textual-similarity-on-sick', 'semantic-textual-similarity-on-sick-r-1'],
    'SIDER': ['molecular-property-prediction-on-sider-1'],
    'SIQA': ['question-answering-on-social-iqa'],     
    'SPGISpeech': ['speech-recognition-on-spgispeech'],
    'SQuAD': ['question-answering-on-squad11-dev'],
    'SST': ['sentiment-analysis-on-sst-2-binary'],
    'STAR': ['video-question-answering-on-situated', 'zero-shot-video-question-answer-on-star', 'zero-shot-video-question-answer-on-star-1'],
    'STS-B': ['semantic-textual-similarity-on-sts-benchmark', 'semantic-textual-similarity-on-sts13', 'semantic-textual-similarity-on-sts14', 'semantic-textual-similarity-on-sts12', 'semantic-textual-similarity-on-sts15', 'semantic-textual-similarity-on-sts16', 'semantic-similarity-on-sts-benchmark'],
    'SVAMP': ['math-word-problem-solving-on-svamp'],
    'SWE-bench': ['bug-fixing-on-swe-bench'],
    'SciQ': ['text-generation-on-sciq'],
    'ScienceQA': ['science-question-answering-on-scienceqa'],
    'Something-Something': ['action-recognition-in-videos-on-something'],
    'Spider': ['text-to-sql-on-spider', 'semantic-parsing-on-spider'],
    'StereoSet': ['bias-detection-on-stereoset-1'],
    'StoryCloze': ['question-answering-on-storycloze'],
    'StrategyQA': ['question-answering-on-strategyqa'],
    'TACRED': ['relation-extraction-on-tacred'],
    'TED-LIUM': ['speech-recognition-on-tedlium'],
    'TGIF-QA': ['tgif-frame-on-tgif-qa', 'zeroshot-video-question-answer-on-tgif-qa', 'zeroshot-video-question-answer-on-tgif-qa-1', 'visual-question-answering-on-tgif-qa'],
    'TREC-COVID': ['zero-shot-text-search-on-trec-covid'],
    'TVQA': ['video-question-answering-on-tvqa', 'zero-shot-video-question-answer-on-tvqa'],
    'TextVQA': ['visual-question-answering-on-textvqa-test-1'],
    'TheoremQA': ['natural-questions-on-theoremqa'],
    'The Pile': ['language-modelling-on-the-pile'],
    'Tox21': ['language-modelling-on-the-pile'],
    'TriviaQA': ['question-answering-on-triviaqa'],
    'TruthfulQA': ['question-answering-on-truthfulqa'],
    'TyDiQA-GoldP': ['cross-lingual-question-answering-on-tydiqa'],
    'UCF101': ['action-recognition-in-videos-on-ucf101', 'self-supervised-action-recognition-on-ucf101', 'text-to-video-generation-on-ucf-101', 'zero-shot-action-recognition-on-ucf101', 'video-generation-on-ucf-101'],
    'UniProtQA': ['question-answering-on-uniprotqa'],
    'VATEX': ['video-retrieval-on-vatex', 'video-captioning-on-vatex-1', 'zero-shot-video-retrieval-on-vatex'],
    'VCR': ['visual-question-answering-on-vcr-q-ar-test', 'visual-question-answering-on-vcr-qa-r-test', 'visual-question-answering-on-vcr-q-a-test'],
    'VGG-Sound': ['audio-classification-on-vggsound'],
    'VNHSGE': ['question-answering-on-vnhsge-english', 'question-answering-on-vnhsge-physics', 'question-answering-on-vnhsge-chemistry', 'question-answering-on-vnhsge-biology', 'question-answering-on-vnhsge-history', 'question-answering-on-vnhsge-geography', 'question-answering-on-vnhsge-literature', 'question-answering-on-vnhsge-mathematics-1', 'question-answering-on-vnhsge-civic'],
    'VQA': ['visual-question-answering-on-vqa-v2-val-1', 'visual-question-answering-on-vqa-v2-test-dev-1', 'visual-question-answering-on-vqa-v2-val', 'visual-question-answering-on-vqa-v2-test-std', 'visual-question-answering-on-vqa-v2-test-dev'],
    'VeRi-776': ['vehicle-re-identification-on-veri-776'],
    'VeRi-Wild': ['vehicle-re-identification-on-veri-wild-small'],
    'ViP-Bench': ['visual-question-answering-on-vip-bench'],
    'VideoInstruct': ['video-based-generative-performance', 'video-based-generative-performance-1', 'video-based-generative-performance-2', 'video-based-generative-performance-3', 'video-based-generative-performance-4', 'video-based-generative-performance-5'],
    'VisDial': ['chat-based-image-retrieval-on-visdial'],
    'VizWiz': ['visual-question-answering-on-vizwiz-2020-vqa'],
    'WHOOPS!': ['explanation-generation-on-whoops', 'image-to-text-retrieval-on-whoops', 'image-captioning-on-whoops', 'visual-question-answering-vqa-on-whoops'],
    'WMT (2014)': ['unsupervised-machine-translation-on-wmt2014-1', 'machine-translation-on-wmt2014-french-english', 'machine-translation-on-wmt2014-english-german', 'unsupervised-machine-translation-on-wmt2014-2', 'machine-translation-on-wmt2014-english-french'],
    'WNLI': ['natural-language-inference-on-wnli'],
    'WSC': ['coreference-resolution-on-winograd-schema'],
    'WebQuestions': ['question-answering-on-webquestions'],
    'WebQuestionsSP': ['knowledge-base-question-answering-on-1', 'semantic-parsing-on-webquestionssp'],
    'WenetSpeech': ['speech-recognition-on-wenetspeech'],
    'WikiText-103': ['language-modelling-on-wikitext-103'],
    'WikiText-2': ['language-modelling-on-wikitext-2'],
    'WinoGrande': ['common-sense-reasoning-on-winogrande'],
    'Winoground': ['visual-reasoning-on-winoground'],
    'XCOPA': ['cross-lingual-transfer-on-xcopa'],
    'XSUM': ['text-summarization-on-x-sum'],
    'YouCook2': ['video-retrieval-on-youcook2', 'video-captioning-on-youcook2', 'zero-shot-video-retrieval-on-youcook2'],
}

# PWC leaderboards are retrieved in https://production-media.paperswithcode.com/about/evaluation-tables.json.gz
dataset_oriented_leaderboards = ['https://zilliz.com/vector-database-benchmark-tool', 'https://www.datacomp.ai/dclm/leaderboard.html', 'https://www.datacomp.ai/dcclip/leaderboard.html', 'https://llm-eval.github.io/pages/leaderboard/advprompt.html#attack-results-view-by-datasets', 'https://huggingface.co/spaces/zhaorui-nb/de-identification-leaderboard']
metric_oriented_leaderboards = ['https://github.com/yuh-zha/AlignScore']
method_oriented_leaderboards = ['https://robustbench.github.io', 'https://teddy-xionggz.github.io/MIRAGE', 'https://www.swebench.com', 'https://jykoh.com/vwa', 'https://huggingface.co/spaces/osunlp/TravelPlannerLeaderboard', 'https://huggingface.co/spaces/JetBrains-Research/long-code-arena', 'https://huggingface.co/spaces/OPTML-Group/UnlearnCanvas-Benchmark', 'https://bird-bench.github.io', 'https://huggingface.co/spaces/stabletoolbench/Stable_Tool_Bench_Leaderboard', 'https://huggingface.co/spaces/GPTEval3D/Leaderboard_dev', 'https://huggingface.co/spaces/boyiwei/CoTaEval_leaderboard', 'https://huggingface.co/spaces/locuslab/tofu_leaderboard', 'https://huggingface.co/spaces/Intel/UnlearnDiffAtk-Benchmark', 'https://github.com/svilupp/Julia-LLM-Leaderboard?tab=readme-ov-file#prompting-templates', 'https://llm-eval.github.io/pages/leaderboard/pe.html', 'https://llm-eval.github.io/pages/leaderboard/advprompt.html#all-results-of-prompts', 'https://huggingface.co/spaces/autogenCTF/agent_ctf_leaderboard']
solution_oriented_leaderboards = ['https://www.datafountain.cn/competitions', 'https://www.drivendata.org/competitions', 'https://tianchi.aliyun.com/competition', 'https://www.codabench.org/competitions/public', 'https://eu.aihub.ml/competitions', 'https://aistudio.baidu.com/competition', 'https://www.hilti-challenge.com', 'https://insightface.ai/challenges', 'https://www.aicrowd.com/challenges', 'https://grand-challenge.org/challenges', 'https://dynabench.org/tasks', 'https://www.kaggle.com/competitions', 'https://eval.ai/web/challenges/list', 'https://taostats.io', 'https://artificialanalysis.ai']

In [3]:
platform_abbreviation_mapping_inverse = {v: k for k, v in platform_abbreviation_mapping.items()}

metrics_synonyms_inverse = {}
for key, values in metrics_synonyms.items():
    for value in values:
        metrics_synonyms_inverse[value] = key

publisher_synonyms_inverse = {}
for key, values in publisher_synonyms.items():
    for value in values:
        publisher_synonyms_inverse[value] = key

ranking_dataframe_organization_tactics_mapping_inverse = {}
for key, values in ranking_dataframe_organization_tactics_mapping.items():
    for value in values:
        ranking_dataframe_organization_tactics_mapping_inverse[value] = key

def convert_df_to_dict(df):
    """
    Convert a DataFrame of leaderboards and metrics to a dictionary.

    :param df: DataFrame with leaderboards and metrics
    :return: Dictionary with leaderboards as keys and list of metrics as values
    """
    leaderboard_data = {}

    # Ensure the DataFrame's first column is considered as the keys
    # and iterate through each row to construct the dictionary
    for _, row in df.iterrows():
        key = row[0]  # The first column serves as the key
        # List comprehension to get non-empty values from the rest of the row
        values = [value for value in row[1:] if pd.notna(value)]
        leaderboard_data[key] = values

    return leaderboard_data

def string_to_list(text, platform=False):
    if pd.isna(text) or text == 'nan':
        return []
    elif platform:
        return [platform_abbreviation_mapping[platform] for platform in text.split(',')]
    else:
        return text.split(',')

def string_to_dict(s, platform=False, list_format=False):
    # Check if the input is np.nan
    if pd.isna(s):
        if list_format:
            return []
        return {}
    
    # Split the string into key-value pairs
    pairs = s.split(',')
    result_dict = {}
    result_list = []
    
    for pair in pairs:
        # Split each pair by ':' to separate keys and values
        key, value = pair.split(':')
        # Split the value by '+' to get the list of items
        value_list = value.split('+')
        # Assign the list to the key in the dictionary
        if platform:
            result_dict[platform_abbreviation_mapping[key]] = value_list
        else:
            result_dict[key] = value_list
        result_list.extend(value_list)
            
    if list_format:
        return result_list
    return result_dict

# Function to format each value based on its own decimal places
def format_individual_value(x):
    if isinstance(x, float):
        # Use string formatting to maintain original decimal places
        return "{:f}".format(x).rstrip('0').rstrip('.')
    return x

def print_empty_folders(root_dir):
    for dirpath, dirname, filenames in os.walk(root_dir):
        # Check if both lists of subdirectories and filenames are empty
        if not filenames and dirpath != root_dir:
            print(f"Empty folder: {dirpath}")
            
def list_directories(folder_path):
    # List all subdirectories within the folder
    directories = [os.path.join(folder_path, d) for d in os.listdir(
        folder_path) if os.path.isdir(os.path.join(folder_path, d))]
    return directories

def publisher_synonyms_mapping(publishers):
    publishers_processed = set()
    for publisher in publishers:
        if publisher in publisher_synonyms_inverse:
            publishers_processed.add(publisher_synonyms_inverse[publisher])
        # elif 'Independent Contributor' == publisher:
        #     publishers_processed.add(leaderboard)
        else:
            publishers_processed.add(publisher)
    return publishers_processed

def keep_rows_by_list_column(df, column_name, keyword_list):
    """
    Filters rows in a DataFrame based on whether all elements in a specified list column are in a given keyword list.
    
    Parameters:
    - df: Pandas DataFrame.
    - column_name: The name of the column containing lists of strings.
    - keyword_list: A list of keywords to check against.
    
    Returns:
    - A filtered Pandas DataFrame.
    """
    # Convert the keyword list to a set for faster membership testing
    keyword_set = set(keyword_list)
    
    # Define a lambda function to check if all elements of a list are in the keyword_set
    all_in_keywords = lambda x: any(element in keyword_set for element in x)
    
    # Apply the lambda function to the specified column and invert the boolean Series to filter rows
    filtered_df = df[df[column_name].apply(all_in_keywords)]
    
    return filtered_df

def filter_rows_by_list_column(df, column_name, keyword_list):
    """
    Filters rows in a DataFrame based on whether all elements in a specified list column are in a given keyword list.
    
    Parameters:
    - df: Pandas DataFrame.
    - column_name: The name of the column containing lists of strings.
    - keyword_list: A list of keywords to check against.
    
    Returns:
    - A filtered Pandas DataFrame.
    """
    # Convert the keyword list to a set for faster membership testing
    keyword_set = set(keyword_list)
    
    # Define a lambda function to check if all elements of a list are in the keyword_set
    all_in_keywords = lambda x: not all(element in keyword_set for element in x)
    
    # Apply the lambda function to the specified column and invert the boolean Series to filter rows
    filtered_df = df[df[column_name].apply(all_in_keywords)]
    
    return filtered_df

In [8]:
import subprocess

# Load the environment variables from key.json
with open(f'{path_key}/.sourcegraph.json') as f:
    env_vars = json.load(f)

# Set the environment variables
os.environ.update(env_vars)

# Execute the shell script and pass path_data as an argument
try:
    subprocess.run(['sh', f'{path_data}/Dependents_Lookup.sh', path_data], check=True)
    print("Script executed successfully.")
except subprocess.CalledProcessError as e:
    print(f"Script execution failed with error: {e}")


Script executed successfully.


In [11]:
import json

with open(f"{path_data}/GitHub_old.json") as f:
    data_old = json.load(f)

with open(f"{path_data}/GitHub.json") as f:
    data_new = json.load(f)

repos_old = [repo["name"] for index, repo in enumerate(data_old["Results"]) if index > 5249]
# repos_old = [repo["name"] for repo in data_old["Results"]]
repos_new = [repo["name"] for repo in data_new["Results"]]

set(repos_old) - set(repos_new)

{'github.com/AliyunContainerService/alicloud-storage-provisioner',
 'github.com/AshwinAmbal/DarkWeb-Crawling-Indexing',
 'github.com/FanshuoZeng/Simignore',
 'github.com/JusperLee/LibriSpace',
 'github.com/KiloProjects/Kilonova',
 'github.com/Rohith-JN/type-frontend',
 'github.com/ShenQingchuan/github-contributor-svg-generator',
 'github.com/SnowCutieOwO/Continue',
 'github.com/TheBevyFlock/bevy_quickstart',
 'github.com/jamesroutley/news.routley.io',
 'github.com/lamini-ai/lamini-examples',
 'github.com/lbenz730/NCAA_Hoops_Play_By_Play',
 'github.com/marianoguerra/future-of-coding-weekly',
 'github.com/max-lee-dev/hackertype',
 'github.com/maxmakesmagic/ormos',
 'github.com/optuna/optuna',
 'github.com/physionetchallenges/physionetchallenges.github.io',
 'github.com/sgtest/monorepo-test-1',
 'github.com/sgtest/monorepo-test-3',
 'github.com/sgtest/monorepo-test-5',
 'github.com/sgtest/monorepo-test-7',
 'github.com/sgtest/monorepo-test-8',
 'github.com/vante-dev/Vante-Bots'}

In [12]:
import json
import webbrowser

with open(f"{path_data}/GitHub_old.json") as f:
    data = json.load(f)

for index, repo in enumerate(data["Results"]):
    if index <= 5249:
        continue
    webbrowser.open(f'https://{repo["name"]}')
    if index % 50 == 49:
        print(index)
        code = input("Press enter key to proceed.")
        if code.lower() == 'esc':
            raise KeyboardInterrupt

5299
5349
5399
5449
5499
5549
5599
5649
5699
5749
5799
5849
5899
5949
5999
6049
6099
6149
6199
6249
6299
6349
6399
6449
6499


In [ ]:
import json
import webbrowser

with open(f"{path_data}/GitHub.json") as f:
    data = json.load(f)

for index, repo in enumerate(data["Results"]):
    webbrowser.open(f'https://{repo["name"]}')
    if index % 50 == 49:
        print(index)
        code = input("Press enter key to proceed.")
        if code.lower() == 'esc':
            raise KeyboardInterrupt

In [ ]:
github_leaderboard_primary = [
    {
        'leaderboard': 'https://github.com/ray-project/llmperf-leaderboard',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/vztu/BVQA_Benchmark',
        'reference': 'https://github.com/vztu/RAPIQUE/blob/e637a3d943010940f67ef7b0c19e2f2c81d63b01/README.md?plain=1#L8'
    },
    {
        'leaderboard': 'https://csbench.github.io/#leaderboard',
        'reference': ''
    },
    {
        'leaderboard': 'https://crowdbenchmark.com/nwpucrowd.html',
        'reference': 'https://github.com/gjy3035/NWPU-Crowd-Sample-Code/blob/fe8e986d23e88f7b9defabbd0b3f6f655803152d/README.md?plain=1#L92'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/Decentralised-AI/falcon-40b/blob/6e61c89591cc7a3328d4010ceaf701dfec03fd18/README.md?plain=1#L183'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/instance-segmentation-on-coco',
        'reference': 'https://github.com/IDEA-Research/DINO/blob/d84a491d41898b3befd8294d1cf2614661fc0953/README.md?plain=1#L35'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/panoptic-segmentation-on-coco-test-dev',
        'reference': 'https://github.com/IDEA-Research/DINO/blob/d84a491d41898b3befd8294d1cf2614661fc0953/README.md?plain=1#L35'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/semantic-segmentation-on-ade20k',
        'reference': 'https://github.com/IDEA-Research/DINO/blob/d84a491d41898b3befd8294d1cf2614661fc0953/README.md?plain=1#L35'
    },
    {
        'leaderboard': 'https://github.com/kaz-Anova/ensemble_amazon',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/MLBazaar/BTB',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/Codium-ai/AlphaCodium',
        'reference': ''
    },
    {
        'leaderboard': 'https://www.nuscenes.org/object-detection',
        'reference': 'https://github.com/VDIGPKU/HENet/blob/main/README.md?plain=1#L13'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/bigcode/bigcode-models-leaderboard',
        'reference': 'https://github.com/continuedev/what-llm-to-use/blob/main/README.md?plain=1#L39'
    },
    {
        'leaderboard': 'https://wilds.stanford.edu/leaderboard',
        'reference': 'https://github.com/LFhase/PAIR/blob/main/README.md?plain=1#L19'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/dsdanielpark/open-llm-datasets/blob/main/README.md?plain=1#L228'
    },
    {
        'leaderboard': 'https://github.com/niais/Awesome-Skeleton-based-Action-Recognition',
        'reference': ''
    },
    {
        'leaderboard': 'https://eval.ai/web/challenges/challenge-page/2210/leaderboard',
        'reference': 'https://github.com/yanconglin/ICP-Flow/blob/main/README.md?plain=1#L90'
    },
    {
        'leaderboard': 'https://rajpurkar.github.io/SQuAD-explorer',
        'reference': 'https://github.com/zihangdai/xlnet/blob/master/README.md?plain=1#L35'
    },
    {
        'leaderboard': 'https://cmedbenchmark.llmzoo.com/static/leaderboard.html',
        'reference': 'https://github.com/X-D-Lab/Sunsimiao/blob/master/README.md?plain=1#L82'
    },
    {
        'leaderboard': 'https://ogb.stanford.edu/docs/lsc/leaderboards',
        'reference': 'https://github.com/lmqfly/Geometry-Deep-Learning-for-Drug-Discovery/blob/main/README.md?plain=1#L104'
    },
    {
        'leaderboard': 'https://eval.ai/web/challenges/challenge-page/1832/leaderboard',
        'reference': 'https://github.com/goel-shashank/CyCLIP/blob/main/README.md?plain=1#L82'
    },
    {
        'leaderboard': 'https://www.cvlibs.net/datasets/kitti/eval_object.php',
        'reference': 'https://github.com/fregu856/3DOD_thesis/blob/master/README.md?plain=1#L616'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/ryoungj/ObsScaling/blob/main/README.md?plain=1#L19'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/adithya-s-k/indic_eval/blob/main/README.md?plain=1#L544'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/Cognitive-Lab/indic_llm_leaderboard',
        'reference': 'https://github.com/adithya-s-k/indic_eval/blob/main/README.md?plain=1#L12'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/arjunbansal/awesome-oss-llm-ift-rlhf/blob/main/README.md?plain=1#L51'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/lmsys/chatbot-arena-leaderboard',
        'reference': 'https://github.com/arjunbansal/awesome-oss-llm-ift-rlhf/blob/main/README.md?plain=1#L52'
    },
    {
        'leaderboard': 'https://orionlab.space.noa.gr/mesogeos',
        'reference': 'https://github.com/Orion-AI-Lab/mesogeos/blob/main/README.md?plain=1#L161'
    },
    {
        'leaderboard': 'https://www.kaggle.com/c/allstate-claims-severity/leaderboard',
        'reference': 'https://github.com/adornes/spark_python_ml_examples/blob/master/README.md?plain=1#L262'
    },
    {
        'leaderboard': 'https://github.com/salesforce/WikiSQL',
        'reference': ''
    },
    {
        'leaderboard': 'https://codalab.lisn.upsaclay.fr/competitions/17640',
        'reference': 'https://github.com/caiyuanhao1998/Retinexformer/blob/master/README.md?plain=1#L111'
    },
    {
        'leaderboard': 'https://eval.ai/web/challenges/challenge-page/356/leaderboard',
        'reference': 'https://github.com/MCG-NJU/SparseBEV/blob/main/README.md?plain=1#L22'
    },
    {
        'leaderboard': 'https://github.com/longvideobench/LongVideoBench',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/crowdAI/crowdai-musical-genre-recognition-starter-kit',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/DetectionTeamUCAS/RRPN_Faster-RCNN_Tensorflow',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/OpenGVLab/MM-NIAH',
        'reference': ''
    },
    {
        'leaderboard': 'https://leaderboard.allenai.org/nycc-matching',
        'reference': 'https://github.com/jmhessel/caption_contest_corpus/blob/main/README.md?plain=1#L109'
    },
    {
        'leaderboard': 'https://microsoft.github.io/MSMARCO-Passage-Ranking-Submissions/leaderboard',
        'reference': 'https://github.com/AlibabaResearch/HLATR/blob/main/readme.md?plain=1#L12'
    },
    {
        'leaderboard': 'https://dki-lab.github.io/GrailQA',
        'reference': 'https://github.com/dki-lab/GrailQA/blob/main/README.md?plain=1#L11'
    },
    {
        'leaderboard': 'https://github.com/SilongYong/SQA3D',
        'reference': ''
    },
    {
        'leaderboard': 'http://host.robots.ox.ac.uk:8080/leaderboard/displaylb.php?challengeid=11&compid=6',
        'reference': 'https://github.com/kjw0612/awesome-deep-vision/blob/master/README.md?plain=1#L171'
    },
    {
        'leaderboard': 'https://github.com/cathyxl/MAgIC',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/JasonForJoy/Leaderboards-for-Multi-Turn-Response-Selection',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/openai/gym/wiki/Leaderboard',
        'reference': 'https://github.com/pat-coady/trpo/blob/master/README.md?plain=1#L9'
    },
    {
        'leaderboard': 'https://paperswithcode.com/dataset/pascalvoc-sp',
        'reference': 'https://github.com/vijaydwivedi75/lrgb/blob/main/README.md?plain=1#L111'
    },
    {
        'leaderboard': 'http://host.robots.ox.ac.uk:8080/leaderboard/displaylb.php?challengeid=11&compid=3',
        'reference': 'https://github.com/SHI-Labs/GFR-DSOD/blob/master/README.md?plain=1#L6'
    },
    {
        'leaderboard': 'https://github.com/OpenMOSS/HalluQA',
        'reference': ''
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/Spico197/Humback/blob/main/README.md?plain=1#L122'
    },
    {
        'leaderboard': 'https://lmarena.ai/?leaderboard',
        'reference': 'https://github.com/ikaijua/Awesome-AITools/blob/main/README.md?plain=1#L109'
    },
    {
        'leaderboard': 'https://artificialanalysis.ai/leaderboards/models',
        'reference': 'https://github.com/ikaijua/Awesome-AITools/blob/main/README.md?plain=1#L110'
    },
    {
        'leaderboard': 'https://artificialanalysis.ai/leaderboards/providers',
        'reference': 'https://github.com/ikaijua/Awesome-AITools/blob/main/README.md?plain=1#L110'
    },
    {
        'leaderboard': 'https://artificialanalysis.ai/text-to-image',
        'reference': 'https://github.com/ikaijua/Awesome-AITools/blob/main/README.md?plain=1#L110'
    },
    {
        'leaderboard': 'https://artificialanalysis.ai/speech-to-text',
        'reference': 'https://github.com/ikaijua/Awesome-AITools/blob/main/README.md?plain=1#L110'
    },
    {
        'leaderboard': 'https://artificialanalysis.ai/text-to-speech',
        'reference': 'https://github.com/ikaijua/Awesome-AITools/blob/main/README.md?plain=1#L110'
    },
    {
        'leaderboard': 'https://mcgill-nlp.github.io/bias-bench',
        'reference': 'https://github.com/McGill-NLP/bias-bench/blob/main/README.md?plain=1#L9'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/visual-question-answering-on-mm-vet',
        'reference': 'https://github.com/yuweihao/MM-Vet/blob/main/README.md?plain=1#L30'
    },
    {
        'leaderboard': 'https://github.com/THUDM/AgentBench',
        'reference': ''
    },
    {
        'leaderboard': 'https://leaderboard.allenai.org/winogrande',
        'reference': 'https://github.com/allenai/winogrande/blob/master/README.md?plain=1#L21'
    },
    {
        'leaderboard': 'https://dynabench.org/tasks',
        'reference': 'https://github.com/eugenesiow/practical-ml/blob/master/README.md?plain=1#L78'
    },
    {
        'leaderboard': 'https://codalab.lisn.upsaclay.fr/competitions/9304',
        'reference': 'https://github.com/huzeyann/MemoryEncodingModel/blob/main/README.md?plain=1#L15'
    },
    {
        'leaderboard': 'https://box.vicos.si/borja/viamaro/index.html#leaderboard',
        'reference': 'https://github.com/bborja/modd/blob/master/README.md?plain=1#L150'
    },
    {
        'leaderboard': 'https://www.kaggle.com/c/ntut-ml-2018-computer-vision/leaderboard',
        'reference': 'https://github.com/hsuRush/DeepANPR/blob/master/README.md?plain=1#L2'
    },
    {
        'leaderboard': 'https://github.com/XuanwuAI/SecEval',
        'reference': ''
    },
    {
        'leaderboard': 'https://few-shot.yyliu.net/miniimagenet.html',
        'reference': 'https://github.com/indussky8/awesome-few-shot-learning/blob/main/README.md?plain=1#L279'
    },
    {
        'leaderboard': 'https://github.com/iarai/NeurIPS2022-traffic4cast',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/cvgroup-njust/CityPersons',
        'reference': ''
    },
    {
        'leaderboard': 'https://ymcui.com/cmrc2019',
        'reference': 'https://github.com/ymcui/cmrc2019/blob/master/README.md?plain=1#L22'
    },
    {
        'leaderboard': 'https://www.kaggle.com/c/fungi-challenge-fgvc-2018/leaderboard',
        'reference': 'https://github.com/visipedia/fgvcx_fungi_comp/blob/master/README.md?plain=1#L10'
    },
    {
        'leaderboard': 'https://eval.ai/web/challenges/challenge-page/1091/leaderboard',
        'reference': 'https://github.com/Raldir/FEVEROUS/blob/main/README.md?plain=1#L171'
    },
    {
        'leaderboard': 'https://rajpurkar.github.io/SQuAD-explorer',
        'reference': 'https://github.com/google-research/bert/blob/master/README.md?plain=1#L194'
    },
    {
        'leaderboard': 'https://www.kaggle.com/c/siim-isic-melanoma-classification/leaderboard',
        'reference': 'https://github.com/haqishen/SIIM-ISIC-Melanoma-Classification-1st-Place-Solution/blob/master/README.md?plain=1#L3'
    },
    {
        'leaderboard': 'https://cocodataset.org/#stuff-leaderboard',
        'reference': 'https://github.com/nightrome/cocostuff/blob/master/README.md?plain=1#L102'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/WeOpenML/PandaLM/blob/main/README.md?plain=1#L52'
    },
    {
        'leaderboard': 'https://github.com/tianyi-lab/HallusionBench',
        'reference': ''
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/uclaml/SPIN/blob/main/README.md?plain=1#L247'
    },
    {
        'leaderboard': 'https://github.com/mlfoundations/dclm',
        'reference': ''
    },
    {
        'leaderboard': 'https://www.boreas.utias.utoronto.ca/#/leaderboard',
        'reference': 'https://github.com/utiasASRL/hero_radar_odometry/blob/master/README.md?plain=1#L7'
    },
    {
        'leaderboard': 'https://codalab.lisn.upsaclay.fr/competitions/880',
        'reference': 'https://github.com/isaaccorley/dfc2022-baseline/blob/master/README.md?plain=1#L34'
    },
    {
        'leaderboard': 'https://haitianliu22.github.io/code-scope-benchmark/leaderboard.html',
        'reference': 'https://github.com/WeixiangYAN/CodeScope/blob/main/README.md?plain=1#L6'
    },
    {
        'leaderboard': 'https://github.com/databricks/databricks-ml-examples',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/OpenBMB/OlympiadBench',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/AIR-Bench/AIR-Bench',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/decis-bench/febench',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/decis-bench/febench',
        'reference': ''
    },
    {
        'leaderboard': 'https://www.cvlibs.net/datasets/kitti/eval_object.php',
        'reference': 'https://github.com/dashidhy/awesome-point-cloud-deep-learning/blob/master/README.md?plain=1#L60'
    },
    {
        'leaderboard': 'https://www.nuscenes.org/object-detection',
        'reference': 'https://github.com/dashidhy/awesome-point-cloud-deep-learning/blob/master/README.md?plain=1#L61'
    },
    {
        'leaderboard': 'https://github.com/DetectionTeamUCAS/R2CNN-Plus-Plus_Tensorflow',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/diffbot/knowledge-net',
        'reference': ''
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/mlabonne/llm-course/blob/main/README.md?plain=1#L254'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/lmsys/chatbot-arena-leaderboard',
        'reference': 'https://github.com/mlabonne/llm-course/blob/main/README.md?plain=1#L262'
    },
    {
        'leaderboard': 'https://kgqa.github.io/leaderboard',
        'reference': 'https://github.com/KGQA/leaderboard'
    },
    {
        'leaderboard': 'https://github.com/open-compass/T-Eval',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/KevinLiao159/Quora',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/inspire-group/patch-defense-leaderboard',
        'reference': 'https://github.com/inspire-group/adv-patch-paper-list/blob/main/README.md?plain=1#L7'
    },
    {
        'leaderboard': 'https://github.com/dynamic-superb/dynamic-superb',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/yizhen20133868/Awesome-TOD-NLG-Survey',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/oshizo/japanese-llm-roleplay-benchmark',
        'reference': ''
    },
    {
        'leaderboard': 'https://waymo.com/open/challenges/2023/motion-prediction',
        'reference': 'https://github.com/zhejz/TrafficBots/blob/main/README.md?plain=1#L51'
    },
    {
        'leaderboard': 'https://paperswithcode.com/dataset/burst',
        'reference': 'https://github.com/Ali2500/BURST-benchmark/blob/main/README.md?plain=1#L112'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/video-super-resolution-on-vid4-4x-upscaling',
        'reference': 'https://github.com/amanchadha/iSeeBetter/blob/master/README.md?plain=1#L19'
    },
    {
        'leaderboard': 'https://github.com/princeton-nlp/LM-Science-Tutor',
        'reference': ''
    },
    {
        'leaderboard': 'https://gluebenchmark.com/leaderboard',
        'reference': 'https://github.com/mwaskom/seaborn-data/blob/master/README.md?plain=1#L26'
    },
    {
        'leaderboard': 'https://lmarena.ai/?leaderboard',
        'reference': 'https://github.com/lm-sys/FastChat/blob/main/README.md?plain=1#L6'
    },
    {
        'leaderboard': 'https://github.com/google-research/meta-dataset',
        'reference': ''
    },
    {
        'leaderboard': 'https://stanfordmlgroup.github.io/competitions/chexpert',
        'reference': 'https://github.com/stanfordmlgroup/MoCo-CXR/blob/main/README.md?plain=1#L71'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/relation-extraction-on-redocred',
        'reference': 'https://github.com/tonytan48/KD-DocRE/blob/main/README.md?plain=1#L8'
    },
    {
        'leaderboard': 'https://eval.ai/web/challenges/challenge-page/1881/leaderboard',
        'reference': 'https://github.com/project-miracl/miracl/blob/main/README.md?plain=1#L23'
    },
    {
        'leaderboard': 'https://fudanselab-classeval.github.io/leaderboard.html',
        'reference': 'https://github.com/FudanSELab/ClassEval/blob/master/README.md?plain=1#L5'
    },
    {
        'leaderboard': 'http://noisylabels.com',
        'reference': 'https://github.com/weijiaheng/Advances-in-Label-Noise-Learning/blob/main/README.md?plain=1#L63'
    },
    {
        'leaderboard': 'https://paperswithcode.com/dataset/animal',
        'reference': 'https://github.com/weijiaheng/Advances-in-Label-Noise-Learning/blob/main/README.md?plain=1#L67'
    },
    {
        'leaderboard': 'https://paperswithcode.com/dataset/clothing1m',
        'reference': 'https://github.com/weijiaheng/Advances-in-Label-Noise-Learning/blob/main/README.md?plain=1#L69'
    },
    {
        'leaderboard': 'https://yale-lily.github.io/sparc',
        'reference': 'https://github.com/taoyds/sparc/blob/master/README.md?plain=1#L3'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/machine-translation-on-wmt2014-german-english',
        'reference': 'https://github.com/joeynmt/joeynmt/blob/main/README.md?plain=1#L280'
    },
    {
        'leaderboard': 'https://github.com/hendrycks/robustness',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/huguyuehuhu/Awesome-Group-Activity-Recognition',
        'reference': ''
    },
    {
        'leaderboard': 'https://mmcheng.net/videosal',
        'reference': 'https://github.com/MichiganCOG/TASED-Net/blob/master/README.md?plain=1#L7'
    },
    {
        'leaderboard': 'https://pmc-patients.github.io',
        'reference': 'https://github.com/pmc-patients/pmc-patients/blob/main/README.md?plain=1#L104'
    },
    {
        'leaderboard': 'https://github.com/Xiangyu-CAS/AICity2020-VOC-ReID',
        'reference': ''
    },
    {
        'leaderboard': 'https://agora-evaluation.is.tuebingen.mpg.de',
        'reference': 'https://github.com/IDEA-Research/OSX/blob/main/README.md?plain=1#L41'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/robust-camera-only-3d-object-detection-on',
        'reference': 'https://github.com/Daniel-xsy/RoboBEV/blob/master/README.md?plain=1#L90'
    },
    {
        'leaderboard': 'https://waymo.com/open/challenges',
        'reference': 'https://github.com/Nightmare-n/GD-MAE/blob/main/README.md?plain=1#L12'
    },
    {
        'leaderboard': 'https://github.com/GAIR-NLP/auto-j',
        'reference': ''
    },
    {
        'leaderboard': 'https://zjysteven.github.io/OpenOOD/#leaderboard',
        'reference': 'https://github.com/Jingkang50/OpenOOD/blob/main/README.md?plain=1#L57'
    },
    {
        'leaderboard': 'https://webots.cloud/competition',
        'reference': 'https://github.com/cyberbotics/wrestling/blob/main/README.md?plain=1#L79'
    },
    {
        'leaderboard': 'https://github.com/TellinaTool/nl2bash',
        'reference': ''
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota',
        'reference': 'https://github.com/paperswithcode/releasing-research-code/blob/master/README.md?plain=1#L94'
    },
    {
        'leaderboard': 'https://eval.ai/web/challenges/list',
        'reference': 'https://github.com/paperswithcode/releasing-research-code/blob/master/README.md?plain=1#L96'
    },
    {
        'leaderboard': 'https://nlpprogress.com',
        'reference': 'https://github.com/paperswithcode/releasing-research-code/blob/master/README.md?plain=1#L97'
    },
    {
        'leaderboard': 'https://github.com/SUFE-AIFLM-Lab/FinEval',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/FuxiaoLiu/LRV-Instruction',
        'reference': ''
    },
    {
        'leaderboard': 'https://scandeval.com',
        'reference': 'https://github.com/BramVanroy/fietje-2/blob/main/README.md?plain=1#L32'
    },
    {
        'leaderboard': 'https://github.com/JasonForJoy/Leaderboards-for-Multi-Turn-Response-Selection',
        'reference': 'https://github.com/chauff/conversationalIR/blob/master/README.md?plain=1#L75'
    },
    {
        'leaderboard': 'https://paperswithcode.com/task/conversational-response-selection',
        'reference': 'https://github.com/chauff/conversationalIR/blob/master/README.md?plain=1#L77'
    },
    {
        'leaderboard': 'https://multi-trust.github.io/#leaderboard',
        'reference': 'https://github.com/thu-ml/MMTrustEval/blob/main/README.md?plain=1#L11'
    },
    {
        'leaderboard': 'https://www.cluebenchmarks.com/nli.html',
        'reference': 'https://github.com/CLUEbenchmark/OCNLI/blob/main/README.md?plain=1#L59'
    },
    {
        'leaderboard': 'https://github.com/princeton-nlp/CharXiv',
        'reference': ''
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/TIGER-Lab/LongICL-Leaderboard',
        'reference': 'https://github.com/TIGER-AI-Lab/LongICLBench/blob/main/README.md?plain=1#L6'
    },
    {
        'leaderboard': 'https://github.com/CLUEbenchmark/CLUE',
        'reference': 'https://github.com/shibing624/pytextclassifier/blob/master/README.md?plain=1#L549'
    },
    {
        'leaderboard': 'https://leaderboard.allenai.org/drop',
        'reference': 'https://github.com/llamazing/numnet_plus/blob/master/README.md?plain=1#L3'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/MLGroupJLU/LLM-eval-survey/blob/main/README.md?plain=1#L343'
    },
    {
        'leaderboard': 'https://agora-evaluation.is.tuebingen.mpg.de',
        'reference': 'https://github.com/ttxskk/AiOS/blob/main/README.md?plain=1#L288'
    },
    {
        'leaderboard': 'https://www.nuscenes.org/object-detection',
        'reference': 'https://github.com/HorizonRobotics/Sparse4D/blob/main/README.md?plain=1#L59'
    },
    {
        'leaderboard': 'https://eval.ai/web/challenges/challenge-page/163/leaderboard',
        'reference': 'https://github.com/airsplay/lxmert/blob/master/README.md?plain=1#L132'
    },
    {
        'leaderboard': 'https://eval.ai/web/challenges/challenge-page/225/leaderboard',
        'reference': 'https://github.com/airsplay/lxmert/blob/master/README.md?plain=1#L216'
    },
    {
        'leaderboard': 'https://eval.ai/web/challenges/challenge-page/225/leaderboard',
        'reference': 'https://github.com/airsplay/lxmert/blob/master/README.md?plain=1#L216'
    },
    {
        'leaderboard': 'https://lil.nlp.cornell.edu/nlvr',
        'reference': 'https://github.com/airsplay/lxmert/blob/master/README.md?plain=1#L271'
    },
    {
        'leaderboard': 'https://evalplus.github.io/leaderboard.html',
        'reference': 'https://github.com/ise-uiuc/magicoder/blob/main/README.md?plain=1#L38'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/mteb/leaderboard',
        'reference': 'https://github.com/qdrant/fastembed/blob/main/README.md?plain=1#L5'
    },
    {
        'leaderboard': 'https://leaderboard.tabbyml.com',
        'reference': 'https://github.com/deepseek-ai/awesome-deepseek-coder/blob/main/README.md?plain=1#L49'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/merveenoyan/awesome-osml-for-devs/blob/main/README.md?plain=1#L130'
    },
    {
        'leaderboard': 'https://mixeval.github.io/#leaderboard',
        'reference': 'https://github.com/Psycoy/MixEval/blob/main/README.md?plain=1#L4'
    },
    {
        'leaderboard': 'https://github.com/BAAI-DCAI/Multimodal-Robustness-Benchmark',
        'reference': ''
    },
    {
        'leaderboard': 'https://vlue-benchmark.github.io/leaderboard.html',
        'reference': 'https://github.com/MichaelZhouwang/VLUE/blob/main/README.md?plain=1#L6'
    },
    {
        'leaderboard': 'https://www.nuscenes.org/object-detection',
        'reference': 'https://github.com/hht1996ok/EA-LSS/blob/main/README.md?plain=1#L6'
    },
    {
        'leaderboard': 'https://github.com/ZaloAI-Jaist/VMLU',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/hkust-nlp/ceval',
        'reference': 'https://github.com/NicholasCao/Awesome-Chinese-ChatGPT/blob/main/README.md?plain=1#L24'
    },
    {
        'leaderboard': 'https://lmarena.ai/?leaderboard',
        'reference': 'https://github.com/NicholasCao/Awesome-Chinese-ChatGPT/blob/main/README.md?plain=1#L25'
    },
    {
        'leaderboard': 'https://www.superclueai.com',
        'reference': 'https://github.com/NicholasCao/Awesome-Chinese-ChatGPT/blob/main/README.md?plain=1#L26'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/NicholasCao/Awesome-Chinese-ChatGPT/blob/main/README.md?plain=1#L27'
    },
    {
        'leaderboard': 'https://github.com/wangxw5/wikiDiverse',
        'reference': ''
    },
    {
        'leaderboard': 'https://csgaobb.github.io/Projects/mscoco-fsod.html',
        'reference': 'https://github.com/gaobb/Few-Shot-Object-Detection-Papers/blob/master/README.md?plain=1#L7'
    },
    {
        'leaderboard': 'https://csgaobb.github.io/Projects/pascalvoc-fsod.html',
        'reference': 'https://github.com/gaobb/Few-Shot-Object-Detection-Papers/blob/master/README.md?plain=1#L7'
    },
    {
        'leaderboard': 'https://superbbenchmark.org/leaderboard',
        'reference': 'https://github.com/sungnyun/ARMHuBERT/blob/master/README.md?plain=1#L26'
    },
    {
        'leaderboard': 'https://github.com/THUDM/LongBench',
        'reference': 'https://github.com/DAMO-NLP-SG/CLEX/blob/main/README.md?plain=1#L162'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/lmsys/chatbot-arena-leaderboard',
        'reference': 'https://github.com/leobeeson/llm_benchmarks/blob/master/README.md?plain=1#L316'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/long-context-understanding-on-mmneedle',
        'reference': 'https://github.com/Wang-ML-Lab/multimodal-needle-in-a-haystack/blob/main/README.md?plain=1#L14'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/unsupervised-domain-adaptation-on-ucf-hmdb',
        'reference': 'https://github.com/ldkong1205/TranSVAE/blob/main/README.md?plain=1#L67'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/unsupervised-domain-adaptation-on-jester',
        'reference': 'https://github.com/ldkong1205/TranSVAE/blob/main/README.md?plain=1#L67'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/unsupervised-domain-adaptation-on-epic',
        'reference': 'https://github.com/ldkong1205/TranSVAE/blob/main/README.md?plain=1#L67'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/chenking2020/FindTheChatGPTer/blob/main/README.md?plain=1#L539'
    },
    {
        'leaderboard': 'https://cevalbenchmark.com/static/leaderboard.html',
        'reference': 'https://github.com/chenking2020/FindTheChatGPTer/blob/main/README.md?plain=1#L549'
    },
    {
        'leaderboard': 'https://leaderboard.allenai.org/mslr-ms2',
        'reference': 'https://github.com/allenai/ms2/blob/master/README.md?plain=1#L3'
    },
    {
        'leaderboard': 'https://github.com/insikk/CBIR_LeaderBoard',
        'reference': 'https://github.com/keloli/CBIR_EXPLORE/blob/master/README.md?plain=1#L6'
    },
    {
        'leaderboard': 'https://github.com/opstower-ai/devops-ai-open-leaderboard',
        'reference': ''
    },
    {
        'leaderboard': 'https://www.cvlibs.net/datasets/kitti',
        'reference': 'https://github.com/SeasonDepth/SeasonDepth/blob/master/README.md?plain=1#L2'
    },
    {
        'leaderboard': 'https://github.com/haonan-li/CMMLU',
        'reference': 'https://github.com/yangjianxin1/Firefly-LLaMA2-Chinese/blob/master/README.MD?plain=1#L112'
    },
    {
        'leaderboard': 'https://open-compass.github.io/T-Eval/leaderboard.html',
        'reference': 'https://github.com/tjtanaa/awesome-large-action-model/blob/main/README.md?plain=1#L20'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/bigcode/bigcode-models-leaderboard',
        'reference': 'https://github.com/codefuse-ai/MFTCoder/blob/main/README.md?plain=1#L55'
    },
    {
        'leaderboard': 'http://host.robots.ox.ac.uk:8080/leaderboard/displaylb.php?challengeid=11&compid=4',
        'reference': 'https://github.com/Smorodov/Deep-learning-object-detection-links./blob/master/readme.md?plain=1#L31'
    },
    {
        'leaderboard': 'https://github.com/talk2car/Talk2Car',
        'reference': ''
    },
    {
        'leaderboard': 'https://atlas-challenge.u-bourgogne.fr/leaderboard',
        'reference': 'https://github.com/uni-medical/STU-Net/blob/main/README.md?plain=1#L11'
    },
    {
        'leaderboard': 'https://sppin.grand-challenge.org/evaluation/final-test-phase/leaderboard',
        'reference': 'https://github.com/uni-medical/STU-Net/blob/main/README.md?plain=1#L12'
    },
    {
        'leaderboard': 'https://autopet-ii.grand-challenge.org/leaderboard',
        'reference': 'https://github.com/uni-medical/STU-Net/blob/main/README.md?plain=1#L13'
    },
    {
        'leaderboard': 'https://codalab.lisn.upsaclay.fr/competitions/12239',
        'reference': 'https://github.com/uni-medical/STU-Net/blob/main/README.md?plain=1#L14'
    },
    {
        'leaderboard': 'https://github.com/hkust-nlp/ceval',
        'reference': ''
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/eduagarcia/open_pt_llm_leaderboard',
        'reference': 'https://github.com/ruanchaves/napolab/blob/main/README.md?plain=1#L16'
    },
    {
        'leaderboard': 'https://github.com/Instance-Search/Instance-Search',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/LudwigStumpp/llm-leaderboard',
        'reference': ''
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/bigcode/bigcode-models-leaderboard',
        'reference': 'https://github.com/bigcode-project/bigcode-evaluation-harness/blob/leaderboard/leaderboard/README.md?plain=1#L11'
    },
    {
        'leaderboard': 'https://github.com/awslabs/diagnostic-robustness-text-to-sql',
        'reference': ''
    },
    {
        'leaderboard': 'https://openlamm.github.io/ch3ef/leaderboard.html',
        'reference': 'https://github.com/OpenGVLab/LAMM/blob/main/README.md?plain=1#L15'
    },
    {
        'leaderboard': 'https://visualcommonsense.com/leaderboard',
        'reference': 'https://github.com/ict-bigdatalab/awesome-pretrained-models-for-information-retrieval/blob/main/README.md?plain=1#L513'
    },
    {
        'leaderboard': 'https://stanfordmlgroup.github.io/competitions/mrnet',
        'reference': 'https://github.com/MisaOgura/MRNet/blob/master/README.md?plain=1#L278'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/GPT-Alternatives/gpt_alternatives/blob/main/README.md?plain=1#L271'
    },
    {
        'leaderboard': 'https://github.com/salesforce/localization-xml-mt',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/ldkong1205/PointCloud-C',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/AsuradaYuci/awesome_video_person_reid/blob/master/leaderboard.md',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/THUDM/ChatGLM-Math',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/LMMMEng/LLD-MMRI2023/blob/main/assets/test_leaderboard.md',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/jianguoz/Few-Shot-Intent-Detection',
        'reference': ''
    },
    {
        'leaderboard': 'https://eval.ai/web/challenges/challenge-page/1256/leaderboard',
        'reference': 'https://github.com/neurallatents/nlb_tools/blob/main/README.md?plain=1#L35'
    },
    {
        'leaderboard': 'https://www.aicrowd.com/challenges/music-demixing-challenge-ismir-2021/leaderboards',
        'reference': 'https://github.com/AIcrowd/music-demixing-challenge-starter-kit/blob/master/README.md?plain=1#L194'
    },
    {
        'leaderboard': 'https://yale-lily.github.io/spider',
        'reference': 'https://github.com/taoyds/spider/blob/master/README.md?plain=1#L3'
    },
    {
        'leaderboard': 'https://www.nist.gov/programs-projects/face-technology-evaluations-frtefate',
        'reference': 'https://github.com/becauseofAI/awesome-face/blob/master/README.md?plain=1#L465'
    },
    {
        'leaderboard': 'https://comma.ai/leaderboard',
        'reference': 'https://github.com/commaai/controls_challenge/blob/master/README.md?plain=1#L8'
    },
    {
        'leaderboard': 'https://leaderboard.allenai.org/sherlock',
        'reference': 'https://github.com/allenai/sherlock/blob/main/README.md?plain=1#L22'
    },
    {
        'leaderboard': 'https://github.com/michuanhaohao/AICITY2021_Track2_DMT',
        'reference': ''
    },
    {
        'leaderboard': 'https://opendrivelab.com/challenge2023',
        'reference': 'https://github.com/er-muyue/BeMapNet/blob/main/README.md?plain=1#L11'
    },
    {
        'leaderboard': 'http://host.robots.ox.ac.uk/leaderboard/displaylb.php?challengeid=11',
        'reference': 'https://github.com/openaifab/hair/blob/master/README.md?plain=1#L62'
    },
    {
        'leaderboard': 'https://paperswithcode.com/task/image-super-resolution',
        'reference': 'https://github.com/isaaccorley/pytorch-enhance/blob/master/README.md?plain=1#L55'
    },
    {
        'leaderboard': 'http://host.robots.ox.ac.uk:8080/leaderboard/displaylb.php?challengeid=11&compid=6',
        'reference': 'https://github.com/autonomousdrivingkr/Awesome-Autonomous-Driving/blob/master/README.md?plain=1#L143'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/instructkr/ko-chatbot-arena-leaderboard',
        'reference': 'https://github.com/HeegyuKim/open-korean-instructions/blob/main/README.md?plain=1#L73'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/instructkr/LogicKor-leaderboard',
        'reference': 'https://github.com/HeegyuKim/open-korean-instructions/blob/main/README.md?plain=1#L74'
    },
    {
        'leaderboard': 'https://lil.nlp.cornell.edu/nlvr',
        'reference': 'https://github.com/lil-lab/nlvr/blob/master/README.md?plain=1#L7'
    },
    {
        'leaderboard': 'https://github.com/cardiffnlp/tweeteval',
        'reference': ''
    },
    {
        'leaderboard': 'https://nealcly.github.io/MuTual-leaderboard',
        'reference': 'https://github.com/Nealcly/MuTual/blob/master/README.md?plain=1#L65'
    },
    {
        'leaderboard': 'https://www.kaggle.com/c/imaterialist-challenge-furniture-2018/leaderboard',
        'reference': 'https://github.com/skrypka/imaterialist-furniture-2018/blob/master/README.md?plain=1#L3'
    },
    {
        'leaderboard': 'https://www.nuscenes.org/tracking',
        'reference': 'https://github.com/dvlab-research/VoxelNeXt/blob/master/README.md?plain=1#L21'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/instance-segmentation-on-coco-minival',
        'reference': 'https://github.com/IDEA-Research/DN-DETR/blob/main/README.md?plain=1#L22'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/panoptic-segmentation-on-coco-minival',
        'reference': 'https://github.com/IDEA-Research/DN-DETR/blob/main/README.md?plain=1#L22'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/semantic-segmentation-on-ade20k',
        'reference': 'https://github.com/IDEA-Research/DN-DETR/blob/main/README.md?plain=1#L22'
    },
    {
        'leaderboard': 'https://few-shot.yyliu.net/miniimagenet.html',
        'reference': 'https://github.com/icoz69/DeepEMD/blob/master/README.md?plain=1#L13'
    },
    {
        'leaderboard': 'https://insightface.ai/mfr21',
        'reference': 'https://github.com/deepinsight/insightface/blob/master/README.md?plain=1#L70'
    },
    {
        'leaderboard': 'https://codalab.lisn.upsaclay.fr/competitions/12599',
        'reference': 'https://github.com/dongkyunk/CheXFusion/blob/main/README.md?plain=1#L27'
    },
    {
        'leaderboard': 'https://github.com/openai/gym/wiki/Leaderboard',
        'reference': 'https://github.com/dgriff777/rl_a3c_pytorch/blob/master/README.MD?plain=1#L22'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/semi-supervised-semantic-segmentation-on-25',
        'reference': 'https://github.com/ldkong1205/LaserMix/blob/main/README.md?plain=1#L73'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/semi-supervised-semantic-segmentation-on-24',
        'reference': 'https://github.com/ldkong1205/LaserMix/blob/main/README.md?plain=1#L73'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/semi-supervised-semantic-segmentation-on-23',
        'reference': 'https://github.com/ldkong1205/LaserMix/blob/main/README.md?plain=1#L73'
    },
    {
        'leaderboard': 'https://paperswithcode.com/paper/lasermix-for-semi-supervised-lidar-semantic',
        'reference': 'https://github.com/ldkong1205/LaserMix/blob/main/README.md?plain=1#L73'
    },
    {
        'leaderboard': 'https://github.com/Coder-Yu/SELFRec',
        'reference': ''
    },
    {
        'leaderboard': 'https://arctic-leaderboard.is.tuebingen.mpg.de/leaderboard',
        'reference': 'https://github.com/zc-alexfan/arctic/blob/master/README.md?plain=1#L44'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/text-to-sql-on-kaggledbqa',
        'reference': 'https://github.com/Chia-Hsuan-Lee/KaggleDBQA/blob/main/README.md?plain=1#L4'
    },
    {
        'leaderboard': 'http://ccl.pku.edu.cn:8084/SpaCE2021/rank',
        'reference': 'https://github.com/2030NLP/SpaCE2021/blob/main/README.md?plain=1#L98'
    },
    {
        'leaderboard': 'http://ccl.pku.edu.cn:8084/SpaCE2021/rank',
        'reference': 'https://github.com/z-fabian/HUMUS-Net/blob/master/README.md?plain=1#L22'
    },
    {
        'leaderboard': 'https://github.com/MinghuiChen43/CIL-ReID',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/pddasig/Machine-Learning-Competition-2020',
        'reference': ''
    },
    {
        'leaderboard': 'https://yale-lily.github.io/spider',
        'reference': 'https://github.com/BeachWang/DAIL-SQL/blob/main/README.md?plain=1#L4'
    },
    {
        'leaderboard': 'http://got-10k.aitestunion.com/leaderboard',
        'reference': 'https://github.com/wangdongdut/Online-Visual-Tracking-SOTA/blob/master/README.md?plain=1#L116'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/Pan-ML/panml/blob/main/README.md?plain=1#L25'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/Slep/LRVSF-Leaderboard',
        'reference': 'https://github.com/Simon-Lepage/CondViT-LRVSF/blob/main/README.md?plain=1#L28'
    },
    {
        'leaderboard': 'https://mmbench.opencompass.org.cn/leaderboard',
        'reference': 'https://github.com/open-compass/MMBench/blob/main/README.md?plain=1#L26'
    },
    {
        'leaderboard': 'https://github.com/THUDM/VisualAgentBench',
        'reference': ''
    },
    {
        'leaderboard': 'https://www.kaggle.com/c/nyu-cv-fall-2018/leaderboard',
        'reference': 'https://github.com/poojahira/gtsrb-pytorch/blob/master/README.md?plain=1#L5'
    },
    {
        'leaderboard': 'https://github.com/lupantech/PromptPG',
        'reference': ''
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/vidore/vidore-leaderboard',
        'reference': 'https://github.com/illuin-tech/vidore-benchmark/blob/main/README.md?plain=1#L14'
    },
    {
        'leaderboard': 'https://www.kaggle.com/competitions/mlsp-2013-birds/leaderboard',
        'reference': 'https://github.com/gaborfodor/MLSP_2013/blob/master/readme.md?plain=1#L42'
    },
    {
        'leaderboard': 'https://github.com/shjung13/Standardized-max-logits',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/k2-fsa/libriheavy',
        'reference': ''
    },
    {
        'leaderboard': 'http://got-10k.aitestunion.com/leaderboard',
        'reference': 'https://github.com/huanglianghua/siamrpn-pytorch/blob/master/README.md?plain=1#L13'
    },
    {
        'leaderboard': 'https://github.com/Xtra-Computing/NIID-Bench',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/brohrer/parameter_efficiency_leaderboard',
        'reference': ''
    },
    {
        'leaderboard': 'https://eval.ai/web/challenges/challenge-page/1247/leaderboard',
        'reference': 'https://github.com/SoonminHwang/rgbt-ped-detection/blob/master/README.md?plain=1#L4'
    },
    {
        'leaderboard': 'https://visualcommonsense.com/leaderboard',
        'reference': 'https://github.com/zhegan27/VILLA/blob/main/README.md?plain=1#L130'
    },
    {
        'leaderboard': 'https://microsoft.github.io/AirSim-NeurIPS2019-Drone-Racing/leaderboard_final.html',
        'reference': 'https://github.com/microsoft/AirSim-NeurIPS2019-Drone-Racing/blob/master/README.md?plain=1#L206'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/BestWishYsh/ChronoMagic-Bench',
        'reference': 'https://github.com/yhZhai/mcm/blob/main/README.md?plain=1#L19'
    },
    {
        'leaderboard': 'https://github.com/hemingkx/Spec-Bench/blob/main/Leaderboard.md',
        'reference': ''
    },
    {
        'leaderboard': 'https://open-xai.github.io/leaderboard',
        'reference': 'https://github.com/AI4LIFE-GROUP/OpenXAI/blob/main/README.md?plain=1#L20'
    },
    {
        'leaderboard': 'https://rank.opencompass.org.cn/leaderboard-llm',
        'reference': 'https://github.com/TommyZihao/ChatMarx/blob/main/README.md?plain=1#L129'
    },
    {
        'leaderboard': 'https://eval.ai/web/challenges/challenge-page/454/leaderboard',
        'reference': 'https://github.com/ZikangZhou/QCNet/blob/main/README.md?plain=1#L9'
    },
    {
        'leaderboard': 'https://eval.ai/web/challenges/challenge-page/1719/leaderboard',
        'reference': 'https://github.com/ZikangZhou/QCNet/blob/main/README.md?plain=1#L10'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/change-point-detection-on-tep',
        'reference': 'https://github.com/YKatser/CPDE/blob/master/README.md?plain=1#L6'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/change-point-detection-on-skab',
        'reference': 'https://github.com/YKatser/CPDE/blob/master/README.md?plain=1#L22'
    },
    {
        'leaderboard': 'https://github.com/ai-forever/digital_peter_aij2020',
        'reference': ''
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/echo840/ocrbench-leaderboard',
        'reference': 'https://github.com/Yuliang-Liu/MultimodalOCR/blob/main/README.md?plain=1#L6'
    },
    {
        'leaderboard': 'https://leaderboard.sea-lion.ai',
        'reference': 'https://github.com/aisingapore/sealion/blob/main/README.md?plain=1#L81'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/image-harmonization-on-iharmony4',
        'reference': 'https://github.com/bcmi/Awesome-Image-Harmonization/blob/master/README.md?plain=1#L35'
    },
    {
        'leaderboard': 'https://challenge.isic-archive.com/landing/2018',
        'reference': 'https://github.com/ngessert/isic2018/blob/master/README.md?plain=1#L4'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/chujiezheng/LLM-Extrapolation/blob/main/README.md?plain=1#L78'
    },
    {
        'leaderboard': 'https://tatsu-lab.github.io/alpaca_eval',
        'reference': 'https://github.com/chujiezheng/LLM-Extrapolation/blob/main/README.md?plain=1#L80'
    },
    {
        'leaderboard': 'https://aider.chat/docs/leaderboards',
        'reference': 'https://github.com/chujiezheng/LLM-Extrapolation/blob/main/README.md?plain=1#L80'
    },
    {
        'leaderboard': 'https://opening-up-chatgpt.github.io',
        'reference': 'https://github.com/opening-up-chatgpt/opening-up-chatgpt.github.io/blob/main/readme.md?plain=1#L1'
    },
    {
        'leaderboard': 'https://leaderboard.allenai.org/visualcomet',
        'reference': 'https://github.com/jamespark3922/visual-comet/blob/master/README.md?plain=1#L153'
    },
    {
        'leaderboard': 'https://github.com/StarStyleSky/awesome-face-detection',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/junqiangchen/LiTS---Liver-Tumor-Segmentation-Challenge',
        'reference': ''
    },
    {
        'leaderboard': 'https://tgb.complexdatalab.com/docs/leader_linkprop',
        'reference': 'https://github.com/shenyangHuang/TGB/blob/main/README.md?plain=1#L27'
    },
    {
        'leaderboard': 'https://github.com/vztu/BVQA_Benchmark',
        'reference': ''
    },
    {
        'leaderboard': 'https://rrc.cvc.uab.es',
        'reference': 'https://github.com/EriCongMa/awesome-transformer-ocr/blob/main/README.md?plain=1#L229'
    },
    {
        'leaderboard': 'https://github.com/open-compass/BotChat',
        'reference': ''
    },
    {
        'leaderboard': 'https://trustllmbenchmark.github.io/TrustLLM-Website/leaderboard.html',
        'reference': 'https://github.com/jphall663/awesome-machine-learning-interpretability/blob/master/README.md?plain=1#L1158'
    },
    {
        'leaderboard': 'https://www.aicrowd.com/challenges/music-demixing-challenge-ismir-2021/leaderboards',
        'reference': 'https://github.com/yoyololicon/music-demixing-challenge-ismir-2021-entry/blob/master/README.md?plain=1#L1'
    },
    {
        'leaderboard': 'https://leaderboard.allenai.org/a-okvqa',
        'reference': 'https://github.com/allenai/aokvqa/blob/main/README.md?plain=1#L5'
    },
    {
        'leaderboard': 'https://www.aicrowd.com/challenges/seismic-facies-identification-challenge/leaderboards',
        'reference': 'https://github.com/AIcrowd/seismic-facies-identification-starter-kit/blob/master/README.md?plain=1#L8'
    },
    {
        'leaderboard': 'https://github.com/haoweiz23/Awesome-Fine-grained-Visual-Classification',
        'reference': ''
    },
    {
        'leaderboard': 'https://hfl-rc.github.io/cmrc2017/leaderboard',
        'reference': 'https://github.com/ymcui/Chinese-RC-Datasets/blob/master/README.md?plain=1#L63'
    },
    {
        'leaderboard': 'https://aistudio.baidu.com/competition/detail/720/0/leaderboard',
        'reference': 'https://github.com/ymcui/Chinese-RC-Datasets/blob/master/README.md?plain=1#L84'
    },
    {
        'leaderboard': 'https://hfl-rc.github.io/cmrc2018/open_challenge',
        'reference': 'https://github.com/ymcui/Chinese-RC-Datasets/blob/master/README.md?plain=1#L100'
    },
    {
        'leaderboard': 'https://www.kaggle.com/c/landmark-recognition-2020/leaderboard',
        'reference': 'https://github.com/bestfitting/instance_level_recognition/blob/main/README.md?plain=1#L5'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/baichuan-inc/Baichuan-7B/blob/main/README.md?plain=1#L128'
    },
    {
        'leaderboard': 'https://github.com/google-research-datasets/seahorse',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/netease-youdao/BCEmbedding',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/yxgeee/MMT-plus',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/infinigence/LVEval',
        'reference': ''
    },
    {
        'leaderboard': 'https://stanfordnlp.github.io/coqa',
        'reference': 'https://github.com/stevezheng23/xlnet_extension_tf/blob/master/README.md?plain=1#L2'
    },
    {
        'leaderboard': 'https://quac.ai',
        'reference': 'https://github.com/stevezheng23/xlnet_extension_tf/blob/master/README.md?plain=1#L4'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/mteb/leaderboard',
        'reference': 'https://github.com/CoIR-team/coir/blob/main/README.md?plain=1#L24'
    },
    {
        'leaderboard': 'https://archersama.github.io/coir',
        'reference': 'https://github.com/CoIR-team/coir/blob/main/README.md?plain=1#L24'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/Xwin-LM/Xwin-LM/blob/main/README.md?plain=1#L83'
    },
    {
        'leaderboard': 'https://www.aicrowd.com/challenges/food-recognition-benchmark-2022/leaderboards',
        'reference': 'https://github.com/AIcrowd/food-recognition-benchmark-starter-kit/blob/master/README.md?plain=1#L193'
    },
    {
        'leaderboard': 'https://leaderboard.carla.org/leaderboard',
        'reference': 'https://github.com/carla-simulator/scenario_runner/blob/master/README.md?plain=1#L49'
    },
    {
        'leaderboard': 'https://rajpurkar.github.io/SQuAD-explorer',
        'reference': 'https://github.com/Nagakiran1/Extending-Google-BERT-as-Question-and-Answering-model-and-Chatbot/blob/master/README.md?plain=1#L422'
    },
    {
        'leaderboard': 'https://github.com/yanyanSann/Long-Tailed-Classification-Leaderboard',
        'reference': 'https://github.com/Keke921/GCLLoss/blob/main/README.md?plain=1#L65'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/yongzhuo/qwen2-sft/blob/master/README.md?plain=1#L106'
    },
    {
        'leaderboard': 'https://eval.ai/web/challenges/challenge-page/103/leaderboard',
        'reference': 'https://github.com/batra-mlp-lab/visdial/blob/master/README.md?plain=1#L293'
    },
    {
        'leaderboard': 'https://github.com/FormulaMonks/llm-benchmarker-suite',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/inspire-group/patch-defense-leaderboard',
        'reference': 'https://github.com/inspire-group/PatchGuard/blob/master/README.md?plain=1#L9'
    },
    {
        'leaderboard': 'https://github.com/open-mmlab/OpenUnReID/blob/master/docs/LEADERBOARD.md',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/declare-lab/instruct-eval',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/VITA-Group/Deep_GCN_Benchmarking',
        'reference': ''
    },
    {
        'leaderboard': 'https://dki-lab.github.io/GrailQA',
        'reference': 'https://github.com/microsoft/vert-papers/blob/master/README.md?plain=1#L16'
    },
    {
        'leaderboard': 'https://mathverse-cuhk.github.io/#leaderboard',
        'reference': 'https://github.com/ZrrSkywalker/MAVIS/blob/main/README.md?plain=1#L10'
    },
    {
        'leaderboard': 'https://eval.ai/web/challenges/challenge-page/244/leaderboard',
        'reference': 'https://github.com/xinke-wang/Awesome-Text-VQA/blob/master/README.md?plain=1#L108'
    },
    {
        'leaderboard': 'https://eval.ai/web/challenges/challenge-page/551/leaderboard',
        'reference': 'https://github.com/xinke-wang/Awesome-Text-VQA/blob/master/README.md?plain=1#L109'
    },
    {
        'leaderboard': 'https://rrc.cvc.uab.es/?ch=11&com=evaluation&task=1',
        'reference': 'https://github.com/xinke-wang/Awesome-Text-VQA/blob/master/README.md?plain=1#L129'
    },
    {
        'leaderboard': 'https://paperswithcode.com/dataset/popgym',
        'reference': 'https://github.com/proroklab/popgym/blob/master/README.md?plain=1#L76'
    },
    {
        'leaderboard': 'https://taolusi.github.io/CSpider-explorer',
        'reference': 'https://github.com/taolusi/chisp/blob/master/README.md?plain=1#L7'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/mlabonne/llm-autoeval/blob/master/README.md?plain=1#L40'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/mlabonne/Yet_Another_LLM_Leaderboard',
        'reference': 'https://github.com/mlabonne/llm-autoeval/blob/master/README.md?plain=1#L67'
    },
    {
        'leaderboard': 'https://gluebenchmark.com/leaderboard',
        'reference': 'https://github.com/charles9n/bert-sklearn/blob/master/README.md?plain=1#L105'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/MIBlue119/awesome-llama-resources/blob/main/README.md?plain=1#L106'
    },
    {
        'leaderboard': 'https://ml.energy/leaderboard',
        'reference': 'https://github.com/AmberLJC/LLMSys-PaperList/blob/main/README.md?plain=1#L158'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/optimum/llm-perf-leaderboard',
        'reference': 'https://github.com/AmberLJC/LLMSys-PaperList/blob/main/README.md?plain=1#L159'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/AmberLJC/LLMSys-PaperList/blob/main/README.md?plain=1#L161'
    },
    {
        'leaderboard': 'https://crfm.stanford.edu/helm',
        'reference': 'https://github.com/AmberLJC/LLMSys-PaperList/blob/main/README.md?plain=1#L162'
    },
    {
        'leaderboard': 'https://lmarena.ai/?leaderboard',
        'reference': 'https://github.com/AmberLJC/LLMSys-PaperList/blob/main/README.md?plain=1#L163'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/openlifescienceai/open_medical_llm_leaderboard',
        'reference': 'https://github.com/AI-in-Health/MedLLMsPracticalGuide/blob/main/README.md?plain=1#L269'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/ymcui/Chinese-LLaMA-Alpaca-3/blob/main/README.md?plain=1#L240'
    },
    {
        'leaderboard': 'https://github.com/RPC-Dataset/RPC-Leaderboard',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/alibaba/AICITY2024_Track2_AliOpenTrek_CityLLaVA',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/THUDM/SciGLM',
        'reference': ''
    },
    {
        'leaderboard': 'https://www.hilti-challenge.com',
        'reference': 'https://github.com/Hilti-Research/hilti-slam-challenge-2021/blob/master/README.md?plain=1#L18'
    },
    {
        'leaderboard': 'https://github.com/thu-coai/Safety-Prompts',
        'reference': ''
    },
    {
        'leaderboard': 'https://opendrivelab.com/challenge2023',
        'reference': 'https://github.com/wenjie710/PivotNet/blob/main/README.md?plain=1#L8'
    },
    {
        'leaderboard': 'https://www.crowdbenchmark.com/nwpucrowdloc.html',
        'reference': 'https://github.com/taohan10200/Awesome-Crowd-Localization/blob/main/README.md?plain=1#L88'
    },
    {
        'leaderboard': 'https://codalab.lisn.upsaclay.fr/competitions/17672',
        'reference': 'https://github.com/cage-challenge/cage-challenge-4/blob/main/README.md?plain=1#L187'
    },
    {
        'leaderboard': 'https://leaderboard.carla.org/leaderboard',
        'reference': 'https://github.com/erdos-project/pylot/blob/master/README.md?plain=1#L266'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/NexaAIDev/domain_llm_leaderboard',
        'reference': 'https://github.com/NexaAI/octopus-v4/blob/main/README.md?plain=1#L26'
    },
    {
        'leaderboard': 'https://github.com/shikras/d-cube/tree/main/eval_sota',
        'reference': 'https://github.com/Charles-Xie/awesome-described-object-detection/blob/main/README.md?plain=1#L42'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/opencompass/open_vlm_leaderboard',
        'reference': 'https://github.com/scenarios/WeMM/blob/main/README.md?plain=1#L5'
    },
    {
        'leaderboard': 'https://opencompass.org.cn/leaderboard-multimodal',
        'reference': 'https://github.com/scenarios/WeMM/blob/main/README.md?plain=1#L15'
    },
    {
        'leaderboard': 'https://github.com/GAIR-NLP/OpenResearcher',
        'reference': ''
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/lmsys/chatbot-arena-leaderboard',
        'reference': 'https://github.com/Hannibal046/Awesome-LLM/blob/main/README.md?plain=1#L132'
    },
    {
        'leaderboard': 'https://mixeval.github.io/#leaderboard',
        'reference': 'https://github.com/Hannibal046/Awesome-LLM/blob/main/README.md?plain=1#L133'
    },
    {
        'leaderboard': 'https://tatsu-lab.github.io/alpaca_eval',
        'reference': 'https://github.com/Hannibal046/Awesome-LLM/blob/main/README.md?plain=1#L134'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/Hannibal046/Awesome-LLM/blob/main/README.md?plain=1#L136'
    },
    {
        'leaderboard': 'https://rank.opencompass.org.cn/leaderboard-llm',
        'reference': 'https://github.com/Hannibal046/Awesome-LLM/blob/main/README.md?plain=1#L137'
    },
    {
        'leaderboard': 'https://gorilla.cs.berkeley.edu/leaderboard.html',
        'reference': 'https://github.com/Hannibal046/Awesome-LLM/blob/main/README.md?plain=1#L138'
    },
    {
        'leaderboard': 'https://github.com/svilupp/Julia-LLM-Leaderboard',
        'reference': 'https://github.com/svilupp/awesome-generative-ai-meets-julia-language/blob/main/README.md?plain=1#L95'
    },
    {
        'leaderboard': 'https://github.com/ethz-spylab/rlhf_trojan_competition',
        'reference': ''
    },
    {
        'leaderboard': 'https://eval.ai/web/challenges/challenge-page/463/leaderboard',
        'reference': 'https://github.com/zd11024/NaviLLM/blob/main/README.md?plain=1#L80'
    },
    {
        'leaderboard': 'https://eval.ai/web/challenges/challenge-page/1715/leaderboard',
        'reference': 'https://github.com/zd11024/NaviLLM/blob/main/README.md?plain=1#L80'
    },
    {
        'leaderboard': 'https://github.com/bcmi/Image-Harmonization-Dataset-iHarmony4',
        'reference': ''
    },
    {
        'leaderboard': 'https://www.nuscenes.org/object-detection',
        'reference': 'https://github.com/nv-tlabs/planning-centric-metrics/blob/master/README.md?plain=1#L26'
    },
    {
        'leaderboard': 'https://www.kaggle.com/c/global-wheat-detection/leaderboard',
        'reference': 'https://github.com/liaopeiyuan/TransferDet/blob/master/README.md?plain=1#L7'
    },
    {
        'leaderboard': 'https://github.com/adobe-research/deft_corpus',
        'reference': ''
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/yongzhuo/LLM-SFT/blob/main/README.md?plain=1#L71'
    },
    {
        'leaderboard': 'https://lv-mhp.github.io/human_parsing_lb',
        'reference': 'https://github.com/ZhaoJ9014/Multi-Human-Parsing/blob/master/README.md?plain=1#L71'
    },
    {
        'leaderboard': 'https://bop.felk.cvut.cz/leaderboards/pose-estimation-unseen-bop23/core-datasets',
        'reference': 'https://github.com/NVlabs/FoundationPose/blob/main/readme.md?plain=1#L16'
    },
    {
        'leaderboard': 'https://www.kaggle.com/c/seizure-prediction/leaderboard',
        'reference': 'https://github.com/april-org/april-ann/blob/master/README.md?plain=1#L299'
    },
    {
        'leaderboard': 'https://github.com/romainloiseau/HelixNet',
        'reference': ''
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/visual-object-tracking-on-lasot',
        'reference': 'https://github.com/Little-Podi/AiATrack/blob/main/README.md?plain=1#L25'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/visual-object-tracking-on-trackingnet',
        'reference': 'https://github.com/Little-Podi/AiATrack/blob/main/README.md?plain=1#L27'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/visual-object-tracking-on-got-10k',
        'reference': 'https://github.com/Little-Podi/AiATrack/blob/main/README.md?plain=1#L28'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/visual-object-tracking-on-needforspeed',
        'reference': 'https://github.com/Little-Podi/AiATrack/blob/main/README.md?plain=1#L29'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/visual-object-tracking-on-otb-100',
        'reference': 'https://github.com/Little-Podi/AiATrack/blob/main/README.md?plain=1#L30'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/visual-object-tracking-on-uav123',
        'reference': 'https://github.com/Little-Podi/AiATrack/blob/main/README.md?plain=1#L31'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/PetroIvaniuk/llms-tools/blob/main/README.md?plain=1#L112'
    },
    {
        'leaderboard': 'https://microsoft.github.io/GLUECoS',
        'reference': ''
    },
    {
        'leaderboard': 'https://hkust-nlp.github.io/agentboard',
        'reference': 'https://github.com/PetroIvaniuk/llms-tools/blob/main/README.md?plain=1#L174'
    },
    {
        'leaderboard': 'https://artificialanalysis.ai/text-to-image',
        'reference': 'https://github.com/PetroIvaniuk/llms-tools/blob/main/README.md?plain=1#L176'
    },
    {
        'leaderboard': 'https://scale.com/leaderboard',
        'reference': 'https://github.com/PetroIvaniuk/llms-tools/blob/main/README.md?plain=1#L177'
    },
    {
        'leaderboard': 'https://crfm.stanford.edu/helm',
        'reference': 'https://github.com/PetroIvaniuk/llms-tools/blob/main/README.md?plain=1#L178'
    },
    {
        'leaderboard': 'https://www.vals.ai',
        'reference': 'https://github.com/PetroIvaniuk/llms-tools/blob/main/README.md?plain=1#L179'
    },
    {
        'leaderboard': 'https://trustllmbenchmark.github.io/TrustLLM-Website/leaderboard.html',
        'reference': 'https://github.com/PetroIvaniuk/llms-tools/blob/main/README.md?plain=1#L180'
    },
    {
        'leaderboard': 'https://lmarena.ai/?leaderboard',
        'reference': 'https://github.com/PetroIvaniuk/llms-tools/blob/main/README.md?plain=1#L181'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/PetroIvaniuk/llms-tools/blob/main/README.md?plain=1#L182'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/optimum/llm-perf-leaderboard',
        'reference': 'https://github.com/PetroIvaniuk/llms-tools/blob/main/README.md?plain=1#L183'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/hallucinations-leaderboard/leaderboard',
        'reference': 'https://github.com/PetroIvaniuk/llms-tools/blob/main/README.md?plain=1#L184'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/NPHardEval/NPHardEval-leaderboard',
        'reference': 'https://github.com/PetroIvaniuk/llms-tools/blob/main/README.md?plain=1#L185'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/AI-Secure/llm-trustworthy-leaderboard',
        'reference': 'https://github.com/PetroIvaniuk/llms-tools/blob/main/README.md?plain=1#L186'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/openlifescienceai/open_medical_llm_leaderboard',
        'reference': 'https://github.com/PetroIvaniuk/llms-tools/blob/main/README.md?plain=1#L187'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/gaia-benchmark/leaderboard',
        'reference': 'https://github.com/PetroIvaniuk/llms-tools/blob/main/README.md?plain=1#L189'
    },
    {
        'leaderboard': 'https://superbbenchmark.org/leaderboard',
        'reference': 'https://github.com/mechanicalsea/lighthubert/blob/main/README.md?plain=1#L7'
    },
    {
        'leaderboard': 'https://github.com/PengYu-Team/Co-LRIO',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/dvlab-research/DSGN',
        'reference': ''
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/3d-lane-detection-on-openlane',
        'reference': 'https://github.com/OpenDriveLab/OpenLane/blob/main/README.md?plain=1#L24'
    },
    {
        'leaderboard': 'https://lmarena.ai/?leaderboard',
        'reference': 'https://github.com/shure-dev/Awesome-LLM-Papers-Comprehensive-Topics/blob/main/README.md?plain=1#L239'
    },
    {
        'leaderboard': 'https://github.com/LudwigStumpp/llm-leaderboard',
        'reference': 'https://github.com/shure-dev/Awesome-LLM-Papers-Comprehensive-Topics/blob/main/README.md?plain=1#L413'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/Abdullahw72/langchain-chatbot-multiple-PDF/blob/master/README.md?plain=1#L56'
    },
    {
        'leaderboard': 'https://leaderboard.allenai.org/propara',
        'reference': 'https://github.com/allenai/propara/blob/master/README.md?plain=1#L12'
    },
    {
        'leaderboard': 'https://leaderboard.allenai.org/scifact',
        'reference': 'https://github.com/allenai/scifact/blob/master/README.md?plain=1#L33'
    },
    {
        'leaderboard': 'https://hkust-nlp.github.io/agentboard',
        'reference': 'https://github.com/hkust-nlp/AgentBoard/blob/main/README.md?plain=1#L18'
    },
    {
        'leaderboard': 'https://adeval.cstcloud.cn/content/leaderboard',
        'reference': 'https://github.com/dawnvince/EasyTSAD/blob/main/README.md?plain=1#L27'
    },
    {
        'leaderboard': 'https://tatsu-lab.github.io/alpaca_eval',
        'reference': 'https://github.com/GeneZC/MiniMA/blob/main/README.md?plain=1#L58'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/GeneZC/MiniMA/blob/main/README.md?plain=1#L62'
    },
    {
        'leaderboard': 'https://github.com/deepchem/moleculenet',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/CLUEbenchmark/CLUE',
        'reference': ''
    },
    {
        'leaderboard': 'https://eval.ai/web/challenges/challenge-page/454/leaderboard',
        'reference': 'https://github.com/Henry1iu/TNT-Trajectory-Prediction/blob/main/README.md?plain=1#L25'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/thestriver/ai-for-javascript-course/blob/main/README.md?plain=1#L411'
    },
    {
        'leaderboard': 'https://tatsu-lab.github.io/alpaca_eval',
        'reference': 'https://github.com/thestriver/ai-for-javascript-course/blob/main/README.md?plain=1#L413'
    },
    {
        'leaderboard': 'https://www.cvlibs.net/datasets/kitti/eval_scene_flow.php',
        'reference': 'https://github.com/jiahaopang/crl/blob/master/README.md?plain=1#L28'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/visual-entailment-on-snli-ve-val',
        'reference': 'https://github.com/necla-ml/SNLI-VE/blob/master/README.md?plain=1#L26'
    },
    {
        'leaderboard': 'https://eval.ai/web/challenges/challenge-page/1938/leaderboard',
        'reference': 'https://github.com/buildingnet/buildingnet_dataset/blob/main/README.md?plain=1#L16'
    },
    {
        'leaderboard': 'https://eval.ai/web/challenges/challenge-page/1663/leaderboard',
        'reference': 'https://github.com/alexa/alexa-with-dstc9-track1-dataset/blob/master/README.md?plain=1#L13'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/mteb/leaderboard',
        'reference': 'https://github.com/skeskinen/llama-lite/blob/master/README.md?plain=1#L67'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/optimum/llm-perf-leaderboard',
        'reference': 'https://github.com/samuelrince/awesome-green-ai/blob/main/README.md?plain=1#L87'
    },
    {
        'leaderboard': 'https://ml.energy/leaderboard',
        'reference': 'https://github.com/samuelrince/awesome-green-ai/blob/main/README.md?plain=1#L88'
    },
    {
        'leaderboard': 'https://sites.google.com/view/pgdl2020/leaderboard',
        'reference': 'https://github.com/parthnatekar/pgdl/blob/master/README.md?plain=1#L7'
    },
    {
        'leaderboard': 'https://sites.google.com/view/pgdl2020/leaderboard',
        'reference': 'https://github.com/z-x-yang/AOT/blob/main/README.md?plain=1#L29'
    },
    {
        'leaderboard': 'https://github.com/ws-choi/Conditioned-Source-Separation-LaSAFT',
        'reference': ''
    },
    {
        'leaderboard': 'https://eu.aihub.ml/competitions/201',
        'reference': 'https://github.com/z-x-yang/AOT/blob/main/README.md?plain=1#L29'
    },
    {
        'leaderboard': 'https://eu.aihub.ml/competitions/201',
        'reference': 'https://github.com/z-x-yang/AOT/blob/main/README.md?plain=1#L31'
    },
    {
        'leaderboard': 'https://eval.ai/web/challenges/challenge-page/606/leaderboard',
        'reference': 'https://github.com/YuankaiQi/REVERIE/blob/master/README.md?plain=1#L8'
    },
    {
        'leaderboard': 'https://eval.ai/web/challenges/challenge-page/2049/leaderboard',
        'reference': 'https://github.com/eric-ai-lab/Aerial-Vision-and-Dialog-Navigation/blob/origin/README.md?plain=1#L16'
    },
    {
        'leaderboard': 'https://github.com/robot-pesg/BotanicGarden/tree/main/leaderboard',
        'reference': ''
    },
    {
        'leaderboard': 'https://www.cvlibs.net/datasets/kitti/eval_object.php?obj_benchmark=3d',
        'reference': 'https://github.com/sshaoshuai/PointRCNN/blob/master/README.md?plain=1#L18'
    },
    {
        'leaderboard': 'https://rrc.cvc.uab.es/?ch=17&com=evaluation&task=4',
        'reference': 'https://github.com/rubenpt91/MP-DocVQA-Framework/blob/master/readme.md?plain=1#L39'
    },
    {
        'leaderboard': 'https://rajpurkar.github.io/SQuAD-explorer',
        'reference': 'https://github.com/allenai/document-qa/blob/master/README.md?plain=1#L167'
    },
    {
        'leaderboard': 'https://github.com/pangeo-data/WeatherBench',
        'reference': ''
    },
    {
        'leaderboard': 'https://cares-ai.github.io/#leaderboard',
        'reference': 'https://github.com/richard-peng-xia/CARES/blob/main/README.md?plain=1#L12'
    },
    {
        'leaderboard': 'https://github.com/GAIR-NLP/benbench',
        'reference': ''
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/eduagarcia/open_pt_llm_leaderboard',
        'reference': 'https://github.com/ajdavidl/Portuguese-NLP/blob/main/README.md?plain=1#L237'
    },
    {
        'leaderboard': 'https://eval.ai/web/challenges/challenge-page/97/leaderboard',
        'reference': 'https://github.com/peteanderson80/Matterport3DSimulator/blob/master/README.md?plain=1#L50'
    },
    {
        'leaderboard': 'https://github.com/Master-PLC/FreDF',
        'reference': ''
    },
    {
        'leaderboard': 'https://lmarena.ai/?leaderboard',
        'reference': 'https://github.com/eugeneyan/open-llms/blob/main/README.md?plain=1#L109'
    },
    {
        'leaderboard': 'https://twitter.com/jefrankle/status/1654631746506301441',
        'reference': 'https://github.com/eugeneyan/open-llms/blob/main/README.md?plain=1#L110'
    },
    {
        'leaderboard': 'https://crfm.stanford.edu/helm',
        'reference': 'https://github.com/eugeneyan/open-llms/blob/main/README.md?plain=1#L111'
    },
    {
        'leaderboard': 'https://github.com/LudwigStumpp/llm-leaderboard',
        'reference': 'https://github.com/eugeneyan/open-llms/blob/main/README.md?plain=1#L112'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/eugeneyan/open-llms/blob/main/README.md?plain=1#L114'
    },
    {
        'leaderboard': 'https://github.com/lfy79001/TableQAKit',
        'reference': ''
    },
    {
        'leaderboard': 'https://leaderboard.allenai.org/open_book_qa',
        'reference': 'https://github.com/allenai/OpenBookQA/blob/main/README.md?plain=1#L16'
    },
    {
        'leaderboard': 'http://www.cvlibs.net/datasets/kitti/eval_tracking.php',
        'reference': 'https://github.com/aleksandrkim61/EagerMOT/blob/open_main/README.md?plain=1#L27'
    },
    {
        'leaderboard': 'https://www.nuscenes.org/tracking',
        'reference': 'https://github.com/aleksandrkim61/EagerMOT/blob/open_main/README.md?plain=1#L28'
    },
    {
        'leaderboard': 'https://tatsu-lab.github.io/alpaca_eval',
        'reference': 'https://github.com/thunlp/UltraChat/blob/main/README.md?plain=1#L35'
    },
    {
        'leaderboard': 'https://github.com/jiachens/ModelNet40-C',
        'reference': ''
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/mteb/leaderboard',
        'reference': 'https://github.com/Anush008/fastembed-go/blob/main/README.md?plain=1#L18'
    },
    {
        'leaderboard': 'https://eval.ai/web/challenges/challenge-page/830/leaderboard',
        'reference': 'https://github.com/microsoft/unilm/blob/master/README.md?plain=1#L139'
    },
    {
        'leaderboard': 'https://superbbenchmark.org/leaderboard',
        'reference': 'https://github.com/microsoft/unilm/blob/master/README.md?plain=1#L143'
    },
    {
        'leaderboard': 'https://sites.research.google/xtreme',
        'reference': 'https://github.com/microsoft/unilm/blob/master/README.md?plain=1#L146'
    },
    {
        'leaderboard': 'https://rrc.cvc.uab.es/?ch=17&com=evaluation&task=1',
        'reference': 'https://github.com/microsoft/unilm/blob/master/README.md?plain=1#L158'
    },
    {
        'leaderboard': 'https://rrc.cvc.uab.es/?ch=13&com=evaluation&task=3',
        'reference': 'https://github.com/microsoft/unilm/blob/master/README.md?plain=1#L158'
    },
    {
        'leaderboard': 'https://rajpurkar.github.io/SQuAD-explorer',
        'reference': 'https://github.com/duyunshu/bert-sentiment-analysis/blob/master/README.md?plain=1#L158'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/code-generation-on-humaneval',
        'reference': 'https://github.com/alekst23/simple-coder/blob/main/README.md?plain=1#L15'
    },
    {
        'leaderboard': 'https://hotpotqa.github.io',
        'reference': 'https://github.com/woshiyyya/DFGN-pytorch/blob/master/readme.md?plain=1#L15'
    },
    {
        'leaderboard': 'https://www.swebench.com',
        'reference': 'https://github.com/aorwall/moatless-tools/blob/main/README.md?plain=1#L5'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/HqWu-HITCS/Awesome-Chinese-LLM/blob/main/README.md?plain=1#L1067'
    },
    {
        'leaderboard': 'https://lvbench.github.io/#leaderboard',
        'reference': 'https://github.com/THUDM/LVBench/blob/main/README.md?plain=1#L11'
    },
    {
        'leaderboard': 'https://github.com/cbvrp-acmmm-2019/cbvrp-acmmm-2019',
        'reference': ''
    },
    {
        'leaderboard': 'https://www.kaggle.com/c/jigsaw-unintended-bias-in-toxicity-classification/leaderboard',
        'reference': 'https://github.com/thuwyh/Jigsaw-Unintended-Bias-in-Toxicity-Classification/blob/master/README.md?plain=1#L2'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/openlifescienceai/open_medical_llm_leaderboard',
        'reference': 'https://github.com/TsinghuaC3I/UltraMedical/blob/main/README.md?plain=1#L129'
    },
    {
        'leaderboard': 'https://leaderboard.carla.org/leaderboard',
        'reference': 'https://github.com/autonomousvision/carla_garage/blob/main/README.md?plain=1#L176'
    },
    {
        'leaderboard': 'https://github.com/yuh-zha/AlignScore',
        'reference': ''
    },
    {
        'leaderboard': 'https://cocodataset.org/#captions-leaderboard',
        'reference': 'https://github.com/peteanderson80/Up-Down-Captioner/blob/master/README.md?plain=1#L3'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/mike-ravkine/can-ai-code-results',
        'reference': 'https://github.com/the-crypt-keeper/can-ai-code/blob/main/README.md?plain=1#L37'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/allenai/WildBench',
        'reference': 'https://github.com/allenai/WildBench/blob/main/README.md?plain=1#L8'
    },
    {
        'leaderboard': 'https://paperswithcode.com/task/node-classification-on-non-homophilic',
        'reference': 'https://github.com/SitaoLuan/ACM-GNN/blob/main/README.md?plain=1#L129'
    },
    {
        'leaderboard': 'https://waymo.com/open/challenges/2020/3d-detection',
        'reference': 'https://github.com/PJLab-ADG/LoGoNet/blob/main/README.md?plain=1#L14'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/3d-depth-estimation-on-relative-human',
        'reference': 'https://github.com/Arthur151/Relative_Human/blob/master/README.md?plain=1#L27'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/mteb/leaderboard',
        'reference': 'https://github.com/infrawhispers/anansi/blob/main/README.md?plain=1#L30'
    },
    {
        'leaderboard': 'https://www.kaggle.com/c/google-quest-challenge/leaderboard',
        'reference': 'https://github.com/oleg-yaroshevskiy/quest_qa_labeling/blob/master/README.md?plain=1#L109'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/jincheng9/gpt-tutorial/blob/main/README.md?plain=1#L114'
    },
    {
        'leaderboard': 'https://www.aicrowd.com/challenges/unity-obstacle-tower-challenge/leaderboards',
        'reference': 'https://github.com/Unity-Technologies/obstacle-tower-source/blob/master/README.md?plain=1#L40'
    },
    {
        'leaderboard': 'https://www.aicrowd.com/challenges/neurips-2021-minerl-diamond-competition/leaderboards',
        'reference': 'https://github.com/minerllabs/competition_submission_template/blob/master/README.md?plain=1#L14'
    },
    {
        'leaderboard': 'http://host.robots.ox.ac.uk:8080/leaderboard/displaylb_dt.php?challengeid=6&compid=4',
        'reference': 'https://github.com/rbgirshick/rcnn/blob/master/README.md?plain=1#L39'
    },
    {
        'leaderboard': 'http://host.robots.ox.ac.uk:8080/leaderboard/displaylb_dt.php?challengeid=11&compid=4',
        'reference': 'https://github.com/rbgirshick/rcnn/blob/master/README.md?plain=1#40'
    },
    {
        'leaderboard': 'https://leaderboard.allenai.org/robothor_objectnav',
        'reference': 'https://github.com/allenai/robothor-challenge/blob/main/README.md?plain=1#L105'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/math-word-problem-solving-on-svamp',
        'reference': 'https://github.com/chuanyang-Zheng/Progressive-Hint/blob/main/README.md?plain=1#L32'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/arithmetic-reasoning-on-gsm8k',
        'reference': 'https://github.com/chuanyang-Zheng/Progressive-Hint/blob/main/README.md?plain=1#L32'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/math-word-problem-solving-on-math',
        'reference': 'https://github.com/chuanyang-Zheng/Progressive-Hint/blob/main/README.md?plain=1#L32'
    },
    {
        'leaderboard': 'https://mmbench.opencompass.org.cn/leaderboard',
        'reference': 'https://github.com/codefuse-ai/CodeFuse-MFT-VLM/blob/master/README.md?plain=1#L16'
    },
    {
        'leaderboard': 'https://github.com/codefuse-ai/codefuse-devops-eval',
        'reference': ''
    },
    {
        'leaderboard': 'https://www.kaggle.com/c/feedback-prize-2021/leaderboard',
        'reference': 'https://github.com/affjljoo3581/Feedback-Prize-Competition/blob/master/README.md?plain=1#L5'
    },
    {
        'leaderboard': 'https://github.com/AI-secure/Certified-Robustness-SoK-Oldver',
        'reference': ''
    },
    {
        'leaderboard': 'https://sokcertifiedrobustness.github.io/leaderboard',
        'reference': 'https://github.com/AI-secure/Certified-Robustness-SoK-Oldver/blob/master/README.md?plain=1#L1'
    },
    {
        'leaderboard': 'https://ml.energy/leaderboard',
        'reference': 'https://github.com/Green-Software-Foundation/awesome-green-software/blob/main/readme.md?plain=1#L125'
    },
    {
        'leaderboard': 'https://waymo.com/open/challenges/2020/3d-detection',
        'reference': 'https://github.com/ADLab-AutoDrive/INT/blob/main/README.md?plain=1#L37'
    },
    {
        'leaderboard': 'https://waymo.com/open/challenges/2023/motion-prediction',
        'reference': 'https://github.com/zhejz/HPTR/blob/main/README.md?plain=1#L26'
    },
    {
        'leaderboard': 'https://eval.ai/web/challenges/challenge-page/1719/leaderboard',
        'reference': 'https://github.com/zhejz/HPTR/blob/main/README.md?plain=1#L69'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/yentinglin/open-tw-llm-leaderboard',
        'reference': 'https://github.com/MiuLab/Taiwan-LLM/blob/main/README.md?plain=1#L77'
    },
    {
        'leaderboard': 'https://www.kaggle.com/c/home-credit-default-risk/leaderboard',
        'reference': 'https://github.com/KazukiOnodera/Home-Credit-Default-Risk/blob/master/README.md?plain=1#L2'
    },
    {
        'leaderboard': 'https://codalab.lisn.upsaclay.fr/competitions/4318',
        'reference': 'https://github.com/shreyashampali/kypt_transformer/blob/master/README.md?plain=1#L120'
    },
    {
        'leaderboard': 'https://codalab.lisn.upsaclay.fr/competitions/4897',
        'reference': 'https://github.com/shreyashampali/kypt_transformer/blob/master/README.md?plain=1#L138'
    },
    {
        'leaderboard': 'https://eval.ai/web/challenges/challenge-page/689/leaderboard',
        'reference': 'https://github.com/martiansideofthemoon/hurdles-longform-qa/blob/main/README.md?plain=1#L115'
    },
    {
        'leaderboard': 'https://codalab.lisn.upsaclay.fr/competitions/6738',
        'reference': 'https://github.com/psunlpgroup/MultiHiertt/blob/main/README.md?plain=1#L13'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/sci-m-wang/LangGPT-tools/blob/main/README.md?plain=1#L29'
    },
    {
        'leaderboard': 'https://skyworkai.github.io/agent-studio',
        'reference': 'https://github.com/SkyworkAI/agent-studio/blob/main/README.md?plain=1#L39'
    },
    {
        'leaderboard': 'https://flageval.baai.ac.cn/#/leaderboard/multimodal',
        'reference': 'https://github.com/Kwai-Kolors/Kolors/blob/master/README.md?plain=1#L60'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/wel3kxial/AIGC_Resources/blob/main/README.md?plain=1#L758'
    },
    {
        'leaderboard': 'https://github.com/THUDM/ReST-MCTS',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/CLUEbenchmark/CLUE',
        'reference': 'https://github.com/aymericdamien/TopDeepLearning/blob/master/README.md?plain=1#L733'
    },
    {
        'leaderboard': 'https://github.com/lampts/my_dl_solutions',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/DengPingFan/DAVSOD',
        'reference': ''
    },
    {
        'leaderboard': 'http://convai.io',
        'reference': 'https://github.com/aliannejadi/ClariQ/blob/master/README.md?plain=1#L475'
    },
    {
        'leaderboard': 'http://got-10k.aitestunion.com/leaderboard',
        'reference': 'https://github.com/got-10k/siamfc/blob/master/README.md?plain=1#L13'
    },
    {
        'leaderboard': 'https://github.com/SiatMMLab/Awesome-Diffusion-Model-Based-Image-Editing-Methods/blob/main/Leaderboard.md',
        'reference': ''
    },
    {
        'leaderboard': 'https://klue-benchmark.com/leaderboard',
        'reference': 'https://github.com/KLUE-benchmark/KLUE/blob/main/README.md?plain=1#L70'
    },
    {
        'leaderboard': 'https://crux-eval.github.io/leaderboard.html',
        'reference': 'https://github.com/facebookresearch/cruxeval/blob/main/README.md?plain=1#L6'
    },
    {
        'leaderboard': 'https://github.com/inspire-group/patch-defense-leaderboard',
        'reference': 'https://github.com/inspire-group/PatchCleanser/blob/main/README.md?plain=1#L27'
    },
    {
        'leaderboard': 'https://github.com/Magnetic2014/RoleEval',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/SoccerNet/sn-spotting',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/sejong-rcv/MLPD-Multi-Label-Pedestrian-Detection',
        'reference': ''
    },
    {
        'leaderboard': 'https://www.aicrowd.com/challenges/imageclef-2019-vqa-med/leaderboards',
        'reference': 'https://github.com/abachaa/VQA-Med-2019/blob/master/README.md?plain=1#L6'
    },
    {
        'leaderboard': 'https://ogb.stanford.edu/docs/leader_nodeprop/#ogbn-arxiv',
        'reference': 'https://github.com/ytchx1999/PyG-OGB-Tricks/blob/main/README.md?plain=1#L20'
    },
    {
        'leaderboard': 'hhttps://ogb.stanford.edu/docs/leader_nodeprop/#ogbn-mag',
        'reference': 'https://github.com/ytchx1999/PyG-OGB-Tricks/blob/main/README.md?plain=1#L25'
    },
    {
        'leaderboard': 'https://ogb.stanford.edu/docs/leader_nodeprop/#ogbn-products',
        'reference': 'https://github.com/ytchx1999/PyG-OGB-Tricks/blob/main/README.md?plain=1#L30'
    },
    {
        'leaderboard': 'https://ogb.stanford.edu/docs/leader_nodeprop/#ogbn-proteins',
        'reference': 'https://github.com/ytchx1999/PyG-OGB-Tricks/blob/main/README.md?plain=1#L36'
    },
    {
        'leaderboard': 'https://eval.ai/web/challenges/challenge-page/514/leaderboard',
        'reference': 'https://github.com/pzzhang/VinVL/blob/main/README.md?plain=1#L31'
    },
    {
        'leaderboard': 'https://eval.ai/web/challenges/challenge-page/355/leaderboard',
        'reference': 'https://github.com/pzzhang/VinVL/blob/main/README.md?plain=1#L31'
    },
    {
        'leaderboard': 'https://leaderboard.allenai.org/vcr',
        'reference': 'https://github.com/pzzhang/VinVL/blob/main/README.md?plain=1#L31'
    },
    {
        'leaderboard': 'https://github.com/layer6ai-labs/dgm-eval',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/salesforce/decaNLP',
        'reference': ''
    },
    {
        'leaderboard': 'https://leaderboard.carla.org/leaderboard',
        'reference': 'https://github.com/carla-simulator/carla/blob/dev/README.md?plain=1#L50'
    },
    {
        'leaderboard': 'https://github.com/eugenesiow/super-image',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/liuye6666/EWR-PGD',
        'reference': ''
    },
    {
        'leaderboard': 'http://host.robots.ox.ac.uk:8080/leaderboard/displaylb.php?challengeid=11&compid=6',
        'reference': 'https://github.com/TheLegendAli/DeepLab-Context/blob/master/README.md?plain=1#L39'
    },
    {
        'leaderboard': 'https://github.com/SoccerNet/sn-tracking',
        'reference': ''
    },
    {
        'leaderboard': 'https://leaderboard.allenai.org/mctaco',
        'reference': 'https://github.com/CogComp/MCTACO/blob/master/README.md?plain=1#L9'
    },
    {
        'leaderboard': 'https://cl-detection2023.grand-challenge.org/evaluation/challenge/leaderboard',
        'reference': 'https://github.com/5k5000/CLdetection2023/blob/master/README.md?plain=1#L7'
    },
    {
        'leaderboard': 'https://cl-detection2023.grand-challenge.org/evaluation/testing/leaderboard',
        'reference': 'https://github.com/5k5000/CLdetection2023/blob/master/README.md?plain=1#L7'
    },
    {
        'leaderboard': 'https://sokcertifiedrobustness.github.io/leaderboard',
        'reference': 'https://github.com/yinizhilian/ICLR2024-Papers-with-Code/blob/main/README.md?plain=1#L2853'
    },
    {
        'leaderboard': 'https://codalab.lisn.upsaclay.fr/competitions/7395',
        'reference': 'https://github.com/thunlp/Knowledge-Plugin/blob/master/README.md?plain=1#L137'
    },
    {
        'leaderboard': 'https://github.com/Kali-Hac/Awesome-Skeleton-Based-Models/tree/main/skeleton-based-action-recognition',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/Kali-Hac/Awesome-Skeleton-Based-Models/tree/main/3D-pose-estimation',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/Kali-Hac/Awesome-Skeleton-Based-Models/tree/main/skeleton-based-person-reID',
        'reference': ''
    },
    {
        'leaderboard': 'https://nl2sql360.github.io/#leaderboard',
        'reference': 'https://github.com/HKUSTDial/NL2SQL360/blob/master/README.md?plain=1#L15'
    },
    {
        'leaderboard': 'https://github.com/yanyanSann/Long-Tailed-Classification-Leaderboard',
        'reference': 'https://github.com/zhangyongshun/BagofTricks-LT/blob/main/README.md?plain=1#L309'
    },
    {
        'leaderboard': 'https://teddy-xionggz.github.io/MIRAGE',
        'reference': 'https://github.com/Teddy-XiongGZ/MIRAGE/blob/main/README.md?plain=1#L115'
    },
    {
        'leaderboard': 'https://github.com/mkliegl/kaggle-Facebook-V',
        'reference': ''
    },
    {
        'leaderboard': 'https://yuchen814.github.io/CodeTransOcean/#leadboard',
        'reference': 'https://github.com/WeixiangYAN/CodeTransOcean/blob/main/README.md?plain=1#L4'
    },
    {
        'leaderboard': 'https://mmmu-benchmark.github.io/#leaderboard',
        'reference': 'https://github.com/MMMU-Benchmark/MMMU/blob/main/README.md?plain=1#L3'
    },
    {
        'leaderboard': 'https://yale-lily.github.io/spider',
        'reference': 'https://github.com/paulfitz/mlsql/blob/master/README.md?plain=1#L173'
    },
    {
        'leaderboard': 'https://github.com/salesforce/WikiSQL',
        'reference': 'https://github.com/paulfitz/mlsql/blob/master/README.md?plain=1#L174'
    },
    {
        'leaderboard': 'https://yale-lily.github.io/sparc',
        'reference': 'https://github.com/paulfitz/mlsql/blob/master/README.md?plain=1#L175'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota',
        'reference': 'https://github.com/Nativeatom/NaturalLanguageProcessing/blob/master/README.md?plain=1#L322'
    },
    {
        'leaderboard': 'https://github.com/hou-yz/MultiviewX',
        'reference': ''
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/bigcode/bigcode-models-leaderboard',
        'reference': 'https://github.com/huybery/Awesome-Code-LLM/blob/main/README.md?plain=1#L68'
    },
    {
        'leaderboard': 'https://bird-bench.github.io',
        'reference': 'https://github.com/huybery/Awesome-Code-LLM/blob/main/README.md?plain=1#L69'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/mike-ravkine/can-ai-code-results',
        'reference': 'https://github.com/huybery/Awesome-Code-LLM/blob/main/README.md?plain=1#L70'
    },
    {
        'leaderboard': 'https://leaderboard.tabbyml.com',
        'reference': 'https://github.com/huybery/Awesome-Code-LLM/blob/main/README.md?plain=1#L71'
    },
    {
        'leaderboard': 'https://crux-eval.github.io/leaderboard.html',
        'reference': 'https://github.com/huybery/Awesome-Code-LLM/blob/main/README.md?plain=1#L72'
    },
    {
        'leaderboard': '(https://evalplus.github.io/leaderboard.html',
        'reference': 'https://github.com/huybery/Awesome-Code-LLM/blob/main/README.md?plain=1#L73'
    },
    {
        'leaderboard': 'https://intercode-benchmark.github.io',
        'reference': 'https://github.com/huybery/Awesome-Code-LLM/blob/main/README.md?plain=1#L76'
    },
    {
        'leaderboard': 'https://accubits.com/open-source-program-synthesis-models-leaderboard',
        'reference': 'https://github.com/huybery/Awesome-Code-LLM/blob/main/README.md?plain=1#L77'
    },
    {
        'leaderboard': 'https://yale-lily.github.io/spider',
        'reference': 'https://github.com/huybery/Awesome-Code-LLM/blob/main/README.md?plain=1#L78'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/mteb/leaderboard',
        'reference': 'https://github.com/Qihoo360/360zhinao/blob/main/README.md?plain=1#L41'
    },
    {
        'leaderboard': 'https://rank.opencompass.org.cn/leaderboard-llm',
        'reference': 'https://github.com/Qihoo360/360zhinao/blob/main/README.md?plain=1#L77'
    },
    {
        'leaderboard': 'https://github.com/MadryLab/cifar10_challenge',
        'reference': ''
    },
    {
        'leaderboard': 'https://datahack.analyticsvidhya.com/contest/ltfs-datascience-finhack-an-online-hackathon/#LeaderBoard',
        'reference': 'https://github.com/rajat5ranjan/AV-LTFS-Data-Science-FinHack-ML-Hackathon/blob/master/README.md?plain=1#L49'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/mteb/leaderboard',
        'reference': 'https://github.com/Wang-Shuo/A-Guide-to-Retrieval-Augmented-LLM/blob/main/README.md?plain=1#L235'
    },
    {
        'leaderboard': 'https://github.com/songlab-cal/tape',
        'reference': ''
    },
    {
        'leaderboard': 'https://www.kaggle.com/c/nyu-cv-fall-2018/leaderboard',
        'reference': 'https://github.com/soumith/traffic-sign-detection-homework/blob/master/README.md?plain=1#L31'
    },
    {
        'leaderboard': 'https://github.com/wenhao728/awesome-diffusion-v2v/blob/main/doc/leaderboard.md',
        'reference': ''
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/TheFinAI/FinBen',
        'reference': 'https://github.com/adlnlp/FinLLMs/blob/main/README.md?plain=1#L61'
    },
    {
        'leaderboard': 'https://codalab.lisn.upsaclay.fr/competitions/10740',
        'reference': 'https://github.com/osyvokon/awesome-ukrainian-nlp/blob/master/README.md?plain=1#L150'
    },
    {
        'leaderboard': 'https://eval.ai/web/challenges/challenge-page/674/leaderboard',
        'reference': 'https://github.com/magnumresearchgroup/Magnum-NLC2CMD/blob/master/README.md?plain=1#L16'
    },
    {
        'leaderboard': 'https://tatsu-lab.github.io/alpaca_eval',
        'reference': 'https://github.com/princeton-nlp/SimPO/blob/main/README.md?plain=1#L8'
    },
    {
        'leaderboard': 'https://www.crowdbenchmark.com/nwpucrowdloc.html',
        'reference': 'https://github.com/taohan10200/IIM/blob/main/README.md?plain=1#L116'
    },
    {
        'leaderboard': 'https://github.com/bolgebrygg/Force-2020-Machine-Learning-competition',
        'reference': ''
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/face-alignment-on-wflw',
        'reference': 'https://github.com/starhiking/HeatmapInHeatmap/blob/main/README.md?plain=1#L7'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/face-alignment-on-cofw',
        'reference': 'https://github.com/starhiking/HeatmapInHeatmap/blob/main/README.md?plain=1#L8'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/face-alignment-on-300w',
        'reference': 'https://github.com/starhiking/HeatmapInHeatmap/blob/main/README.md?plain=1#L9'
    },
    {
        'leaderboard': 'https://matbench.materialsproject.org/Leaderboards%20Per-Task/matbench_v0.1_matbench_dielectric',
        'reference': 'https://github.com/MasterAI-EAM/Darwin/blob/main/README.md?plain=1#L12'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/hf-audio/open_asr_leaderboard',
        'reference': 'https://github.com/huggingface/open_asr_leaderboard/blob/main/README.md?plain=1#L3'
    },
    {
        'leaderboard': 'https://blumenstiel.github.io/mess-benchmark/leaderboard',
        'reference': 'https://github.com/blumenstiel/MESS/blob/main/README.md?plain=1#L44'
    },
    {
        'leaderboard': 'https://github.com/aliyun/cflue',
        'reference': ''
    },
    {
        'leaderboard': 'https://rajpurkar.github.io/SQuAD-explorer',
        'reference': 'https://github.com/unilight/R-NET-in-Tensorflow/blob/master/README.md?plain=1#L87'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/hf-audio/open_asr_leaderboard',
        'reference': 'https://github.com/nyrahealth/CrisperWhisper/blob/main/README.md?plain=1#L37'
    },
    {
        'leaderboard': 'https://github.com/Doragd/Awesome-Sentence-Embedding',
        'reference': ''
    },
    {
        'leaderboard': 'https://ymcui.com/expmrc',
        'reference': 'https://github.com/ymcui/expmrc/blob/main/README.md?plain=1#L23'
    },
    {
        'leaderboard': 'https://www.aicrowd.com/challenges/www-2018-challenge-learning-to-recognize-musical-genre/leaderboards',
        'reference': 'https://github.com/kristijanbartol/Deep-Music-Tagger/blob/master/README.md?plain=1#L84'
    },
    {
        'leaderboard': 'https://leaderboard.allenai.org/musique_ans',
        'reference': 'https://github.com/StonyBrookNLP/musique/blob/main/README.md?plain=1#L69'
    },
    {
        'leaderboard': 'https://leaderboard.allenai.org/musique_full',
        'reference': 'https://github.com/StonyBrookNLP/musique/blob/main/README.md?plain=1#L69'
    },
    {
        'leaderboard': 'https://github.com/songlab-cal/tape-neurips2019',
        'reference': ''
    },
    {
        'leaderboard': 'https://ogb.stanford.edu/docs/leader_linkprop',
        'reference': 'https://github.com/facebookresearch/SEAL_OGB/blob/main/README.md?plain=1#L11'
    },
    {
        'leaderboard': 'https://github.com/coastalcph/lex-glue',
        'reference': ''
    },
    {
        'leaderboard': 'https://leaderboard.allenai.org/arc',
        'reference': 'https://github.com/allenai/macaw/blob/main/README.md?plain=1#L142'
    },
    {
        'leaderboard': 'https://leaderboard.allenai.org/arc_easy',
        'reference': 'https://github.com/allenai/macaw/blob/main/README.md?plain=1#L143'
    },
    {
        'leaderboard': 'https://leaderboard.allenai.org/genie-arcda',
        'reference': 'https://github.com/allenai/macaw/blob/main/README.md?plain=1#L144'
    },
    {
        'leaderboard': 'https://github.com/JUNJIE99/MLVU',
        'reference': ''
    },
    {
        'leaderboard': 'http://www.cvlibs.net/datasets/kitti/eval_object.php?obj_benchmark=3d',
        'reference': 'https://github.com/charlesq34/frustum-pointnets/blob/master/README.md?plain=1#L9'
    },
    {
        'leaderboard': 'http://host.robots.ox.ac.uk:8080/leaderboard/displaylb.php?challengeid=11&compid=4',
        'reference': 'https://github.com/DataXujing/CNN-paper2/blob/master/README.md?plain=1#L930'
    },
    {
        'leaderboard': 'http://host.robots.ox.ac.uk:8080/leaderboard/displaylb.php?challengeid=11&compid=6',
        'reference': 'https://github.com/HymEric/Segmentation-Series-Chaos/blob/master/README.md?plain=1#L50'
    },
    {
        'leaderboard': 'https://paperswithcode.com/task/semantic-segmentation',
        'reference': 'https://github.com/HymEric/Segmentation-Series-Chaos/blob/master/README.md?plain=1#L50'
    },
    {
        'leaderboard': 'https://klejbenchmark.com/leaderboard',
        'reference': 'https://github.com/allegro/HerBERT/blob/master/README.md?plain=1#L12'
    },
    {
        'leaderboard': 'https://leaderboard.allenai.org/break',
        'reference': 'https://github.com/allenai/Break/blob/master/README.md?plain=1#L18'
    },
    {
        'leaderboard': 'https://leaderboard.allenai.org/break_high_level',
        'reference': 'https://github.com/allenai/Break/blob/master/README.md?plain=1#L19'
    },
    {
        'leaderboard': 'https://www.kaggle.com/c/korean-gender-bias-detection/leaderboard',
        'reference': 'https://github.com/kocohub/korean-hate-speech/blob/master/README.md?plain=1#L75'
    },
    {
        'leaderboard': 'https://www.kaggle.com/c/korean-bias-detection/leaderboard',
        'reference': 'https://github.com/kocohub/korean-hate-speech/blob/master/README.md?plain=1#L76'
    },
    {
        'leaderboard': 'https://www.kaggle.com/c/korean-hate-speech-detection/leaderboard',
        'reference': 'https://github.com/kocohub/korean-hate-speech/blob/master/README.md?plain=1#L77'
    },
    {
        'leaderboard': 'http://www.cvlibs.net/datasets/kitti/eval_object.php?obj_benchmark=3d',
        'reference': 'https://github.com/chonepieceyb/reading-frustum-pointnets-code/blob/master/README.md?plain=1#L19'
    },
    {
        'leaderboard': 'https://aistudio.baidu.com/competition/detail/157/0/leaderboard',
        'reference': 'https://github.com/baidu/DuReader/blob/master/README.md?plain=1#L11'
    },
    {
        'leaderboard': 'https://aistudio.baidu.com/competition/detail/116/0/leaderboard',
        'reference': 'https://github.com/baidu/DuReader/blob/master/README.md?plain=1#L13'
    },
    {
        'leaderboard': 'https://aistudio.baidu.com/competition/detail/66/0/leaderboard',
        'reference': 'https://github.com/baidu/DuReader/blob/master/README.md?plain=1#L15'
    },
    {
        'leaderboard': 'https://aistudio.baidu.com/competition/detail/49/0/leaderboard',
        'reference': 'https://github.com/baidu/DuReader/blob/master/README.md?plain=1#L17'
    },
    {
        'leaderboard': 'https://aistudio.baidu.com/competition/detail/720/0/leaderboard',
        'reference': 'https://github.com/baidu/DuReader/blob/master/README.md?plain=1#L21'
    },
    {
        'leaderboard': 'https://github.com/vectara/hallucination-leaderboard',
        'reference': 'https://github.com/Barnacle-ai/awesome-llm-list/blob/main/README.md?plain=1#L115'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/gaia-benchmark/leaderboard',
        'reference': 'https://github.com/Barnacle-ai/awesome-llm-list/blob/main/README.md?plain=1#L116'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/Barnacle-ai/awesome-llm-list/blob/main/README.md?plain=1#L117'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/lmsys/chatbot-arena-leaderboard',
        'reference': 'https://github.com/Barnacle-ai/awesome-llm-list/blob/main/README.md?plain=1#L118'
    },
    {
        'leaderboard': 'https://tatsu-lab.github.io/alpaca_eval',
        'reference': 'https://github.com/Barnacle-ai/awesome-llm-list/blob/main/README.md?plain=1#L119'
    },
    {
        'leaderboard': 'https://inklab.usc.edu/CommonGen/leaderboard.html',
        'reference': 'https://github.com/Barnacle-ai/awesome-llm-list/blob/main/README.md?plain=1#L120'
    },
    {
        'leaderboard': 'https://opencompass.org.cn/leaderboard-llm',
        'reference': 'https://github.com/Barnacle-ai/awesome-llm-list/blob/main/README.md?plain=1#L121'
    },
    {
        'leaderboard': 'https://github.com/RecList/evalRS-CIKM-2022',
        'reference': ''
    },
    {
        'leaderboard': 'https://paperswithcode.com/dataset/worldview-3-pancollection',
        'reference': 'https://github.com/XiaoXiao-Woo/PanCollection/blob/main/README.md?plain=1#L184'
    },
    {
        'leaderboard': 'https://www.nuscenes.org/object-detection',
        'reference': 'https://github.com/fudan-zvg/PolarFormer/blob/main/README.md?plain=1#L21'
    },
    {
        'leaderboard': 'https://www.nuscenes.org/object-detection',
        'reference': 'https://github.com/fudan-zvg/PolarFormer/blob/main/README.md?plain=1#L22'
    },
    {
        'leaderboard': 'https://github.com/pkunlp-icler/PCA-EVAL',
        'reference': ''
    },
    {
        'leaderboard': 'https://stanfordmlgroup.github.io/competitions/mura',
        'reference': 'https://github.com/beamandrew/medical-data/blob/master/README.md?plain=1#L422'
    },
    {
        'leaderboard': 'https://github.com/Mercury7353/PyBench',
        'reference': ''
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/upstage/open-ko-llm-leaderboard',
        'reference': 'https://github.com/Marker-Inc-Korea/KO-Platypus/blob/main/README.md?plain=1#L77'
    },
    {
        'leaderboard': 'https://leaderboard.allenai.org/qasc',
        'reference': 'https://github.com/allenai/qasc/blob/main/README.md?plain=1#L10'
    },
    {
        'leaderboard': 'https://www.kaggle.com/c/open-images-2019-instance-segmentation/leaderboard',
        'reference': 'https://github.com/ZFTurbo/Keras-Mask-RCNN-for-Open-Images-2019-Instance-Segmentation/blob/master/README.md?plain=1#L4'
    },
    {
        'leaderboard': 'https://github.com/layumi/Person_reID_baseline_pytorch/tree/master/leaderboard',
        'reference': 'https://github.com/bismex/Awesome-person-re-identification/blob/master/README.md?plain=1#L799'
    },
    {
        'leaderboard': 'https://github.com/Mercury7353/PyBench',
        'reference': 'https://github.com/zilliztech/VectorDBBench/blob/main/README.md?plain=1#L6'
    },
    {
        'leaderboard': 'https://zilliz.com/vector-database-benchmark-tool',
        'reference': 'https://github.com/zilliztech/VectorDBBench/blob/main/README.md?plain=1#L6'
    },
    {
        'leaderboard': 'https://github.com/h2oai/h2o-LLM-eval',
        'reference': ''
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota',
        'reference': 'https://github.com/Robinwho/Deep-Learning/blob/master/README.md?plain=1#L151'
    },
    {
        'leaderboard': 'https://tenrec0.github.io/#leaderboard',
        'reference': 'https://github.com/yuangh-x/2022-NIPS-Tenrec/blob/master/README.md?plain=1#L18'
    },
    {
        'leaderboard': 'https://github.com/cardiffnlp/tweetnlp/blob/main/FINETUNING_RESULT.md',
        'reference': ''
    },
    {
        'leaderboard': 'https://rowanzellers.com/advice/#leaderboard',
        'reference': 'https://github.com/rowanz/turingadvice/blob/master/README.md?plain=1#L78'
    },
    {
        'leaderboard': 'https://evalai.cloudcv.org/web/challenges/challenge-page/503/leaderboard',
        'reference': 'https://github.com/yuweihao/reclor/blob/master/README.md?plain=1#L27'
    },
    {
        'leaderboard': 'https://github.com/JBoRu/Awesome-KBQA/blob/main/leaderboard/cfq.md',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/JBoRu/Awesome-KBQA/blob/main/leaderboard/complexquestions.md',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/JBoRu/Awesome-KBQA/blob/main/leaderboard/complexwebquestions.md',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/JBoRu/Awesome-KBQA/blob/main/leaderboard/grailqa.md',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/JBoRu/Awesome-KBQA/blob/main/leaderboard/kqapro.md',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/JBoRu/Awesome-KBQA/blob/main/leaderboard/lc-quad.md',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/JBoRu/Awesome-KBQA/blob/main/leaderboard/metaqa.md',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/JBoRu/Awesome-KBQA/blob/main/leaderboard/webquestions.md',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/JBoRu/Awesome-KBQA/blob/main/leaderboard/webquestionsp.md',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/JBoRu/Awesome-KBQA/blob/main/leaderboard/cfq.md',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/JBoRu/Awesome-KBQA/blob/main/leaderboard/cfq.md',
        'reference': ''
    },
    {
        'leaderboard': 'https://dki-lab.github.io/GrailQA',
        'reference': 'https://github.com/JBoRu/Awesome-KBQA/blob/main/README.md?plain=1#L37'
    },
    {
        'leaderboard': 'https://github.com/google-research/google-research/tree/master/cfq',
        'reference': 'https://github.com/JBoRu/Awesome-KBQA/blob/main/README.md?plain=1#L35'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/trajectory-prediction-on-stanford-drone',
        'reference': 'https://github.com/karttikeya/awesome-human-pose-prediction/blob/master/README.md?plain=1#L26'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/osunlp/TravelPlannerLeaderboard',
        'reference': 'https://github.com/OSU-NLP-Group/TravelPlanner/blob/main/README.md?plain=1#L122'
    },
    {
        'leaderboard': 'https://opendataval.github.io/leaderboards',
        'reference': 'https://github.com/opendataval/opendataval/blob/main/README.md?plain=1#L24'
    },
    {
        'leaderboard': 'https://research.nianticlabs.com/mapfree-reloc-benchmark/leaderboard',
        'reference': 'https://github.com/nianticlabs/map-free-reloc/blob/main/README.md?plain=1#L28'
    },
    {
        'leaderboard': 'https://github.com/ocean-data-challenges/2020a_SSH_mapping_NATL60',
        'reference': ''
    },
    {
        'leaderboard': 'https://microsoft.github.io/XGLUE',
        'reference': 'https://github.com/microsoft/XGLUE/blob/master/README.md?plain=1#L4'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/Auto-Arena/Leaderboard',
        'reference': 'https://github.com/DAMO-NLP-SG/Auto-Arena-LLMs/blob/main/readme.md?plain=1#L3'
    },
    {
        'leaderboard': 'https://github.com/DetectionTeamUCAS/R2CNN_Faster-RCNN_Tensorflow',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/openai/gym/wiki/Leaderboard',
        'reference': 'https://github.com/Datatouille/rl-workshop/blob/master/README.md?plain=1#L93'
    },
    {
        'leaderboard': 'https://codalab.lisn.upsaclay.fr/competitions/1846',
        'reference': 'https://github.com/czyssrs/FinQA/blob/main/README.md?plain=1#L19'
    },
    {
        'leaderboard': 'https://codalab.lisn.upsaclay.fr/competitions/4138',
        'reference': 'https://github.com/czyssrs/FinQA/blob/main/README.md?plain=1#L21'
    },
    {
        'leaderboard': 'https://cocodataset.org/#keypoints-leaderboard',
        'reference': 'https://github.com/caiyuanhao1998/RSN/blob/master/README.md?plain=1#L18'
    },
    {
        'leaderboard': 'https://ai.google.com/research/tydiqa',
        'reference': 'https://github.com/google-research-datasets/tydiqa/blob/master/README.md?plain=1#L163'
    },
    {
        'leaderboard': 'https://github.com/primeqa/primeqa',
        'reference': ''
    },
    {
        'leaderboard': 'https://leaderboard.allenai.org/anli',
        'reference': 'https://github.com/allenai/rainbow/blob/master/readme.md?plain=1#L409'
    },
    {
        'leaderboard': 'https://www.tau-nlp.org/csqa-leaderboard',
        'reference': 'https://github.com/allenai/rainbow/blob/master/readme.md?plain=1#L419'
    },
    {
        'leaderboard': 'https://leaderboard.allenai.org/cosmosqa',
        'reference': 'https://github.com/allenai/rainbow/blob/master/readme.md?plain=1#L422'
    },
    {
        'leaderboard': 'https://leaderboard.allenai.org/hellaswag',
        'reference': 'https://github.com/allenai/rainbow/blob/master/readme.md?plain=1#L424'
    },
    {
        'leaderboard': 'https://leaderboard.allenai.org/physicaliqa',
        'reference': 'https://github.com/allenai/rainbow/blob/master/readme.md?plain=1#L430'
    },
    {
        'leaderboard': 'https://leaderboard.allenai.org/rainbow',
        'reference': 'https://github.com/allenai/rainbow/blob/master/readme.md?plain=1#L444'
    },
    {
        'leaderboard': 'https://leaderboard.allenai.org/socialiqa',
        'reference': 'https://github.com/allenai/rainbow/blob/master/readme.md?plain=1#L449'
    },
    {
        'leaderboard': 'https://leaderboard.allenai.org/winogrande',
        'reference': 'https://github.com/allenai/rainbow/blob/master/readme.md?plain=1#L458'
    },
    {
        'leaderboard': 'https://github.com/MadryLab/backgrounds_challenge',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/BUPT-GAMMA/OpenHGNN/blob/main/openhgnn/dataset/ohgb.md',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/morecry/CharacterEval',
        'reference': ''
    },
    {
        'leaderboard': 'https://docs.google.com/spreadsheets/d/1M801lEpBbKSNwP-vDBkC_pF7LdyGU1f_ufZb_NWNBZQ',
        'reference': 'https://github.com/web-arena-x/webarena/blob/main/README.md?plain=1#L20'
    },
    {
        'leaderboard': 'https://bedlam-leaderboard.is.tuebingen.mpg.de/leaderboard',
        'reference': 'https://github.com/pixelite1201/BEDLAM/blob/master/README.md?plain=1#L75'
    },
    {
        'leaderboard': 'https://evalplus.github.io/leaderboard.html',
        'reference': 'https://github.com/nlpxucan/WizardLM/blob/main/README.md?plain=1#L32'
    },
    {
        'leaderboard': 'https://github.com/LLVM-AD/MAPLM',
        'reference': ''
    },
    {
        'leaderboard': 'https://rajpurkar.github.io/SQuAD-explorer',
        'reference': 'https://github.com/chiayewken/bert-qa/blob/master/README.md?plain=1#L71'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/HaizeLabs/red-teaming-resistance-benchmark',
        'reference': 'https://github.com/haizelabs/redteaming-resistance-benchmark/blob/main/README.md?plain=1#L4'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/bigcode/bigcode-models-leaderboard',
        'reference': 'https://github.com/bigcode-project/starcoder2/blob/main/README.md?plain=1#L168'
    },
    {
        'leaderboard': 'https://aider.chat/docs/leaderboards',
        'reference': 'https://github.com/SakanaAI/AI-Scientist/blob/main/README.md?plain=1#L271'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/dccuchile/CC6205/blob/master/README.md?plain=1#L102'
    },
    {
        'leaderboard': 'https://eu.aihub.ml/competitions/201',
        'reference': 'https://github.com/yoxu515/aot-benchmark/blob/main/README.md?plain=1#L13'
    },
    {
        'leaderboard': 'https://aesbench.github.io',
        'reference': 'https://github.com/yipoh/AesBench/blob/main/README.md?plain=1#L44'
    },
    {
        'leaderboard': 'https://github.com/OpenGenerativeAI/llm-colosseum',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/GaParmar/clean-fid#cleanfid-leaderboard-for-common-tasks',
        'reference': ''
    },
    {
        'leaderboard': 'https://hotpotqa.github.io',
        'reference': 'https://github.com/qipeng/golden-retriever/blob/master/README.md?plain=1#L5'
    },
    {
        'leaderboard': 'https://waymo.com/open/challenges/2021/real-time-3d-prediction',
        'reference': 'https://github.com/Nightmare-n/GraphRCNN/blob/main/README.md?plain=1#L10'
    },
    {
        'leaderboard': 'https://github.com/gabrielhuang/awesome-few-shot-object-detection',
        'reference': ''
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/liguodongiot/llm-resource/blob/main/README.md?plain=1#L210'
    },
    {
        'leaderboard': 'https://comma.ai/leaderboard',
        'reference': 'https://github.com/commaai/commavq/blob/master/README.md?plain=1#L22'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/mteb/leaderboard',
        'reference': 'https://github.com/Anush008/fastembed-js/blob/main/README.md?plain=1#L14'
    },
    {
        'leaderboard': 'https://video-mme.github.io/home_page.html#leaderboard',
        'reference': 'https://github.com/BradyFU/Video-MME/blob/main/README.md?plain=1#L144'
    },
    {
        'leaderboard': 'https://rajpurkar.github.io/SQuAD-explorer',
        'reference': 'https://github.com/stepthom/text_mining_resources/blob/master/README.md?plain=1#L438'
    },
    {
        'leaderboard': 'https://gluebenchmark.com/leaderboard',
        'reference': 'https://github.com/stepthom/text_mining_resources/blob/master/README.md?plain=1#L441'
    },
    {
        'leaderboard': 'https://decathlon-10.grand-challenge.org/evaluation/challenge/leaderboard',
        'reference': 'https://github.com/ljwztc/CLIP-Driven-Universal-Model/blob/main/README.md?plain=1#L14'
    },
    {
        'leaderboard': 'https://github.com/IGNF/FLAIR-2',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/FranxYao/chain-of-thought-hub',
        'reference': ''
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/Nexusflow/Nexus_Function_Calling_Leaderboard',
        'reference': 'https://github.com/nexusflowai/NexusRaven-V2/blob/master/README.md?plain=1#L9'
    },
    {
        'leaderboard': 'https://github.com/nyu-dl/dl4ir-doc2query',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/SoccerNet/sn-calibration',
        'reference': ''
    },
    {
        'leaderboard': 'https://lmarena.ai/?leaderboard',
        'reference': 'https://github.com/rodion-m/awesome_ai_for_programmers/blob/main/README.md?plain=1#L410'
    },
    {
        'leaderboard': 'https://github.com/lm-sys/arena-hard-auto',
        'reference': 'https://github.com/rodion-m/awesome_ai_for_programmers/blob/main/README.md?plain=1#L411'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/bigcode/bigcode-models-leaderboard',
        'reference': 'https://github.com/rodion-m/awesome_ai_for_programmers/blob/main/README.md?plain=1#L413'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/gaia-benchmark/leaderboard',
        'reference': 'https://github.com/rodion-m/awesome_ai_for_programmers/blob/main/README.md?plain=1#L415'
    },
    {
        'leaderboard': 'https://www.trustbit.tech/en/llm-benchmarks',
        'reference': 'https://github.com/rodion-m/awesome_ai_for_programmers/blob/main/README.md?plain=1#L416'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/rodion-m/awesome_ai_for_programmers/blob/main/README.md?plain=1#L417'
    },
    {
        'leaderboard': 'https://bird-bench.github.io',
        'reference': 'https://github.com/rodion-m/awesome_ai_for_programmers/blob/main/README.md?plain=1#L418'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/mteb/leaderboard',
        'reference': 'https://github.com/rodion-m/awesome_ai_for_programmers/blob/main/README.md?plain=1#L419'
    },
    {
        'leaderboard': 'https://github.com/vectara/hallucination-leaderboard',
        'reference': 'https://github.com/jxzhangjhu/Awesome-LLM-Uncertainty-Reliability-Robustness/blob/main/README.md?plain=1#L657'
    },
    {
        'leaderboard': 'https://pages.nist.gov/jarvis_leaderboard',
        'reference': 'https://github.com/usnistgov/chemnlp/blob/main/README.md?plain=1#L96'
    },
    {
        'leaderboard': 'https://github.com/vectara/hallucination-leaderboard',
        'reference': 'https://github.com/EdinburghNLP/awesome-hallucination-detection/blob/main/README.md?plain=1#L393'
    },
    {
        'leaderboard': 'https://eval.ai/web/challenges/challenge-page/454/leaderboard',
        'reference': 'https://github.com/AIprogrammer/vehicle-trajectory-prediction/blob/master/README.md?plain=1#L74'
    },
    {
        'leaderboard': 'https://github.com/declare-lab/multimodal-deep-learning',
        'reference': ''
    },
    {
        'leaderboard': 'https://aistudio.baidu.com/competition/detail/157/0/leaderboard',
        'reference': 'https://github.com/PaddlePaddle/RocketQA/blob/main/README.md?plain=1#L20'
    },
    {
        'leaderboard': 'https://github.com/seg/2016-ml-contest',
        'reference': ''
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/named-entity-recognition-on-few-nerd-sup',
        'reference': 'https://github.com/tomaarsen/SpanMarkerNER/blob/main/README.md?plain=1#L148'
    },
    {
        'leaderboard': 'https://videovista.github.io/#leaderboard',
        'reference': 'https://github.com/KangarooGroup/Kangaroo/blob/main/README.md?plain=1#L6'
    },
    {
        'leaderboard': 'https://video-mme.github.io/home_page.html#leaderboard',
        'reference': 'https://github.com/KangarooGroup/Kangaroo/blob/main/README.md?plain=1#L7'
    },
    {
        'leaderboard': 'https://github.com/Pranav-chib/End-to-End-Autonomous-Driving',
        'reference': ''
    },
    {
        'leaderboard': 'https://leaderboard.carla.org/leaderboard',
        'reference': 'https://github.com/Pranav-chib/End-to-End-Autonomous-Driving/blob/main/README.md?plain=1#L332'
    },
    {
        'leaderboard': 'https://github.com/NREL/BuildingsBench',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/GAIR-NLP/MetaCritique',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/sociocom/JMED-LLM',
        'reference': ''
    },
    {
        'leaderboard': 'https://gorilla.cs.berkeley.edu/leaderboard.html',
        'reference': 'https://github.com/OpenBMB/MiniCPM/blob/main/README.md?plain=1#L274'
    },
    {
        'leaderboard': 'http://got-10k.aitestunion.com/leaderboard',
        'reference': 'https://github.com/huanglianghua/siamfc-pytorch/blob/master/README.md?plain=1#L20'
    },
    {
        'leaderboard': 'https://hotpotqa.github.io',
        'reference': 'https://github.com/shmsw25/DecompRC/blob/main/README.md?plain=1#L31'
    },
    {
        'leaderboard': 'https://super.gluebenchmark.com/leaderboard',
        'reference': 'https://github.com/nlpfromscratch/nlp-llms-resources/blob/master/README.md?plain=1#L509'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/lmsys/chatbot-arena-leaderboard',
        'reference': 'https://github.com/nlpfromscratch/nlp-llms-resources/blob/master/README.md?plain=1#L517'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/nlpfromscratch/nlp-llms-resources/blob/master/README.md?plain=1#L518'
    },
    {
        'leaderboard': 'https://tatsu-lab.github.io/alpaca_eval',
        'reference': 'https://github.com/nlpfromscratch/nlp-llms-resources/blob/master/README.md?plain=1#L519'
    },
    {
        'leaderboard': 'https://opencompass.org.cn/leaderboard-llm',
        'reference': 'https://github.com/nlpfromscratch/nlp-llms-resources/blob/master/README.md?plain=1#L520'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/uukuguy/multi_loras/blob/main/README.md?plain=1#L125'
    },
    {
        'leaderboard': 'https://mmstar-benchmark.github.io/#Leaderboard%20Title',
        'reference': 'https://github.com/MMStar-Benchmark/MMStar/blob/main/README.md?plain=1#L3'
    },
    {
        'leaderboard': 'https://rank.opencompass.org.cn/leaderboard-multimodal',
        'reference': 'https://github.com/MMStar-Benchmark/MMStar/blob/main/README.md?plain=1#L15'
    },
    {
        'leaderboard': 'https://robustbench.github.io',
        'reference': 'https://github.com/RobustBench/robustbench/blob/master/README.md?plain=1#L6'
    },
    {
        'leaderboard': 'https://github.com/fra31/auto-attack',
        'reference': 'https://github.com/RobustBench/robustbench/blob/master/README.md?plain=1#L50'
    },
    {
        'leaderboard': 'https://github.com/brohrer/parameter_efficiency_leaderboard',
        'reference': 'https://github.com/brohrer/sharpened-cosine-similarity/blob/main/README.md?plain=1#L49'
    },
    {
        'leaderboard': 'https://routingchallenge.mit.edu/last-mile-routing-challenge-team-performance-and-leaderboard',
        'reference': 'https://github.com/aws-samples/amazon-sagemaker-amazon-routing-challenge-sol/blob/main/README.md?plain=1#L12'
    },
    {
        'leaderboard': 'https://eval.ai/web/challenges/challenge-page/1839/leaderboard',
        'reference': 'https://github.com/microsoft/GLIP/blob/main/README.md?plain=1#L342'
    },
    {
        'leaderboard': 'https://github.com/HanxunH/RobustWRN',
        'reference': ''
    },
    {
        'leaderboard': 'https://robustbench.github.io',
        'reference': 'https://github.com/HanxunH/RobustWRN/blob/main/README.md?plain=1#L54'
    },
    {
        'leaderboard': 'https://paperswithcode.com/dataset/imagenet-s',
        'reference': 'https://github.com/LUSSeg/ImageNet-S/blob/main/README.md?plain=1#L5'
    },
    {
        'leaderboard': 'https://www.tau-nlp.org/csqa-leaderboard',
        'reference': 'https://github.com/jonathanherzig/commonsenseqa/blob/master/README.md?plain=1#L118'
    },
    {
        'leaderboard': 'https://tbrain.trendmicro.com.tw/Competitions/Details/19',
        'reference': 'https://github.com/yaoching0/Traditional-Chinese-Street-View-Text-Recognition/blob/main/README.md?plain=1#L2'
    },
    {
        'leaderboard': 'https://github.com/cardiffnlp/xlm-t',
        'reference': ''
    },
    {
        'leaderboard': 'https://www.aicrowd.com/challenges/trajnet-a-trajectory-forecasting-challenge/leaderboards',
        'reference': 'https://github.com/theDebugger811/human-trajectory-forecasting-papers/blob/master/README.md?plain=1#L124'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/multi-object-tracking-on-personpath22',
        'reference': 'https://github.com/amazon-science/tracking-dataset/blob/main/README.md?plain=1#L28'
    },
    {
        'leaderboard': 'https://github.com/Hritikbansal/videophy',
        'reference': ''
    },
    {
        'leaderboard': 'https://allenai.github.io/re-align/just_eval.html#leaderboard',
        'reference': 'https://github.com/Re-Align/just-eval/blob/main/README.md?plain=1#L10'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/abhinand5/tamil-llama/blob/main/README.md?plain=1#L80'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/BramVanroy/open_dutch_llm_leaderboard',
        'reference': 'https://github.com/Rijgersberg/GEITje/blob/main/README.md?plain=1#L202'
    },
    {
        'leaderboard': 'https://github.com/darraghdog/avito-demand',
        'reference': ''
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/Leeroo-AI/leeroo_orchestrator/blob/main/README.md?plain=1#L146'
    },
    {
        'leaderboard': 'https://waymo.com/open/challenges/2021/real-time-3d-prediction',
        'reference': 'https://github.com/gwenzhang/Voxel-Mamba/blob/master/README.md?plain=1#L29'
    },
    {
        'leaderboard': 'https://www.nuscenes.org/object-detection',
        'reference': 'https://github.com/gwenzhang/Voxel-Mamba/blob/master/README.md?plain=1#L41'
    },
    {
        'leaderboard': 'https://waymo.com/open/challenges/2020/3d-detection',
        'reference': 'https://github.com/PJLab-ADG/DetZero/blob/main/README.md?plain=1#L30'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/lmmlzn/Awesome-LLMs-Datasets/blob/main/README.md?plain=1#L81'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/mteb/leaderboard',
        'reference': 'https://github.com/lmmlzn/Awesome-LLMs-Datasets/blob/main/README.md?plain=1#L81'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/mteb/leaderboard',
        'reference': 'https://github.com/lmmlzn/Awesome-LLMs-Datasets/blob/main/README.md?plain=1#L81'
    },
    {
        'leaderboard': 'https://github.com/FlagOpen/FlagEmbedding/tree/master/C_MTEB',
        'reference': 'https://github.com/lmmlzn/Awesome-LLMs-Datasets/blob/main/README.md?plain=1#L81'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/lmsys/chatbot-arena-leaderboard',
        'reference': 'https://github.com/lmmlzn/Awesome-LLMs-Datasets/blob/main/README.md?plain=1#L1648'
    },
    {
        'leaderboard': 'https://domaineval.github.io/leaderboard.html',
        'reference': 'https://github.com/lmmlzn/Awesome-LLMs-Datasets/blob/main/README.md?plain=1#L2707'
    },
    {
        'leaderboard': 'https://multi-trust.github.io/#leaderboard',
        'reference': 'https://github.com/lmmlzn/Awesome-LLMs-Datasets/blob/main/README.md?plain=1#L4339'
    },
    {
        'leaderboard': 'https://github.com/google-research-datasets/ToTTo',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/krystalan/Multi-hopRC',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/google/BIG-bench',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/nitsaick/kits19-challenge',
        'reference': ''
    },
    {
        'leaderboard': 'https://av.superbbenchmark.org/leaderboard',
        'reference': 'https://github.com/roger-tseng/av-superb/blob/main/README.md?plain=1#L5'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/IS2Lab/S-Eval',
        'reference': 'https://github.com/IS2Lab/S-Eval/blob/main/README.md?plain=1#L267'
    },
    {
        'leaderboard': 'https://eurocity-dataset.tudelft.nl/eval/benchmarks/detection',
        'reference': 'https://github.com/hasanirtiza/Pedestron/blob/master/README.md?plain=1#L29'
    },
    {
        'leaderboard': 'https://github.com/JUNJIE99/MLVU',
        'reference': 'https://github.com/NVlabs/VILA/blob/main/README.md?plain=1#L19'
    },
    {
        'leaderboard': 'https://mmmu-benchmark.github.io/#leaderboard',
        'reference': 'https://github.com/NVlabs/VILA/blob/main/README.md?plain=1#L20'
    },
    {
        'leaderboard': 'https://video-mme.github.io/home_page.html#leaderboard',
        'reference': 'https://github.com/NVlabs/VILA/blob/main/README.md?plain=1#L20'
    },
    {
        'leaderboard': 'https://opencompass.org.cn/leaderboard-multimodal',
        'reference': 'https://github.com/NVlabs/VILA/blob/main/README.md?plain=1#L171'
    },
    {
        'leaderboard': 'https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge/leaderboard',
        'reference': 'https://github.com/RodolfoFerro/psychopathology-fer-assistant/blob/master/README.md?plain=1#L161'
    },
    {
        'leaderboard': 'http://cuge.baai.ac.cn/#/leaderboard',
        'reference': 'https://github.com/TsinghuaAI/CUGE/blob/main/README.md?plain=1#L25'
    },
    {
        'leaderboard': 'https://gorilla.cs.berkeley.edu/leaderboard',
        'reference': 'https://github.com/ShishirPatil/gorilla/blob/main/README.md?plain=1#L8'
    },
    {
        'leaderboard': 'https://github.com/bytedance/MTVQA',
        'reference': ''
    },
    {
        'leaderboard': 'https://livecodebench.github.io/leaderboard.html',
        'reference': 'https://github.com/LiveCodeBench/LiveCodeBench/blob/main/README.md?plain=1#L7'
    },
    {
        'leaderboard': 'https://github.com/DeepPavlovAdmin/convai',
        'reference': ''
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/image-classification-on-dtd',
        'reference': 'https://github.com/ZhangYuanhan-AI/Bamboo/blob/main/README.md?plain=1#L43'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/image-classification-on-food-101-1',
        'reference': 'https://github.com/ZhangYuanhan-AI/Bamboo/blob/main/README.md?plain=1#L44'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/fine-grained-image-classification-on-sun397',
        'reference': 'https://github.com/ZhangYuanhan-AI/Bamboo/blob/main/README.md?plain=1#L45'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/image-classification-on-flowers-102',
        'reference': 'https://github.com/ZhangYuanhan-AI/Bamboo/blob/main/README.md?plain=1#L46'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/fine-grained-image-classification-on-caltech',
        'reference': 'https://github.com/ZhangYuanhan-AI/Bamboo/blob/main/README.md?plain=1#L47'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/fine-grained-image-classification-on-oxford-1',
        'reference': 'https://github.com/ZhangYuanhan-AI/Bamboo/blob/main/README.md?plain=1#L48'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/image-classification-on-cifar-100',
        'reference': 'https://github.com/ZhangYuanhan-AI/Bamboo/blob/main/README.md?plain=1#L49'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/fine-grained-image-classification-on-stanford',
        'reference': 'https://github.com/ZhangYuanhan-AI/Bamboo/blob/main/README.md?plain=1#L50'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/image-classification-on-cifar-10',
        'reference': 'https://github.com/ZhangYuanhan-AI/Bamboo/blob/main/README.md?plain=1#L51'
    },
    {
        'leaderboard': 'https://klejbenchmark.com/leaderboard',
        'reference': 'https://github.com/Ermlab/PoLitBert/blob/master/README.md?plain=1#L145'
    },
    {
        'leaderboard': 'https://few-shot.yyliu.net/miniimagenet.html',
        'reference': 'https://github.com/yaoyao-liu/few-shot-classification-leaderboard/blob/main/README.md?plain=1#L5'
    },
    {
        'leaderboard': 'https://tianchi.aliyun.com/competition',
        'reference': 'https://github.com/gathierry/FashionAI-KeyPointsDetectionOfApparel/blob/master/README.md?plain=1#L125'
    },
    {
        'leaderboard': 'https://www.tanksandtemples.org/leaderboard',
        'reference': 'https://github.com/yhw-yhw/PVAMVSNet/blob/master/README.md?plain=1#L56'
    },
    {
        'leaderboard': 'https://github.com/Tsinghua-MARS-Lab/Online-HD-Map-Construction-CVPR2023',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/jingtaozhan/RepBERT-Index',
        'reference': ''
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/llm-jp/awesome-japanese-llm/blob/main/README.md?plain=1#L384'
    },
    {
        'leaderboard': 'https://github.com/GammaTauAI/leetcode-hard-gym',
        'reference': ''
    },
    {
        'leaderboard': 'https://lamp-benchmark.github.io/leaderboard',
        'reference': 'https://github.com/Longyichen/Alpaca-family-library/blob/main/README.md?plain=1#L1287'
    },
    {
        'leaderboard': 'https://github.com/kaiko-ai/eva',
        'reference': ''
    },
    {
        'leaderboard': 'http://ednet-leaderboard.s3-website-ap-northeast-1.amazonaws.com',
        'reference': 'https://github.com/riiid/ednet/blob/master/README.md?plain=1#L5'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/allenai/reward-bench',
        'reference': 'https://github.com/RLHFlow/RLHF-Reward-Modeling/blob/main/README.md?plain=1#L65'
    },
    {
        'leaderboard': 'https://superbbenchmark.org/leaderboard',
        'reference': 'https://github.com/pyf98/DPHuBERT/blob/master/README.md?plain=1#L27'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/mteb/leaderboard',
        'reference': 'https://github.com/ghimiresunil/LLM-PowerHouse-A-Curated-Guide-for-Large-Language-Models-with-Custom-Training-and-Inferencing/blob/main/README.md?plain=1#L464'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/lmsys/chatbot-arena-leaderboard',
        'reference': 'https://github.com/ghimiresunil/LLM-PowerHouse-A-Curated-Guide-for-Large-Language-Models-with-Custom-Training-and-Inferencing/blob/main/README.md?plain=1#L329'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/ghimiresunil/LLM-PowerHouse-A-Curated-Guide-for-Large-Language-Models-with-Custom-Training-and-Inferencing/blob/main/README.md?plain=1#L318'
    },
    {
        'leaderboard': 'https://github.com/yushundong/PyGDebias',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/MadryLab/mnist_challenge',
        'reference': ''
    },
    {
        'leaderboard': 'https://marble-bm.shef.ac.uk/leaderboard',
        'reference': 'https://github.com/yizhilll/MERT/blob/main/README.md?plain=1#L8'
    },
    {
        'leaderboard': 'https://yale-lily.github.io/spider',
        'reference': 'https://github.com/AlibabaResearch/DAMO-ConvAI/blob/main/README.md?plain=1#L24'
    },
    {
        'leaderboard': 'https://yale-lily.github.io/sparc',
        'reference': 'https://github.com/AlibabaResearch/DAMO-ConvAI/blob/main/README.md?plain=1#L26'
    },
    {
        'leaderboard': 'https://yale-lily.github.io/cosql',
        'reference': 'https://github.com/AlibabaResearch/DAMO-ConvAI/blob/main/README.md?plain=1#L26'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/sentiment-analysis-on-sst-2-binary',
        'reference': 'https://github.com/dsindex/iclassifier/blob/master/README.md?plain=1#L430'
    },
    {
        'leaderboard': 'https://github.com/DiligentPanda/Tencent_Ads_Algo_2018',
        'reference': ''
    },
    {
        'leaderboard': 'https://paperswithcode.com/task/rgb-d-salient-object-detection',
        'reference': 'https://github.com/zwbx/BTS-Net/blob/main/README.md?plain=1#L13'
    },
    {
        'leaderboard': 'https://paperswithcode.com/dataset/medmcqa',
        'reference': 'https://github.com/medmcqa/medmcqa/blob/main/README.md?plain=1#L45'
    },
    {
        'leaderboard': 'https://waymo.com/open/challenges/2023/motion-prediction',
        'reference': 'https://github.com/sshaoshuai/MTR/blob/master/README.md?plain=1#L16'
    },
    {
        'leaderboard': 'https://www.kaggle.com/c/freesound-audio-tagging-2019/leaderboard',
        'reference': 'https://github.com/ebouteillon/freesound-audio-tagging-2019/blob/master/README.md?plain=1#L3'
    },
    {
        'leaderboard': 'https://github.com/fastai/imagenette',
        'reference': 'https://github.com/lessw2020/Ranger-Mish-ImageWoof-5/blob/master/README.md?plain=1#L50'
    },
    {
        'leaderboard': 'https://fewnlu.github.io',
        'reference': 'https://github.com/THUDM/FewNLU/blob/main/README.md?plain=1#L16'
    },
    {
        'leaderboard': 'https://korquad.github.io',
        'reference': 'https://github.com/lyeoni/KorQuAD/blob/master/README.md?plain=1#L9'
    },
    {
        'leaderboard': 'https://github.com/helme/ecg_ptbxl_benchmarking',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/HaohanWang/ImageNet-Sketch',
        'reference': ''
    },
    {
        'leaderboard': 'https://rrc.cvc.uab.es/?ch=17&com=evaluation&task=1',
        'reference': 'https://github.com/anisha2102/docvqa/blob/master/README.md?plain=1#L2'
    },
    {
        'leaderboard': 'https://cevalbenchmark.com/static/leaderboard.html',
        'reference': 'https://github.com/InternLM/InternLM-techreport/blob/main/README.md?plain=1#L61'
    },
    {
        'leaderboard': 'https://github.com/pfnet-research/japanese-lm-fin-harness',
        'reference': ''
    },
    {
        'leaderboard': 'https://www.tanksandtemples.org/leaderboard',
        'reference': 'https://github.com/FangjinhuaWang/IterMVS/blob/main/README.md?plain=1#L97'
    },
    {
        'leaderboard': 'https://stanfordmlgroup.github.io/competitions/mrnet',
        'reference': 'https://github.com/yashbhalgat/MRNet-Competition/blob/master/README.md?plain=1#L2'
    },
    {
        'leaderboard': 'https://www.aicrowd.com/challenges/music-demixing-challenge-ismir-2021/leaderboards',
        'reference': 'https://github.com/ws-choi/LASAFT-Net-v2/blob/main/README.md?plain=1#L25'
    },
    {
        'leaderboard': 'https://github.com/rajasagashe/JuICe',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/FDU-VTS/Awesome-Person-Re-Identification',
        'reference': ''
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/allenai/WildBench',
        'reference': 'https://github.com/magpie-align/magpie/blob/main/README.md?plain=1#L24'
    },
    {
        'leaderboard': 'https://github.com/taohan10200/WEATHER-5K',
        'reference': ''
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/conversational-web-navigation-on-weblinx',
        'reference': 'https://github.com/McGill-NLP/weblinx/blob/main/README.md?plain=1#L7'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/felipemaiapolo/tinyBenchmarks/blob/main/README.md?plain=1#L22'
    },
    {
        'leaderboard': 'https://github.com/JiaxinZhuang/Skin-Lesion-Recognition.Pytorch/blob/master/Leaderboard.png',
        'reference': ''
    },
    {
        'leaderboard': 'https://www.drivendata.org/competitions/70/hateful-memes-phase-2/leaderboard',
        'reference': 'https://github.com/rizavelioglu/hateful_memes-hate_detectron/blob/main/README.md?plain=1#L34'
    },
    {
        'leaderboard': 'http://got-10k.aitestunion.com/leaderboard',
        'reference': 'https://github.com/got-10k/toolkit/blob/master/README.md?plain=1#L165'
    },
    {
        'leaderboard': 'https://github.com/SmartLi8/stella',
        'reference': ''
    },
    {
        'leaderboard': 'https://gluebenchmark.com/leaderboard',
        'reference': 'https://github.com/fastnlp/ElasticBERT/blob/main/README.md?plain=1#L41'
    },
    {
        'leaderboard': 'https://l-eval.github.io',
        'reference': 'https://github.com/OpenLMLab/LEval/blob/main/README.md?plain=1#L19'
    },
    {
        'leaderboard': 'https://www.kaggle.com/c/google-ai-open-images-object-detection-track/leaderboard',
        'reference': 'https://github.com/ZFTurbo/Keras-RetinaNet-for-Open-Images-Challenge-2018/blob/master/README.md?plain=1#L4'
    },
    {
        'leaderboard': 'https://github.com/bethgelab/robust-detection-benchmark',
        'reference': ''
    },
    {
        'leaderboard': 'https://eval.ai/web/challenges/challenge-page/161/leaderboard',
        'reference': 'https://github.com/salesforce/VD-BERT/blob/master/README.md?plain=1#L9'
    },
    {
        'leaderboard': 'https://www.ai4climatecoop.org',
        'reference': 'https://github.com/mila-iqia/climate-cooperation-competition/blob/main/README.md?plain=1#L22'
    },
    {
        'leaderboard': 'https://github.com/cambridgeltl/xcopa',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/GPT-Fathom/GPT-Fathom',
        'reference': ''
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/GPT-Fathom/GPT-Fathom/blob/main/README.md?plain=1#L94'
    },
    {
        'leaderboard': 'https://github.com/ecker-lab/object-centric-representation-benchmark',
        'reference': ''
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/AIR-Bench/leaderboard',
        'reference': 'https://github.com/FlagOpen/FlagEmbedding/blob/master/README.md?plain=1#L57'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/mteb/leaderboard',
        'reference': 'https://github.com/FlagOpen/FlagEmbedding/blob/master/README.md?plain=1#L74'
    },
    {
        'leaderboard': 'https://github.com/bcollazo/catanatron',
        'reference': ''
    },
    {
        'leaderboard': 'https://evalai.cloudcv.org/featured-challenges/80/leaderboard',
        'reference': 'https://github.com/jokieleung/awesome-visual-question-answering/blob/master/README.md?plain=1#L439'
    },
    {
        'leaderboard': 'https://evalai.cloudcv.org/featured-challenges/1/leaderboard',
        'reference': 'https://github.com/jokieleung/awesome-visual-question-answering/blob/master/README.md?plain=1#L441'
    },
    {
        'leaderboard': 'https://eval.ai/web/challenges/challenge-page/244/leaderboard',
        'reference': 'https://github.com/jokieleung/awesome-visual-question-answering/blob/master/README.md?plain=1#L445'
    },
    {
        'leaderboard': 'https://github.com/cdancette/vqa-cp-leaderboard',
        'reference': 'https://github.com/jokieleung/awesome-visual-question-answering/blob/master/README.md?plain=1#L449'
    },
    {
        'leaderboard': 'http://host.robots.ox.ac.uk:8080/leaderboard/displaylb.php?challengeid=11&compid=3',
        'reference': 'https://github.com/kuangliu/torchcv/blob/master/README.md?plain=1#L21'
    },
    {
        'leaderboard': 'http://host.robots.ox.ac.uk:8080/leaderboard/displaylb.php?challengeid=11&compid=6',
        'reference': 'https://github.com/TheLegendAli/DeepLab-Context2/blob/master/README.md?plain=1#L61'
    },
    {
        'leaderboard': 'https://rank.opencompass.org.cn',
        'reference': 'https://github.com/open-compass/opencompass/blob/main/README.md?plain=1#L78'
    },
    {
        'leaderboard': 'https://github.com/ku21fan/COO-Comic-Onomatopoeia',
        'reference': ''
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/InstaDeepAI/nucleotide_transformer_benchmark',
        'reference': 'https://github.com/kuleshov-group/caduceus/blob/main/README.md?plain=1#L273'
    },
    {
        'leaderboard': 'https://chartmimic.github.io',
        'reference': 'https://github.com/ChartMimic/ChartMimic/blob/main/README.md?plain=1#L17'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/uonlp/open_multilingual_llm_leaderboard',
        'reference': 'https://github.com/nlp-uoregon/mlmm-evaluation/blob/main/README.md?plain=1#L59'
    },
    {
        'leaderboard': 'https://microsoft.github.io/MSMARCO-Document-Ranking-Submissions/leaderboard',
        'reference': 'https://github.com/thunlp/BERT-KPE/blob/master/README.md?plain=1#L168'
    },
    {
        'leaderboard': 'https://github.com/PJLab-ADG/DriveArena',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/xeneta/LeadQualifier',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/thuml/Time-Series-Library',
        'reference': ''
    },
    {
        'leaderboard': 'https://eval.ai/web/challenges/challenge-page/1897/leaderboard',
        'reference': 'https://github.com/mddunlap924/LangChain-SynData-RAG-Eval/blob/main/README.md?plain=1#L169'
    },
    {
        'leaderboard': 'https://dataset.org/dream',
        'reference': 'https://github.com/nlpdata/dream/blob/master/README.md?plain=1#L18'
    },
    {
        'leaderboard': 'https://dfki-ric-underactuated-lab.github.io/real_ai_gym_leaderboard',
        'reference': 'https://github.com/dfki-ric-underactuated-lab/double_pendulum/blob/main/README.md?plain=1#L180'
    },
    {
        'leaderboard': 'https://github.com/SpeechColab/GigaSpeech',
        'reference': ''
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/yangjianxin1/Firefly/blob/master/README.md?plain=1#L88'
    },
    {
        'leaderboard': 'https://www.kaggle.com/c/landmark-retrieval-2021/leaderboard',
        'reference': 'https://github.com/WesleyZhang1991/Google_Landmark_Retrieval_2021_2nd_Place_Solution/blob/master/README.md?plain=1#L77'
    },
    {
        'leaderboard': 'https://github.com/scottcha/OpenAvalancheProject',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/salesforce/QAConv',
        'reference': ''
    },
    {
        'leaderboard': 'https://www.kaggle.com/competitions/restaurant-revenue-prediction/leaderboard',
        'reference': 'https://github.com/justmarkham/DAT8/blob/master/README.md?plain=1#L477'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/uclaml/SPPO/blob/main/README.md?plain=1#L147'
    },
    {
        'leaderboard': 'https://apolloscape.auto/leader_board.html',
        'reference': 'https://github.com/LincanLi98/Awesome-Data-Centric-Autonomous-Driving/blob/main/README.md?plain=1#L68'
    },
    {
        'leaderboard': 'https://motional.com/news/2023-nuplan-challenge',
        'reference': 'https://github.com/LincanLi98/Awesome-Data-Centric-Autonomous-Driving/blob/main/README.md?plain=1#L107'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/3d-multi-object-tracking-on-nuscenes',
        'reference': 'https://github.com/LincanLi98/Awesome-Data-Centric-Autonomous-Driving/blob/main/README.md?plain=1#L84'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/3d-object-detection-on-kitti-cars-moderate',
        'reference': 'https://github.com/LincanLi98/Awesome-Data-Centric-Autonomous-Driving/blob/main/README.md?plain=1#L84'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/semantic-segmentation-on-cityscapes',
        'reference': 'https://github.com/LincanLi98/Awesome-Data-Centric-Autonomous-Driving/blob/main/README.md?plain=1#L84'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/3d-semantic-segmentation-on-semantickitti',
        'reference': 'https://github.com/LincanLi98/Awesome-Data-Centric-Autonomous-Driving/blob/main/README.md?plain=1#L84'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/video-panoptic-segmentation-on-cityscapes-vps',
        'reference': 'https://github.com/LincanLi98/Awesome-Data-Centric-Autonomous-Driving/blob/main/README.md?plain=1#L84'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/lane-detection-on-culane',
        'reference': 'https://github.com/LincanLi98/Awesome-Data-Centric-Autonomous-Driving/blob/main/README.md?plain=1#L84'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/point-cloud-retrieval-on-oxford-robotcar',
        'reference': 'https://github.com/LincanLi98/Awesome-Data-Centric-Autonomous-Driving/blob/main/README.md?plain=1#L84'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/prediction-of-occupancy-grid-maps-on-occ3d',
        'reference': 'https://github.com/LincanLi98/Awesome-Data-Centric-Autonomous-Driving/blob/main/README.md?plain=1#L84'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/3d-object-tracking-on-argoverse-cvpr-2020',
        'reference': 'https://github.com/LincanLi98/Awesome-Data-Centric-Autonomous-Driving/blob/main/README.md?plain=1#L85'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/traffic-object-detection-on-bdd100k-val',
        'reference': 'https://github.com/LincanLi98/Awesome-Data-Centric-Autonomous-Driving/blob/main/README.md?plain=1#L85'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/multi-object-tracking-and-segmentation-on-3',
        'reference': 'https://github.com/LincanLi98/Awesome-Data-Centric-Autonomous-Driving/blob/main/README.md?plain=1#L85'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/lidar-semantic-segmentation-on-nuscenes',
        'reference': 'https://github.com/LincanLi98/Awesome-Data-Centric-Autonomous-Driving/blob/main/README.md?plain=1#L85'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/lane-detection-on-tusimple',
        'reference': 'https://github.com/LincanLi98/Awesome-Data-Centric-Autonomous-Driving/blob/main/README.md?plain=1#L85'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/multi-object-tracking-on-mot17',
        'reference': 'https://github.com/LincanLi98/Awesome-Data-Centric-Autonomous-Driving/blob/main/README.md?plain=1#L86'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/3d-object-detection-on-waymo-pedestrian',
        'reference': 'https://github.com/LincanLi98/Awesome-Data-Centric-Autonomous-Driving/blob/main/README.md?plain=1#L86'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/semantic-segmentation-on-kitti-360',
        'reference': 'https://github.com/LincanLi98/Awesome-Data-Centric-Autonomous-Driving/blob/main/README.md?plain=1#L86'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/3d-lane-detection-on-openlane',
        'reference': 'https://github.com/LincanLi98/Awesome-Data-Centric-Autonomous-Driving/blob/main/README.md?plain=1#L86'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/3d-object-detection-on-waymo-vehicle',
        'reference': 'https://github.com/LincanLi98/Awesome-Data-Centric-Autonomous-Driving/blob/main/README.md?plain=1#L87'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/lane-detection-on-curvelanes',
        'reference': 'https://github.com/LincanLi98/Awesome-Data-Centric-Autonomous-Driving/blob/main/README.md?plain=1#L87'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/trajectory-prediction-on-apolloscape-1',
        'reference': 'https://github.com/LincanLi98/Awesome-Data-Centric-Autonomous-Driving/blob/main/README.md?plain=1#L96'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/trajectory-prediction-on-nuscenes',
        'reference': 'https://github.com/LincanLi98/Awesome-Data-Centric-Autonomous-Driving/blob/main/README.md?plain=1#L97'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/motion-forecasting-on-argoverse-cvpr-2020',
        'reference': 'https://github.com/LincanLi98/Awesome-Data-Centric-Autonomous-Driving/blob/main/README.md?plain=1#L98'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/trajectory-prediction-on-lyft-level-5',
        'reference': 'https://github.com/LincanLi98/Awesome-Data-Centric-Autonomous-Driving/blob/main/README.md?plain=1#L100'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/autonomous-driving-on-carla-leaderboard',
        'reference': 'https://github.com/LincanLi98/Awesome-Data-Centric-Autonomous-Driving/blob/main/README.md?plain=1#L108'
    },
    {
        'leaderboard': 'https://ogb.stanford.edu/docs/leader_linkprop/#ogbl-wikikg2',
        'reference': 'https://github.com/migalkin/NodePiece/blob/main/readme.md?plain=1#L138'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/themanyone/whisper_dictation/blob/main/README.md?plain=1#L82'
    },
    {
        'leaderboard': 'https://github.com/zeyofu/BLINK_Benchmark',
        'reference': ''
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/mteb/leaderboard',
        'reference': 'https://github.com/thinknimble/embeddings-search-demo/blob/main/README.md?plain=1#L17'
    },
    {
        'leaderboard': 'https://github.com/vztu/BVQA_Benchmark',
        'reference': 'https://github.com/vztu/VIDEVAL/blob/master/README.md?plain=1#L9'
    },
    {
        'leaderboard': 'https://github.com/BinWang28/EvalRank-Embedding-Evaluation',
        'reference': ''
    },
    {
        'leaderboard': 'https://blackboxbench.github.io',
        'reference': 'https://github.com/SCLBD/BlackboxBench/blob/main/README.md?plain=1#L10'
    },
    {
        'leaderboard': 'https://tatsu-lab.github.io/alpaca_eval',
        'reference': 'https://github.com/xfactlab/orpo/blob/main/README.md?plain=1#L8'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/xfactlab/orpo/blob/main/README.md?plain=1#L8'
    },
    {
        'leaderboard': 'https://github.com/QQ-MM/Video-CCAM',
        'reference': ''
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/facial-expression-recognition-on-fer2013',
        'reference': 'https://github.com/LetheSec/Fer2013-Facial-Emotion-Recognition-Pytorch/blob/main/README.md?plain=1#L21'
    },
    {
        'leaderboard': 'https://espere-1119-song.github.io/LOVEU-CVPR-24-Track-1-Leaderboard',
        'reference': 'https://github.com/rese1f/MovieChat/blob/main/README.md?plain=1#L24'
    },
    {
        'leaderboard': 'https://github.com/csdongxian/AWP',
        'reference': ''
    },
    {
        'leaderboard': 'https://visualqa.org/roe.html',
        'reference': 'https://github.com/CCYChongyanChen/VQA_AlgorithmDatasets/blob/master/readme.md?plain=1#L26'
    },
    {
        'leaderboard': 'https://eval.ai/web/challenges/challenge-page/1560/leaderboard',
        'reference': 'https://github.com/CCYChongyanChen/VQA_AlgorithmDatasets/blob/master/readme.md?plain=1#L51'
    },
    {
        'leaderboard': 'https://github.com/neulab/REALSumm',
        'reference': ''
    },
    {
        'leaderboard': 'https://eval.ai/web/challenges/challenge-page/503/leaderboard',
        'reference': 'https://github.com/SiyuanWangw/LReasoner/blob/master/README.md?plain=1#L4'
    },
    {
        'leaderboard': 'https://robustbench.github.io',
        'reference': 'https://github.com/fra31/auto-attack/blob/master/README.md?plain=1#L139'
    },
    {
        'leaderboard': 'https://ai.google.com/research/NaturalQuestions',
        'reference': 'https://github.com/google-research-datasets/natural-questions/blob/master/README.md?plain=1#L9'
    },
    {
        'leaderboard': 'https://github.com/sintel-dev/Orion',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/ParallelDots/generic-sku-detection-benchmark',
        'reference': ''
    },
    {
        'leaderboard': 'https://ogb.stanford.edu/docs/lsc/leaderboards',
        'reference': 'https://github.com/PaddlePaddle/PaddleHelix/blob/dev/README.md?plain=1#L31'
    },
    {
        'leaderboard': 'https://github.com/reka-ai/reka-vibe-eval',
        'reference': ''
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/mallahyari/llm-hub/blob/main/README.md?plain=1#L462'
    },
    {
        'leaderboard': 'https://russiansuperglue.com/leaderboard/2',
        'reference': 'https://github.com/RussianNLP/RussianSuperGLUE/blob/master/README.md?plain=1#L28'
    },
    {
        'leaderboard': 'https://microsoft.github.io/CodeXGLUE',
        'reference': 'https://github.com/microsoft/CodeXGLUE/blob/main/README.md?plain=1#L33'
    },
    {
        'leaderboard': 'https://github.com/fastai/imagenette',
        'reference': ''
    },
    {
        'leaderboard': 'https://ai.google.com/research/rxr/competition',
        'reference': 'https://github.com/google-research-datasets/RxR/blob/main/README.md?plain=1#L128'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/locuslab/tofu_leaderboard',
        'reference': 'https://github.com/locuslab/tofu/blob/main/README.md?plain=1#L11'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/lmsys/chatbot-arena-leaderboard',
        'reference': 'https://github.com/mshojaei77/Awesome-AI/blob/main/README.md?plain=1#L69'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/mshojaei77/Awesome-AI/blob/main/README.md?plain=1#L69'
    },
    {
        'leaderboard': 'https://tatsu-lab.github.io/alpaca_eval',
        'reference': 'https://github.com/CLUEbenchmark/SuperCLUE-Open/blob/main/README.md?plain=1#L269'
    },
    {
        'leaderboard': 'https://microsoft.github.io/XGLUE',
        'reference': 'https://github.com/microsoft/Unicoder/blob/master/README.md?plain=1#L2'
    },
    {
        'leaderboard': 'https://github.com/gjy3035/Awesome-Crowd-Counting',
        'reference': ''
    },
    {
        'leaderboard': 'https://eval.ai/web/challenges/challenge-page/453/leaderboard',
        'reference': 'https://github.com/johnwlambert/argoverse_cbgs_kf_tracker/blob/master/README.md?plain=1#L15'
    },
    {
        'leaderboard': 'https://github.com/zwq2018/Multi-modal-Self-instruct',
        'reference': ''
    },
    {
        'leaderboard': 'https://www.tanksandtemples.org/leaderboard',
        'reference': 'https://github.com/XYZ-qiyh/Awesome-Learning-MVS/blob/main/README.md?plain=1#L132'
    },
    {
        'leaderboard': 'https://proteinshake.ai/#leaderboard',
        'reference': 'https://github.com/BorgwardtLab/proteinshake/blob/main/README.md?plain=1#L24'
    },
    {
        'leaderboard': 'https://stanfordnlp.github.io/coqa',
        'reference': 'https://github.com/microsoft/SDNet/blob/master/README.md?plain=1#L3'
    },
    {
        'leaderboard': 'https://kobiso.github.io/Computer-Vision-Leaderboard/imagenet',
        'reference': 'https://github.com/kobiso/Computer-Vision-Leaderboard/blob/master/README.md?plain=1#L10'
    },
    {
        'leaderboard': 'https://kobiso.github.io/Computer-Vision-Leaderboard/sop',
        'reference': 'https://github.com/kobiso/Computer-Vision-Leaderboard/blob/master/README.md?plain=1#L11'
    },
    {
        'leaderboard': 'https://kobiso.github.io/Computer-Vision-Leaderboard/cars',
        'reference': 'https://github.com/kobiso/Computer-Vision-Leaderboard/blob/master/README.md?plain=1#L12'
    },
    {
        'leaderboard': 'https://kobiso.github.io/Computer-Vision-Leaderboard/cub',
        'reference': 'https://github.com/kobiso/Computer-Vision-Leaderboard/blob/master/README.md?plain=1#L13'
    },
    {
        'leaderboard': 'https://kobiso.github.io/Computer-Vision-Leaderboard/in-shop',
        'reference': 'https://github.com/kobiso/Computer-Vision-Leaderboard/blob/master/README.md?plain=1#L14'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/relbench/leaderboard',
        'reference': 'https://github.com/snap-stanford/relbench/blob/main/README.md?plain=1#L27'
    },
    {
        'leaderboard': 'https://boson.ai/rpbench',
        'reference': 'https://github.com/Neph0s/awesome-llm-role-playing-with-persona/blob/main/README.md?plain=1#L23'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/bigcode/bigcode-models-leaderboard',
        'reference': 'https://github.com/OpenCodeInterpreter/OpenCodeInterpreter/blob/main/README.md?plain=1#L137'
    },
    {
        'leaderboard': 'https://github.com/OpenGVLab/MMT-Bench',
        'reference': ''
    },
    {
        'leaderboard': 'https://open-compass.github.io/MathBench',
        'reference': 'https://github.com/open-compass/MathBench/blob/main/README.md?plain=1#L19'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/teticio/llama-squad/blob/main/README.md?plain=1#L264'
    },
    {
        'leaderboard': 'https://paperswithcode.com/task/entity-alignment',
        'reference': 'https://github.com/THU-KEG/Entity_Alignment_Papers/blob/master/README.md?plain=1#L6'
    },
    {
        'leaderboard': 'https://leaderboard.allenai.org/flex',
        'reference': 'https://github.com/allenai/flex/blob/main/README.md?plain=1#L18'
    },
    {
        'leaderboard': 'https://leaderboard.allenai.org/flex_meta',
        'reference': 'https://github.com/allenai/flex/blob/main/README.md?plain=1#L19'
    },
    {
        'leaderboard': 'https://codalab.lisn.upsaclay.fr/competitions/9121',
        'reference': 'https://github.com/bao18/open_earth_map/blob/main/README.md?plain=1#L55'
    },
    {
        'leaderboard': 'https://cl-detection2023.grand-challenge.org/evaluation/challenge/leaderboard',
        'reference': 'https://github.com/szuboy/CL-Detection2023/blob/main/README.md?plain=1#L12'
    },
    {
        'leaderboard': 'https://leaderboard.carla.org/leaderboard',
        'reference': 'https://github.com/OpenDriveLab/End-to-end-Autonomous-Driving/blob/main/README.md?plain=1#L134'
    },
    {
        'leaderboard': 'https://eval.ai/web/challenges/challenge-page/1856/leaderboard',
        'reference': 'https://github.com/OpenDriveLab/End-to-end-Autonomous-Driving/blob/main/README.md?plain=1#L138'
    },
    {
        'leaderboard': 'https://lamalab-org.github.io/chem-bench/leaderboard',
        'reference': 'https://github.com/materials-data-facility/matchem-llm/blob/main/README.md?plain=1#L21'
    },
    {
        'leaderboard': 'https://hf.co/spaces/mteb/leaderboard',
        'reference': 'https://github.com/neuml/txtai/blob/master/README.md?plain=1#L197'
    },
    {
        'leaderboard': 'https://lmarena.ai/?leaderboard',
        'reference': 'https://github.com/neuml/txtai/blob/master/README.md?plain=1#L198'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/neuml/txtai/blob/master/README.md?plain=1#L199'
    },
    {
        'leaderboard': 'https://ogb.stanford.edu/docs/lsc/leaderboards/#pcqm4mv2',
        'reference': 'https://github.com/graphcore/ogb-lsc-pcqm4mv2/blob/main/README.md?plain=1#L42'
    },
    {
        'leaderboard': 'https://www.cvlibs.net/datasets/kitti/eval_scene_flow.php?benchmark=stereo',
        'reference': 'https://github.com/JiaRenChang/PSMNet/blob/master/README.md?plain=1#L113'
    },
    {
        'leaderboard': 'https://rucola-benchmark.com/leaderboard',
        'reference': 'https://github.com/RussianNLP/RuCoLA/blob/main/README.md?plain=1#L7'
    },
    {
        'leaderboard': 'https://github.com/Kinkelin/WordleCompetition',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/scicode-bench/SciCode',
        'reference': ''
    },
    {
        'leaderboard': 'https://www.cvlibs.net/datasets/kitti/eval_scene_flow.php?benchmark=flow',
        'reference': 'https://github.com/gengshan-y/VCN/blob/master/README.md?plain=1#L22'
    },
    {
        'leaderboard': 'https://waymo.com/open/challenges/2021/real-time-3d-prediction',
        'reference': 'https://github.com/tusen-ai/RangeDet/blob/main/README.md?plain=1#L79'
    },
    {
        'leaderboard': 'https://www.tau-nlp.org/csqa-leaderboard',
        'reference': 'https://github.com/microsoft/KEAR/blob/main/README.md?plain=1#L18'
    },
    {
        'leaderboard': 'https://tatsu-lab.github.io/alpaca_eval',
        'reference': 'https://github.com/01-ai/Yi/blob/main/README.md?plain=1#L136'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/01-ai/Yi/blob/main/README.md?plain=1#L1263'
    },
    {
        'leaderboard': 'https://rank.opencompass.org.cn/leaderboard-llm',
        'reference': 'https://github.com/01-ai/Yi/blob/main/README.md?plain=1#L1265'
    },
    {
        'leaderboard': 'https://dki-lab.github.io/GrailQA',
        'reference': 'https://github.com/microsoft/KC/blob/main/README.md?plain=1#L16'
    },
    {
        'leaderboard': 'https://github.com/ChineseGLUE/ChineseGLUE',
        'reference': ''
    },
    {
        'leaderboard': 'https://leaderboard.allenai.org/alfred',
        'reference': 'https://github.com/askforalfred/alfred/blob/master/README.md?plain=1#L141'
    },
    {
        'leaderboard': 'https://github.com/MozerWang/Loong',
        'reference': ''
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/object-detection-on-coco',
        'reference': 'https://github.com/digantamisra98/Mish/blob/master/README.md?plain=1#L47'
    },
    {
        'leaderboard': 'https://github.com/SpeechColab/Leaderboard',
        'reference': ''
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/optimum/llm-perf-leaderboard',
        'reference': 'https://github.com/horseee/Awesome-Efficient-LLM/blob/main/leaderboard.md?plain=1#L5'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/AI-Secure/llm-trustworthy-leaderboard',
        'reference': 'https://github.com/horseee/Awesome-Efficient-LLM/blob/main/leaderboard.md?plain=1#L6'
    },
    {
        'leaderboard': 'https://github.com/ray-project/llmperf-leaderboard',
        'reference': 'https://github.com/horseee/Awesome-Efficient-LLM/blob/main/leaderboard.md?plain=1#L7'
    },
    {
        'leaderboard': 'https://artificialanalysis.ai/leaderboards/providers',
        'reference': 'https://github.com/horseee/Awesome-Efficient-LLM/blob/main/leaderboard.md?plain=1#L8'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/ml-energy/leaderboard',
        'reference': 'https://github.com/horseee/Awesome-Efficient-LLM/blob/main/leaderboard.md?plain=1#L9'
    },
    {
        'leaderboard': 'https://artificialanalysis.ai/leaderboards/models',
        'reference': 'https://github.com/horseee/Awesome-Efficient-LLM/blob/main/leaderboard.md?plain=1#L10'
    },
    {
        'leaderboard': 'https://leaderboard.withmartian.com',
        'reference': 'https://github.com/horseee/Awesome-Efficient-LLM/blob/main/leaderboard.md?plain=1#L11'
    },
    {
        'leaderboard': 'https://bop.felk.cvut.cz/leaderboards',
        'reference': 'https://github.com/ylabbe/cosypose/blob/master/README.md?plain=1#L51'
    },
    {
        'leaderboard': 'https://github.com/hendrycks/ethics',
        'reference': ''
    },
    {
        'leaderboard': 'https://codalab.lisn.upsaclay.fr/competitions/11784',
        'reference': 'https://github.com/tanfiona/CausalNewsCorpus/blob/master/README.md?plain=1#L36'
    },
    {
        'leaderboard': 'https://www.creatis.insa-lyon.fr/Challenge/acdc/results.html',
        'reference': 'https://github.com/MIC-DKFZ/ACDC2017/blob/master/README.md?plain=1#L9'
    },
    {
        'leaderboard': 'https://github.com/cs-chan/Total-Text-Dataset',
        'reference': ''
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/ntunlp/OpenSource-LLMs-better-than-OpenAI/blob/main/README.md?plain=1#L13'
    },
    {
        'leaderboard': 'https://github.com/THUDM/NaturalCodeBench',
        'reference': ''
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/mlfoundations/VisIT-Bench-Leaderboard',
        'reference': 'https://github.com/mlfoundations/VisIT-Bench/blob/main/README.md?plain=1#L3'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/AudioLLMs/AudioBench-Leaderboard',
        'reference': 'https://github.com/AudioLLMs/AudioLLM/blob/main/README.md?plain=1#L66'
    },
    {
        'leaderboard': 'https://www.kaggle.com/c/rsna-intracranial-hemorrhage-detection/leaderboard',
        'reference': 'https://github.com/darraghdog/rsna/blob/master/README.md?plain=1#L87'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/EleutherAI/lm-evaluation-harness/blob/main/README.md?plain=1#L50'
    },
    {
        'leaderboard': 'https://microsoft.github.io/MSMARCO-Document-Ranking-Submissions/leaderboard',
        'reference': 'https://github.com/castorini/docTTTTTquery/blob/master/README.md?plain=1#L12'
    },
    {
        'leaderboard': 'https://leaderboard.allenai.org',
        'reference': 'https://github.com/allenai/aristo-leaderboard/blob/master/README.md?plain=1#L1'
    },
    {
        'leaderboard': 'https://junjie-chu.github.io/Public_Comprehensive_Assessment_Jailbreak/leaderboard',
        'reference': 'https://github.com/TrustAIRLab/Comprehensive_Jailbreak_Assessment/blob/main/README.md?plain=1#L80'
    },
    {
        'leaderboard': 'https://github.com/LuoweiZhou/YouCook2-Leaderboard',
        'reference': ''
    },
    {
        'leaderboard': 'https://pages.nist.gov/jarvis_leaderboard',
        'reference': 'https://github.com/usnistgov/jarvis_leaderboard/blob/main/README.md?plain=1#L14'
    },
    {
        'leaderboard': 'https://opus.nlpl.eu/dashboard',
        'reference': 'https://github.com/Helsinki-NLP/Tatoeba-Challenge/blob/master/README.md?plain=1#L10'
    },
    {
        'leaderboard': 'https://www.kaggle.com/competitions/2023-gwdata-bootcamp/leaderboard',
        'reference': 'https://github.com/iphysresearch/GWData-Bootcamp/blob/main/README.md?plain=1#L623'
    },
    {
        'leaderboard': 'https://github.com/bcmi/Awesome-Generative-Image-Composition',
        'reference': ''
    },
    {
        'leaderboard': 'https://leaderboard.allenai.org',
        'reference': 'https://github.com/allenai/ai2thor-rearrangement/blob/main/README.md?plain=1#L352'
    },
    {
        'leaderboard': 'https://github.com/AINativeLab/gptstore-data-backup',
        'reference': ''
    },
    {
        'leaderboard': 'https://www.scrolls-benchmark.com/leaderboard',
        'reference': 'https://github.com/tau-nlp/scrolls/blob/main/README.md?plain=1#L7'
    },
    {
        'leaderboard': 'https://github.com/Helsinki-NLP/OPUS-MT-leaderboard',
        'reference': 'https://github.com/Helsinki-NLP/OPUS/blob/main/README.md?plain=1#L44'
    },
    {
        'leaderboard': 'https://paperswithcode.com/dataset/foodseg103',
        'reference': 'https://github.com/LARC-CMU-SMU/FoodSeg103-Benchmark-v1/blob/main/README.md?plain=1#L35'
    },
    {
        'leaderboard': 'https://eval.ai/web/challenges/challenge-page/1697/leaderboard',
        'reference': 'https://github.com/alexa/massive/blob/main/README.md?plain=1#L12'
    },
    {
        'leaderboard': 'https://github.com/shikras/d-cube/tree/main/eval_sota',
        'reference': 'https://github.com/shikras/d-cube/blob/main/README.md?plain=1#L30'
    },
    {
        'leaderboard': 'https://codetlingua.github.io/leaderboard.html',
        'reference': 'https://github.com/Intelligent-CAT-Lab/PLTranslationEmpirical/blob/main/README.md?plain=1#L22'
    },
    {
        'leaderboard': 'https://github.com/eosphoros-ai/Awesome-Text2SQL',
        'reference': ''
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/AILab-CVC/SEED-Bench_Leaderboard',
        'reference': 'https://github.com/AILab-CVC/SEED-Bench/blob/main/README.md?plain=1#L31'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/AGC2024-P/e2e-driving-navsim',
        'reference': 'https://github.com/autonomousvision/navsim/blob/main/README.md?plain=1#L6'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/optimum/llm-perf-leaderboard',
        'reference': 'https://github.com/huggingface/optimum-benchmark/blob/main/README.md?plain=1#L22'
    },
    {
        'leaderboard': 'https://appworld.dev/leaderboard',
        'reference': 'https://github.com/open-thought/system-2-research/blob/main/README.md?plain=1#L318'
    },
    {
        'leaderboard': 'https://docs.google.com/spreadsheets/d/1M801lEpBbKSNwP-vDBkC_pF7LdyGU1f_ufZb_NWNBZQ',
        'reference': 'https://github.com/open-thought/system-2-research/blob/main/README.md?plain=1#L320'
    },
    {
        'leaderboard': 'https://www.kaggle.com/competitions/arc-prize-2024/leaderboard',
        'reference': 'https://github.com/open-thought/system-2-research/blob/main/README.md?plain=1#L321'
    },
    {
        'leaderboard': 'https://gaia-benchmark-leaderboard.hf.space',
        'reference': 'https://github.com/open-thought/system-2-research/blob/main/README.md?plain=1#L323'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/allenai/ZebraLogic',
        'reference': 'https://github.com/open-thought/system-2-research/blob/main/README.md?plain=1#L326'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/bigcode/bigcodebench-leaderboard',
        'reference': 'https://github.com/open-thought/system-2-research/blob/main/README.md?plain=1#L330'
    },
    {
        'leaderboard': 'https://codalab.lisn.upsaclay.fr/competitions/4318',
        'reference': 'https://github.com/amathislab/HOISDF/blob/main/README.md?plain=1#L74'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/vidore/vidore-leaderboard',
        'reference': 'https://github.com/illuin-tech/colpali/blob/main/README.md?plain=1#L14'
    },
    {
        'leaderboard': 'https://ultraeval.openbmb.cn/rank',
        'reference': 'https://github.com/OpenBMB/UltraEval/blob/main/README.md?plain=1#L227'
    },
    {
        'leaderboard': 'https://good.readthedocs.io/en/latest/leaderboard.html',
        'reference': 'https://github.com/divelab/GOOD/blob/GOODv1/README.md?plain=1#L54'
    },
    {
        'leaderboard': 'https://github.com/hendrycks/test',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/google-research/long-range-arena',
        'reference': ''
    },
    {
        'leaderboard': 'https://pages.cs.huji.ac.il/adiyoss-lab/salmon',
        'reference': 'https://github.com/slp-rl/salmon/blob/main/README.md?plain=1#L62'
    },
    {
        'leaderboard': 'https://leaderboard.allenai.org/strategyqa',
        'reference': 'https://github.com/eladsegal/strategyqa/blob/main/README.md?plain=1#L146'
    },
    {
        'leaderboard': 'https://gluebenchmark.com/leaderboard',
        'reference': 'https://github.com/yoshitomo-matsubara/torchdistill/blob/main/README.md?plain=1#L81'
    },
    {
        'leaderboard': 'https://github.com/hendrycks/test',
        'reference': 'https://github.com/openseg-group/openseg.pytorch/blob/master/README.md?plain=1#L37'
    },
    {
        'leaderboard': 'https://github.com/JokerJohn/PRCV-VSLAM-Challenge-2022',
        'reference': ''
    },
    {
        'leaderboard': 'https://www.kaggle.com/c/santander-customer-transaction-prediction/leaderboard',
        'reference': 'https://github.com/KazukiOnodera/Santander-Customer-Transaction-Prediction/blob/master/README.md?plain=1#L3'
    },
    {
        'leaderboard': 'https://github.com/JokerJohn/PRCV-VSLAM-Challenge-2022',
        'reference': 'https://github.com/openseg-group/openseg.pytorch/blob/master/README.md?plain=1#L40'
    },
    {
        'leaderboard': 'https://github.com/MadryLab/mnist_challenge',
        'reference': 'https://github.com/tianzheng4/Distributionally-Adversarial-Attack/blob/master/README.md?plain=1#L26'
    },
    {
        'leaderboard': 'https://github.com/MadryLab/cifar10_challenge',
        'reference': 'https://github.com/tianzheng4/Distributionally-Adversarial-Attack/blob/master/README.md?plain=1#L28'
    },
    {
        'leaderboard': 'https://www.kaggle.com/competitions/llm-merging-competition/leaderboard',
        'reference': 'https://github.com/llm-merging/LLM-Merging/blob/main/README.md?plain=1#L91'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/margsli/merging_competition',
        'reference': 'https://github.com/llm-merging/LLM-Merging/blob/main/README.md?plain=1#L94'
    },
    {
        'leaderboard': 'https://lmarena.ai/?leaderboard',
        'reference': 'https://github.com/tjunlp-lab/Awesome-LLMs-Evaluation-Papers/blob/main/README.md?plain=1#L1333'
    },
    {
        'leaderboard': 'http://www.lavicleva.com/',
        'reference': 'https://github.com/tjunlp-lab/Awesome-LLMs-Evaluation-Papers/blob/main/README.md?plain=1#L1334'
    },
    {
        'leaderboard': 'https://flageval.baai.ac.cn/#/leaderboard',
        'reference': 'https://github.com/tjunlp-lab/Awesome-LLMs-Evaluation-Papers/blob/main/README.md?plain=1#L1335'
    },
    {
        'leaderboard': 'https://crfm.stanford.edu/helm',
        'reference': 'https://github.com/tjunlp-lab/Awesome-LLMs-Evaluation-Papers/blob/main/README.md?plain=1#L1336'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/tjunlp-lab/Awesome-LLMs-Evaluation-Papers/blob/main/README.md?plain=1#L1337'
    },
    {
        'leaderboard': 'https://rank.opencompass.org.cn',
        'reference': 'https://github.com/tjunlp-lab/Awesome-LLMs-Evaluation-Papers/blob/main/README.md?plain=1#L1340'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/upstage/open-ko-llm-leaderboard',
        'reference': 'https://github.com/tjunlp-lab/Awesome-LLMs-Evaluation-Papers/blob/main/README.md?plain=1#L1341'
    },
    {
        'leaderboard': 'https://www.superclueai.com',
        'reference': 'https://github.com/tjunlp-lab/Awesome-LLMs-Evaluation-Papers/blob/main/README.md?plain=1#L1342'
    },
    {
        'leaderboard': 'https://toloka.ai/llm-leaderboard',
        'reference': 'https://github.com/tjunlp-lab/Awesome-LLMs-Evaluation-Papers/blob/main/README.md?plain=1#L1344'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/uonlp/open_multilingual_llm_leaderboard',
        'reference': 'https://github.com/tjunlp-lab/Awesome-LLMs-Evaluation-Papers/blob/main/README.md?plain=1#L1345'
    },
    {
        'leaderboard': 'http://openeval.org.cn/rank',
        'reference': 'https://github.com/tjunlp-lab/Awesome-LLMs-Evaluation-Papers/blob/main/README.md?plain=1#L1346'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/AngoHF/ANGO-Leaderboard',
        'reference': 'https://github.com/tjunlp-lab/Awesome-LLMs-Evaluation-Papers/blob/main/README.md?plain=1#L1347'
    },
    {
        'leaderboard': 'https://cevalbenchmark.com/static/leaderboard.html',
        'reference': 'https://github.com/tjunlp-lab/Awesome-LLMs-Evaluation-Papers/blob/main/README.md?plain=1#L1348'
    },
    {
        'leaderboard': 'http://cgeval.besteasy.com',
        'reference': 'https://github.com/tjunlp-lab/Awesome-LLMs-Evaluation-Papers/blob/main/README.md?plain=1#L1349'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/CoreyMorris/MMLU-by-task-Leaderboard',
        'reference': 'https://github.com/tjunlp-lab/Awesome-LLMs-Evaluation-Papers/blob/main/README.md?plain=1#L1350'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/AILab-CVC/SEED-Bench_Leaderboard',
        'reference': 'https://github.com/tjunlp-lab/Awesome-LLMs-Evaluation-Papers/blob/main/README.md?plain=1#L1352'
    },
    {
        'leaderboard': 'https://super.gluebenchmark.com/leaderboard',
        'reference': 'https://github.com/tjunlp-lab/Awesome-LLMs-Evaluation-Papers/blob/main/README.md?plain=1#L1353'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/qiantong-xu/toolbench-leaderboard',
        'reference': 'https://github.com/tjunlp-lab/Awesome-LLMs-Evaluation-Papers/blob/main/README.md?plain=1#L1354'
    },
    {
        'leaderboard': 'https://github.com/vectara/hallucination-leaderboard',
        'reference': 'https://github.com/tjunlp-lab/Awesome-LLMs-Evaluation-Papers/blob/main/README.md?plain=1#L1355'
    },
    {
        'leaderboard': 'https://tatsu-lab.github.io/alpaca_eval',
        'reference': 'https://github.com/tjunlp-lab/Awesome-LLMs-Evaluation-Papers/blob/main/README.md?plain=1#L1356'
    },
    {
        'leaderboard': 'https://llmbench.ai/agent/data',
        'reference': 'https://github.com/tjunlp-lab/Awesome-LLMs-Evaluation-Papers/blob/main/README.md?plain=1#L1357'
    },
    {
        'leaderboard': 'https://intercode-benchmark.github.io',
        'reference': 'https://github.com/tjunlp-lab/Awesome-LLMs-Evaluation-Papers/blob/main/README.md?plain=1#L1358'
    },
    {
        'leaderboard': 'https://llmbench.ai/safety/data',
        'reference': 'https://github.com/tjunlp-lab/Awesome-LLMs-Evaluation-Papers/blob/main/README.md?plain=1#L1359'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/InstaDeepAI/nucleotide_transformer_benchmark',
        'reference': 'https://github.com/tjunlp-lab/Awesome-LLMs-Evaluation-Papers/blob/main/README.md?plain=1#L1360'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/bigcode/bigcode-models-leaderboard',
        'reference': 'https://github.com/tjunlp-lab/Awesome-LLMs-Evaluation-Papers/blob/main/README.md?plain=1#L1362'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/optimum/llm-perf-leaderboard',
        'reference': 'https://github.com/tjunlp-lab/Awesome-LLMs-Evaluation-Papers/blob/main/README.md?plain=1#L1363'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/mteb/leaderboard',
        'reference': 'https://github.com/xtekky/gpt4local/blob/main/README.md?plain=1#L110'
    },
    {
        'leaderboard': 'https://www.nuscenes.org/object-detection',
        'reference': 'https://github.com/Sense-X/HoP/blob/main/README.md?plain=1#L11'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/loveu-tgve/loveu-tgve-leaderboard',
        'reference': 'https://github.com/showlab/loveu-tgve-2023/blob/main/README.md?plain=1#L5'
    },
    {
        'leaderboard': 'https://acdc.vision.ee.ethz.ch/benchmarks',
        'reference': 'https://github.com/brdav/refign/blob/main/README.md?plain=1#L18'
    },
    {
        'leaderboard': 'https://codalab.lisn.upsaclay.fr/competitions/3783',
        'reference': 'https://github.com/brdav/refign/blob/main/README.md?plain=1#L18'
    },
    {
        'leaderboard': 'https://ml.energy/leaderboard',
        'reference': 'https://github.com/ml-energy/zeus/blob/master/README.md?plain=1#L22'
    },
    {
        'leaderboard': 'https://adamszq.github.io/smp2019ecdt_task1',
        'reference': 'https://github.com/OnionWang/SMP2019-ECDT-NLU/blob/master/README.md?plain=1#L6'
    },
    {
        'leaderboard': 'https://www.cvlibs.net/datasets/kitti/eval_scene_flow.php?benchmark=stereo',
        'reference': 'https://github.com/gangweiX/ACVNet/blob/main/README.md?plain=1#L85'
    },
    {
        'leaderboard': 'https://appworld.dev/leaderboard',
        'reference': 'https://github.com/StonyBrookNLP/appworld/blob/main/README.md?plain=1#L1135'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/Intel/low_bit_open_llm_leaderboard',
        'reference': 'https://github.com/intel/auto-round/blob/main/README.md?plain=1#L19'
    },
    {
        'leaderboard': 'https://www.cvlibs.net/datasets/kitti/eval_stereo_flow.php?benchmark=stereo',
        'reference': 'https://github.com/aeolusguan/NMRF/blob/main/README.md?plain=1#L21'
    },
    {
        'leaderboard': 'https://www.cvlibs.net/datasets/kitti/eval_scene_flow.php?benchmark=stereo',
        'reference': 'https://github.com/aeolusguan/NMRF/blob/main/README.md?plain=1#L21'
    },
    {
        'leaderboard': 'https://www.swebench.com',
        'reference': 'https://github.com/nus-apr/auto-code-rover/blob/main/README.md?plain=1#L29'
    },
    {
        'leaderboard': 'https://github.com/onejune2018/Awesome-LLM-Eval',
        'reference': ''
    },
    {
        'leaderboard': 'https://hf.co/spaces/allenai/reward-bench',
        'reference': 'https://github.com/onejune2018/Awesome-LLM-Eval/blob/main/README.md?plain=1#L117'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/onejune2018/Awesome-LLM-Eval/blob/main/README.md?plain=1#L152'
    },
    {
        'leaderboard': 'https://mceval.github.io/leaderboard.html',
        'reference': 'https://github.com/onejune2018/Awesome-LLM-Eval/blob/main/README.md?plain=1#L238'
    },
    {
        'leaderboard': 'https://llmbench.ai/agent/data',
        'reference': 'https://github.com/onejune2018/Awesome-LLM-Eval/blob/main/README.md?plain=1#L288'
    },
    {
        'leaderboard': 'https://tatsu-lab.github.io/alpaca_eval',
        'reference': 'https://github.com/onejune2018/Awesome-LLM-Eval/blob/main/README.md?plain=1#L289'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/AngoHF/ANGO-Leaderboard',
        'reference': 'https://github.com/onejune2018/Awesome-LLM-Eval/blob/main/README.md?plain=1#L290'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/bigcode/bigcode-models-leaderboard',
        'reference': 'https://github.com/onejune2018/Awesome-LLM-Eval/blob/main/README.md?plain=1#L291'
    },
    {
        'leaderboard': 'https://lmarena.ai/?leaderboard',
        'reference': 'https://github.com/onejune2018/Awesome-LLM-Eval/blob/main/README.md?plain=1#L292'
    },
    {
        'leaderboard': 'http://www.lavicleva.com',
        'reference': 'https://github.com/onejune2018/Awesome-LLM-Eval/blob/main/README.md?plain=1#L293'
    },
    {
        'leaderboard': 'https://cevalbenchmark.com/static/leaderboard.html',
        'reference': 'https://github.com/onejune2018/Awesome-LLM-Eval/blob/main/README.md?plain=1#L294'
    },
    {
        'leaderboard': 'https://flageval.baai.ac.cn/#/leaderboard',
        'reference': 'https://github.com/onejune2018/Awesome-LLM-Eval/blob/main/README.md?plain=1#L295'
    },
    {
        'leaderboard': 'https://github.com/vectara/hallucination-leaderboard',
        'reference': 'https://github.com/onejune2018/Awesome-LLM-Eval/blob/main/README.md?plain=1#L296'
    },
    {
        'leaderboard': 'https://crfm.stanford.edu/helm',
        'reference': 'https://github.com/onejune2018/Awesome-LLM-Eval/blob/main/README.md?plain=1#L297'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/onejune2018/Awesome-LLM-Eval/blob/main/README.md?plain=1#L298'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/optimum/llm-perf-leaderboard',
        'reference': 'https://github.com/onejune2018/Awesome-LLM-Eval/blob/main/README.md?plain=1#L299'
    },
    {
        'leaderboard': 'https://intercode-benchmark.github.io',
        'reference': 'https://github.com/onejune2018/Awesome-LLM-Eval/blob/main/README.md?plain=1#L301'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/CoreyMorris/MMLU-by-task-Leaderboard',
        'reference': 'https://github.com/onejune2018/Awesome-LLM-Eval/blob/main/README.md?plain=1#L305'
    },
    {
        'leaderboard': 'https://rank.opencompass.org.cn',
        'reference': 'https://github.com/onejune2018/Awesome-LLM-Eval/blob/main/README.md?plain=1#L306'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/uonlp/open_multilingual_llm_leaderboard',
        'reference': 'https://github.com/onejune2018/Awesome-LLM-Eval/blob/main/README.md?plain=1#L309'
    },
    {
        'leaderboard': 'https://llmbench.ai/safety',
        'reference': 'https://github.com/onejune2018/Awesome-LLM-Eval/blob/main/README.md?plain=1#L310'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/AILab-CVC/SEED-Bench_Leaderboard',
        'reference': 'https://github.com/onejune2018/Awesome-LLM-Eval/blob/main/README.md?plain=1#L311'
    },
    {
        'leaderboard': 'https://www.superclueai.com',
        'reference': 'https://github.com/onejune2018/Awesome-LLM-Eval/blob/main/README.md?plain=1#L312'
    },
    {
        'leaderboard': 'https://super.gluebenchmark.com/leaderboard',
        'reference': 'https://github.com/onejune2018/Awesome-LLM-Eval/blob/main/README.md?plain=1#L313'
    },
    {
        'leaderboard': 'https://toloka.ai/llm-leaderboard',
        'reference': 'https://github.com/onejune2018/Awesome-LLM-Eval/blob/main/README.md?plain=1#L315'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/qiantong-xu/toolbench-leaderboard',
        'reference': 'https://github.com/onejune2018/Awesome-LLM-Eval/blob/main/README.md?plain=1#L316'
    },
    {
        'leaderboard': 'https://github.com/VRU-NExT/VideoQA',
        'reference': ''
    },
    {
        'leaderboard': 'https://whoiswho.biendata.xyz',
        'reference': 'https://github.com/napsternxg/awesome-scholarly-data-analysis/blob/master/README.md?plain=1#L222'
    },
    {
        'leaderboard': 'https://bop.felk.cvut.cz/leaderboards',
        'reference': 'https://github.com/yuecideng/Misc3D/blob/main/README.md?plain=1#L30'
    },
    {
        'leaderboard': 'https://modelscope.cn/leaderboard/58/ranking?type=free',
        'reference': 'https://github.com/modelscope/evalscope/blob/main/README.md?plain=1#L270'
    },
    {
        'leaderboard': 'https://www.nuscenes.org/object-detection',
        'reference': 'https://github.com/chaytonmin/Awesome-BEV-Perception-Multi-Cameras/blob/main/README.md?plain=1#L104'
    },
    {
        'leaderboard': 'https://github.com/OFA-Sys/AIR-Bench',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/zehuichen123/Graph-DETR4D',
        'reference': ''
    },
    {
        'leaderboard': 'https://leaderboard.carla.org/leaderboard',
        'reference': 'https://github.com/Kin-Zhang/carla-expert/blob/master/README.md?plain=1#L8'
    },
    {
        'leaderboard': 'https://github.com/bcmi/Object-Shadow-Generation-Dataset-DESOBA',
        'reference': ''
    },
    {
        'leaderboard': 'https://good.readthedocs.io/en/latest/leaderboard.html',
        'reference': 'https://github.com/Graph-COM/GSAT/blob/main/README.md?plain=1#L20'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/mteb/leaderboard',
        'reference': 'https://github.com/embeddings-benchmark/mteb/blob/main/README.md?plain=1#L22'
    },
    {
        'leaderboard': 'http://host.robots.ox.ac.uk:8080/leaderboard/displaylb.php?challengeid=11&compid=6',
        'reference': 'https://github.com/SharifAmit/DilatedFCNSegmentation/blob/master/README.md?plain=1#L46'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/mteb/leaderboard',
        'reference': 'https://github.com/orionw/FollowIR/blob/master/README.md?plain=1#L9'
    },
    {
        'leaderboard': 'https://paperswithcode.com/task/object-detection',
        'reference': 'https://github.com/ddlee-cn/Obj_Det_Progress_Tracker/blob/master/README.md?plain=1#L10'
    },
    {
        'leaderboard': 'https://github.com/thuml/TimesNet',
        'reference': ''
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/Intel/powered_by_intel_llm_leaderboard',
        'reference': 'https://github.com/huggingface/optimum-habana/blob/main/README.md?plain=1#L277'
    },
    {
        'leaderboard': 'https://github.com/openphilanthropy/unrestricted-adversarial-examples',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/thu-coai/SafetyBench',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/fixie-ai/ai-benchmarks',
        'reference': ''
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/mteb/leaderboard',
        'reference': 'https://github.com/mrdbourke/rag-resources/blob/main/README.md?plain=1#L80'
    },
    {
        'leaderboard': 'https://github.com/casmlab/NPHardEval',
        'reference': ''
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/GTBench/GTBench',
        'reference': 'https://github.com/jinhaoduan/GTBench/blob/main/README.md?plain=1#L5'
    },
    {
        'leaderboard': 'https://github.com/1x-technologies/1xgpt',
        'reference': ''
    },
    {
        'leaderboard': 'https://www.cvlibs.net/datasets/kitti/eval_depth.php?benchmark=depth_completion',
        'reference': 'https://github.com/JUGGHM/PENet_ICRA2021/blob/main/README.md?plain=1#L27'
    },
    {
        'leaderboard': 'https://leaderboard.allenai.org/a-okvqa',
        'reference': 'https://github.com/MILVLG/prophet/blob/main/README.md?plain=1#L288'
    },
    {
        'leaderboard': 'https://yale-lily.github.io//spider',
        'reference': 'https://github.com/X-LANCE/text2sql-lgesql/blob/main/README.md?plain=1#L83'
    },
    {
        'leaderboard': 'http://backdoorbench.com/leader_cifar10',
        'reference': 'https://github.com/SCLBD/BackdoorBench/blob/main/README.md?plain=1#L79'
    },
    {
        'leaderboard': 'https://cmedbenchmark.llmzoo.com/static/leaderboard.html',
        'reference': 'https://github.com/FreedomIntelligence/CMB/blob/main/README.md?plain=1#L43'
    },
    {
        'leaderboard': 'https://github.com/yaodongyu/TRADES',
        'reference': ''
    },
    {
        'leaderboard': 'http://www.swebench.com',
        'reference': 'https://github.com/princeton-nlp/SWE-bench/blob/main/README.md?plain=1#L30'
    },
    {
        'leaderboard': 'https://junjie-chu.github.io/Public_Comprehensive_Assessment_Jailbreak/leaderboard',
        'reference': 'https://github.com/Junjie-Chu/CJA_Comprehensive_Jailbreak_Assessment/blob/main/README.md?plain=1#L80'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/zhengzangw/awesome-huge-models/blob/main/README.md?plain=1#L55'
    },
    {
        'leaderboard': 'https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/leaderboard',
        'reference': 'https://github.com/predibase/llm_distillation_playbook/blob/main/README.md?plain=1#L114'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/predibase/llm_distillation_playbook/blob/main/README.md?plain=1#L204'
    },
    {
        'leaderboard': 'https://crfm.stanford.edu/helm/lite/latest/#/leaderboard',
        'reference': 'https://github.com/predibase/llm_distillation_playbook/blob/main/README.md?plain=1#L204'
    },
    {
        'leaderboard': 'https://cogdl.ai/grb/leaderboard',
        'reference': 'https://github.com/THUDM/grb/blob/master/README.md?plain=1#L7'
    },
    {
        'leaderboard': 'https://boson.ai/rpbench',
        'reference': 'https://github.com/boson-ai/RPBench-Auto/blob/main/README.md?plain=1#L2'
    },
    {
        'leaderboard': 'https://www.kaggle.com/c/siim-acr-pneumothorax-segmentation/leaderboard',
        'reference': 'https://github.com/earhian/SIIM-ACR-Pneumothorax-Segmentation-5th/blob/master/README.md?plain=1#L2'
    },
    {
        'leaderboard': 'https://visualcommonsense.com/leaderboard',
        'reference': 'https://github.com/jackroos/VL-BERT/blob/master/README.md?plain=1#L154'
    },
    {
        'leaderboard': 'https://nlp.cs.unc.edu/data/jielei/tvqa/tvqa_public_html/leaderboard.html',
        'reference': 'https://github.com/jayleicn/TVQA/blob/master/README.md?plain=1#L10'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/named-entity-recognition-ner-on-conll-2003',
        'reference': 'https://github.com/dsindex/ntagger/blob/master/README.md?plain=1#L539'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/named-entity-recognition-on-conll',
        'reference': 'https://github.com/dsindex/ntagger/blob/master/README.md?plain=1#L556'
    },
    {
        'leaderboard': 'http://deephack.me/leaderboard_hack',
        'reference': 'https://github.com/sld/convai-bot-1337/blob/master/README.md?plain=1#L100'
    },
    {
        'leaderboard': 'https://www.kaggle.com/c/outbrain-click-prediction/leaderboard',
        'reference': 'https://github.com/gabrielspmoreira/kaggle_outbrain_click_prediction_google_cloud_ml_engine/blob/master/README.md?plain=1#L27'
    },
    {
        'leaderboard': 'https://www.tanksandtemples.org/leaderboard',
        'reference': 'https://github.com/whubaichuan/M3VSNet/blob/master/README.md?plain=1#L52'
    },
    {
        'leaderboard': 'https://www.cvlibs.net/datasets/kitti/eval_object.php?obj_benchmark=3d',
        'reference': 'https://github.com/hlesmqh/WS3D/blob/master/README.md?plain=1#L11'
    },
    {
        'leaderboard': 'https://www.kaggle.com/c/homesite-quote-conversion/leaderboard',
        'reference': 'https://github.com/Far0n/kaggle-homesite/blob/master/README.md?plain=1#L2'
    },
    {
        'leaderboard': 'https://github.com/lm-sys/arena-hard-auto',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/lyuqin/HydraNet-WikiSQL/releases/tag/20200207_105347',
        'reference': ''
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/RMT-team/babilong',
        'reference': 'https://github.com/booydar/babilong/blob/main/README.md?plain=1#L7'
    },
    {
        'leaderboard': 'https://ai.google.com/research/rxr/competition?active_tab=leaderboard',
        'reference': 'https://github.com/jacobkrantz/VLN-CE/blob/master/README.md?plain=1#L135'
    },
    {
        'leaderboard': 'https://github.com/Lordog/R-Judge',
        'reference': ''
    },
    {
        'leaderboard': 'https://nlp.cs.washington.edu/xorqa',
        'reference': 'https://github.com/AkariAsai/XORQA/blob/main/README.md?plain=1#L5'
    },
    {
        'leaderboard': 'https://github.com/SpeechColab/GigaSpeech2',
        'reference': ''
    },
    {
        'leaderboard': 'https://www.kaggle.com/c/walmart-recruiting-store-sales-forecasting/leaderboard',
        'reference': 'https://github.com/davidthaler/Walmart_competition_code/blob/master/README.md?plain=1#L21'
    },
    {
        'leaderboard': 'https://microsoft.github.io/task_oriented_dialogue_as_dataflow_synthesis',
        'reference': 'https://github.com/microsoft/task_oriented_dialogue_as_dataflow_synthesis/blob/master/README.md?plain=1#L52'
    },
    {
        'leaderboard': 'https://rank.opencompass.org.cn/leaderboard-llm',
        'reference': 'https://github.com/OrionStarAI/OrionStar-Yi-34B-Chat/blob/main/README.MD?plain=1#L62'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/UpstageAI/evalverse/blob/main/README.md?plain=1#L88'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/upstage/open-ko-llm-leaderboard',
        'reference': 'https://github.com/UpstageAI/evalverse/blob/main/README.md?plain=1#L154'
    },
    {
        'leaderboard': 'https://gluebenchmark.com/leaderboard',
        'reference': 'https://github.com/zphang/bert_on_stilts/blob/master/README.md?plain=1#L9'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/visual-question-answering-on-vip-bench',
        'reference': 'https://github.com/WisconsinAIVision/ViP-LLaVA/blob/main/README.md?plain=1#L10'
    },
    {
        'leaderboard': 'https://github.com/Xiangyu-CAS/Yet-Another-reid-baseline',
        'reference': ''
    },
    {
        'leaderboard': 'https://gorilla.cs.berkeley.edu/leaderboard.html',
        'reference': 'https://github.com/SalesforceAIResearch/xLAM/blob/main/README.md?plain=1#L34'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/upstage/open-ko-llm-leaderboard',
        'reference': 'https://github.com/davidkim205/nox/blob/main/README.md?plain=1#L9'
    },
    {
        'leaderboard': 'https://github.com/vectara/hallucination-leaderboard',
        'reference': ''
    },
    {
        'leaderboard': 'https://hilti-challenge.com/leader-board-2022.html',
        'reference': 'https://github.com/MAVIS-SLAM/OpenMAVIS/blob/master/README.md?plain=1#L29'
    },
    {
        'leaderboard': 'https://multi-swe-bench.github.io',
        'reference': 'https://github.com/NL2Code/CodeR/blob/main/README.md?plain=1#L8'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/lyx97/TempCompass',
        'reference': 'https://github.com/llyx97/TempCompass/blob/main/README.md?plain=1#L40'
    },
    {
        'leaderboard': 'https://mathvision-cuhk.github.io/#leaderboard',
        'reference': 'https://github.com/mathvision-cuhk/MATH-V/blob/main/README.md?plain=1#L15'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/out-of-distribution-detection-on-cifar-100-vs',
        'reference': 'https://github.com/stanislavfort/exploring_the_limits_of_OOD_detection/blob/main/README.md?plain=1#L5'
    },
    {
        'leaderboard': 'https://waymo.com/open/challenges/2020/3d-detection',
        'reference': 'https://github.com/MaxLeo-LMZ/BEVFusion_Learningnote/blob/main/README.md?plain=1#L54'
    },
    {
        'leaderboard': 'https://llm-eval.github.io/pages/leaderboard.html',
        'reference': 'https://github.com/microsoft/promptbench/blob/main/README.md?plain=1#L48'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/SunLemuria/OpenGPTAndBeyond/blob/main/README.md?plain=1#L212'
    },
    {
        'leaderboard': 'https://opencompass.org.cn/leaderboard-llm',
        'reference': 'https://github.com/SunLemuria/OpenGPTAndBeyond/blob/main/README.md?plain=1#L327'
    },
    {
        'leaderboard': 'https://github.com/findalexli/SciGraphQA',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/duncanwp/ClimateBench',
        'reference': ''
    },
    {
        'leaderboard': 'https://pages.nist.gov/jarvis_leaderboard',
        'reference': 'https://github.com/usnistgov/alignn/blob/main/README.md?plain=1#L264'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/ParsBench/leaderboard',
        'reference': 'https://github.com/ParsBench/ParsBench/blob/main/README.md?plain=1#L150'
    },
    {
        'leaderboard': 'https://github.com/mtbench101/mt-bench-101',
        'reference': ''
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/spoken-language-understanding-on-spoken-squad',
        'reference': 'https://github.com/Chia-Hsuan-Lee/Spoken-SQuAD/blob/master/README.md?plain=1#L28'
    },
    {
        'leaderboard': 'https://eval.ai/web/challenges/challenge-page/1897/leaderboard',
        'reference': 'https://github.com/beir-cellar/beir/blob/main/README.md?plain=1#L57'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/mteb/leaderboard',
        'reference': 'https://github.com/magdalenakuhn17/awesome-cheap-llms/blob/main/README.md?plain=1#L33'
    },
    {
        'leaderboard': 'https://cmedbenchmark.llmzoo.com/static/leaderboard.html',
        'reference': 'https://github.com/WangRongsheng/CareGPT/blob/main/README.md?plain=1#L80'
    },
    {
        'leaderboard': 'https://medbench.opencompass.org.cn/leaderboard',
        'reference': 'https://github.com/WangRongsheng/CareGPT/blob/main/README.md?plain=1#L839'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/openlifescienceai/open_medical_llm_leaderboard',
        'reference': 'https://github.com/WangRongsheng/CareGPT/blob/main/README.md?plain=1#L840'
    },
    {
        'leaderboard': 'https://github.com/ZhuiyiTechnology/AutoIE',
        'reference': ''
    },
    {
        'leaderboard': 'https://ymcui.com/cmrc2018',
        'reference': 'https://github.com/ymcui/cmrc2018/blob/master/README.md?plain=1#L23'
    },
    {
        'leaderboard': 'https://jailbreakbench.github.io/#leaderboard',
        'reference': 'https://github.com/JailbreakBench/jailbreakbench/blob/main/README.md?plain=1#L21'
    },
    {
        'leaderboard': 'https://rank.opencompass.org.cn/leaderboard-multimodal',
        'reference': 'https://github.com/shell-nlp/gpt_server/blob/main/README.md?plain=1#L226'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/mteb/leaderboard',
        'reference': 'https://github.com/shell-nlp/gpt_server/blob/main/README.md?plain=1#L258'
    },
    {
        'leaderboard': 'https://github.com/zehuichen123/AutoAlignV2',
        'reference': ''
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/image-classification-on-omnibenchmark',
        'reference': 'https://github.com/ZhangYuanhan-AI/OmniBenchmark/blob/main/README.md?plain=1#L25'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/multi-frame-super-resolution-on-proba-v',
        'reference': 'https://github.com/Suanmd/TR-MISR/blob/master/README.md?plain=1#L82'
    },
    {
        'leaderboard': 'https://rank.opencompass.org.cn/leaderboard-multimodal',
        'reference': 'https://github.com/open-compass/VLMEvalKit/blob/main/README.md?plain=1#L11'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/opencompass/open_vlm_leaderboard',
        'reference': 'https://github.com/open-compass/VLMEvalKit/blob/main/README.md?plain=1#L18'
    },
    {
        'leaderboard': 'https://github.com/cage-challenge/cage-challenge-3',
        'reference': ''
    },
    {
        'leaderboard': 'https://www.kaggle.com/c/epfml-text/leaderboard',
        'reference': 'https://github.com/Wronskia/Sentiment-Analysis-on-Twitter-data/blob/master/README.md?plain=1#L38'
    },
    {
        'leaderboard': 'https://leaderboard.carla.org/leaderboard',
        'reference': 'https://github.com/opendilab/InterFuser/blob/main/README.md?plain=1#L10'
    },
    {
        'leaderboard': 'https://github.com/matsui528/annbench',
        'reference': ''
    },
    {
        'leaderboard': 'https://codalab.lisn.upsaclay.fr/competitions/7088',
        'reference': 'https://github.com/SCNU-RISLAB/MF-MOS/blob/main/README.md?plain=1#L10'
    },
    {
        'leaderboard': 'https://github.com/awsaf49/artifact',
        'reference': ''
    },
    {
        'leaderboard': 'https://asoca.grand-challenge.org/evaluation/challenge/leaderboard',
        'reference': 'https://github.com/JunMa11/SOTA-MedSeg/blob/master/README.md?plain=1#L27'
    },
    {
        'leaderboard': 'https://video-mme.github.io/home_page.html#leaderboard',
        'reference': 'https://github.com/BradyFU/Awesome-Multimodal-Large-Language-Models/blob/main/README.md?plain=1#L41'
    },
    {
        'leaderboard': 'https://github.com/BradyFU/Awesome-Multimodal-Large-Language-Models/tree/Evaluation',
        'reference': 'https://github.com/BradyFU/Awesome-Multimodal-Large-Language-Models/blob/main/README.md?plain=1#L54'
    },
    {
        'leaderboard': 'https://tgb.complexdatalab.com',
        'reference': 'https://github.com/yule-BUAA/DyGLib/blob/master/README.md?plain=1#L10'
    },
    {
        'leaderboard': 'https://matbench.materialsproject.org',
        'reference': 'https://github.com/ppdebreuck/modnet/blob/master/README.md?plain=1#L18'
    },
    {
        'leaderboard': 'https://github.com/THUDM/LongBench',
        'reference': ''
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/facebook/CyberSecEval',
        'reference': 'https://github.com/meta-llama/PurpleLlama/blob/main/README.md?plain=1#L90'
    },
    {
        'leaderboard': 'https://matbench-discovery.materialsproject.org',
        'reference': 'https://github.com/janosh/matbench-discovery/blob/main/readme.md?plain=1#L18'
    },
    {
        'leaderboard': 'https://github.com/fastai/imagenette',
        'reference': 'https://github.com/KeremTurgutlu/self_supervised/blob/main/README.md?plain=1#L179'
    },
    {
        'leaderboard': 'http://host.robots.ox.ac.uk:8080/leaderboard/displaylb.php?challengeid=11&compid=6',
        'reference': 'https://github.com/xmyqsh/deeplab-v2/blob/master/README.md?plain=1#L70'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/filipecalegario/awesome-generative-ai/blob/main/README.md?plain=1#L509'
    },
    {
        'leaderboard': 'https://llm.extractum.io/static/llm-leaderboards',
        'reference': 'https://github.com/filipecalegario/awesome-generative-ai/blob/main/README.md?plain=1#L737'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/allenai/reward-bench',
        'reference': 'https://github.com/filipecalegario/awesome-generative-ai/blob/main/README.md?plain=1#L738'
    },
    {
        'leaderboard': 'https://github.com/vectara/hallucination-leaderboard',
        'reference': 'https://github.com/filipecalegario/awesome-generative-ai/blob/main/README.md?plain=1#L775'
    },
    {
        'leaderboard': 'https://github.com/GAIR-NLP/factool',
        'reference': ''
    },
    {
        'leaderboard': 'https://ogb.stanford.edu/docs/leader_linkprop',
        'reference': 'https://github.com/melifluos/subgraph-sketching/blob/main/README.md?plain=1#L100'
    },
    {
        'leaderboard': 'https://github.com/TIGER-AI-Lab/MMLU-Pro',
        'reference': ''
    },
    {
        'leaderboard': 'https://destwang.github.io/CTC2021-explorer',
        'reference': 'https://github.com/destwang/CTC2021/blob/main/README.md?plain=1#L33'
    },
    {
        'leaderboard': 'https://eval.ai/web/challenges/challenge-page/830/leaderboard',
        'reference': 'https://github.com/WillDreamer/Aurora/blob/main/README.md?plain=1#L45'
    },
    {
        'leaderboard': 'https://rank.opencompass.org.cn/leaderboard-multimodal',
        'reference': 'https://github.com/YunxinLi/LingCloud/blob/main/README.md?plain=1#L22'
    },
    {
        'leaderboard': 'https://www.kaggle.com/c/Kannada-MNIST/leaderboard',
        'reference': 'https://github.com/vinayprabhu/Kannada_MNIST/blob/master/README.md?plain=1#L8'
    },
    {
        'leaderboard': 'https://www.cs.ubc.ca/research/image-matching-challenge/2021/leaderboard/',
        'reference': 'https://github.com/ubc-vision/image-matching-benchmark/blob/master/README.md?plain=1#L14'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/q-future/Q-Bench-Leaderboard',
        'reference': 'https://github.com/Q-Future/Q-Bench/blob/master/README.md?plain=1#L134'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/openlifescienceai/open_medical_llm_leaderboard',
        'reference': 'https://github.com/burglarhobbit/Awesome-Medical-Large-Language-Models/blob/main/README.md?plain=1#L6'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/bigcode/bigcodebench-leaderboard',
        'reference': 'https://github.com/huggingface/blog/blob/main/leaderboard-bigcodebench.md?plain=1#L104'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/hf-vision/object_detection_leaderboard',
        'reference': 'https://github.com/huggingface/blog/blob/main/object-detection-leaderboard.md?plain=1#L16'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/huggingface/blog/blob/main/open-llm-leaderboard-drop.md?plain=1#L15'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/huggingface/blog/blob/main/open-llm-leaderboard-mmlu.md?plain=1#L14'
    },
    {
        'leaderboard': 'https://lmarena.ai/?leaderboard',
        'reference': 'https://github.com/huggingface/blog/blob/main/open-llm-leaderboard-rlhf.md?plain=1#L40'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/huggingface/blog/blob/main/open-llm-leaderboard-rlhf.md?plain=1#L42'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/OALL/Open-Arabic-LLM-Leaderboard',
        'reference': 'https://github.com/huggingface/blog/blob/main/leaderboard-arabic.md?plain=1#L105'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/ArtificialAnalysis/LLM-Performance-Leaderboard',
        'reference': 'https://github.com/huggingface/blog/blob/main/leaderboard-artificial-analysis.md?plain=1#L22'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/ArtificialAnalysis/Text-to-Image-Leaderboard',
        'reference': 'https://github.com/huggingface/blog/blob/main/leaderboard-artificial-analysis2.md?plain=1#L21'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/OpenGenAI/parti-prompts-leaderboard',
        'reference': 'https://github.com/huggingface/blog/blob/main/leaderboard-artificial-analysis2.md?plain=1#L53'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/ucla-contextual/contextual_leaderboard',
        'reference': 'https://github.com/huggingface/blog/blob/main/leaderboard-contextual.md?plain=1#L29'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/logikon/open_cot_leaderboard',
        'reference': 'https://github.com/huggingface/blog/blob/main/leaderboard-cot.md?plain=1#L176'
    },
    {
        'leaderboard': 'https://decodingtrust.github.io/leaderboard',
        'reference': 'https://github.com/huggingface/blog/blob/main/leaderboard-decodingtrust.md?plain=1#L20'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/AI-Secure/llm-trustworthy-leaderboard',
        'reference': 'https://github.com/huggingface/blog/blob/main/leaderboard-decodingtrust.md?plain=1#L24'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/HaizeLabs/red-teaming-resistance-benchmark',
        'reference': 'https://github.com/huggingface/blog/blob/main/leaderboard-haizelab.md?plain=1#L23'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/hallucinations-leaderboard/leaderboard',
        'reference': 'https://github.com/huggingface/blog/blob/main/leaderboard-hallucinations.md?plain=1#L23'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/huggingface/blog/blob/main/leaderboard-hebrew.md?plain=1#L50'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/hebrew-llm-leaderboard/leaderboard',
        'reference': 'https://github.com/huggingface/blog/blob/main/leaderboard-hebrew.md?plain=1#L55'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/livecodebench/leaderboard',
        'reference': 'https://github.com/huggingface/blog/blob/main/leaderboard-livecodebench.md?plain=1#L22'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/facebook/CyberSecEval',
        'reference': 'https://github.com/huggingface/blog/blob/main/leaderboard-llamaguard.md?plain=1#L56'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/openlifescienceai/open_medical_llm_leaderboard',
        'reference': 'https://github.com/huggingface/blog/blob/main/leaderboard-medicalllm.md?plain=1#L162'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/NPHardEval/NPHardEval-leaderboard',
        'reference': 'https://github.com/huggingface/blog/blob/main/leaderboard-nphardeval.md?plain=1#L16'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/PatronusAI/leaderboard',
        'reference': 'https://github.com/huggingface/blog/blob/main/leaderboard-patronus.md?plain=1#L14'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/upstage/open-ko-llm-leaderboard',
        'reference': 'https://github.com/huggingface/blog/blob/main/leaderboard-upstage.md?plain=1#L15'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard',
        'reference': 'https://github.com/huggingface/blog/blob/main/leaderboard-vectara.md?plain=1#L13'
    },
    {
        'leaderboard': 'https://epic-kitchens.github.io/2024',
        'reference': 'https://github.com/fpv-iplab/rulstm/blob/master/README.md?plain=1#L178'
    },
    {
        'leaderboard': 'https://leaderboard.allenai.org/vidsitu-verbs',
        'reference': 'https://github.com/TheShadow29/VidSitu/blob/main/README.md?plain=1#L126'
    },
    {
        'leaderboard': 'https://leaderboard.allenai.org/vidsitu-semantic-roles',
        'reference': 'https://github.com/TheShadow29/VidSitu/blob/main/README.md?plain=1#L127'
    },
    {
        'leaderboard': 'https://leaderboard.allenai.org/vidsitu-event-relations',
        'reference': 'https://github.com/TheShadow29/VidSitu/blob/main/README.md?plain=1#L128'
    },
    {
        'leaderboard': 'https://github.com/LuoweiZhou/YouCook2-Leaderboard',
        'reference': 'https://github.com/salesforce/densecap/blob/master/README.md?plain=1#L108'
    },
    {
        'leaderboard': 'https://3dmatch.cs.princeton.edu/#keypoint-matching-benchmark',
        'reference': 'https://github.com/andyzeng/3dmatch-toolbox/blob/master/README.md?plain=1#L249'
    },
    {
        'leaderboard': 'https://github.com/APEXLAB/CodeApex',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/bboylyg/ABL',
        'reference': ''
    },
    {
        'leaderboard': 'https://microsoft.github.io/glge',
        'reference': 'https://github.com/microsoft/glge/blob/master/README.md?plain=1#L58'
    },
    {
        'leaderboard': 'https://gluebenchmark.com/leaderboard',
        'reference': 'https://github.com/airaria/TextBrewer/blob/master/README.md?plain=1#L75'
    },
    {
        'leaderboard': 'https://github.com/openai/gym/wiki/Leaderboard#pendulum-v0',
        'reference': 'https://github.com/msinto93/D4PG/blob/master/README.md?plain=1#L12'
    },
    {
        'leaderboard': 'https://ogb.stanford.edu/kddcup2021/results',
        'reference': 'https://github.com/PaddlePaddle/PGL/blob/main/README.md?plain=1#L29'
    },
    {
        'leaderboard': 'https://leaderboard.allenai.org/grit-ablation-restricted',
        'reference': 'https://github.com/allenai/grit_official/blob/main/README.md?plain=1#L93'
    },
    {
        'leaderboard': 'https://leaderboard.allenai.org/grit-ablation-unrestricted',
        'reference': 'https://github.com/allenai/grit_official/blob/main/README.md?plain=1#L94'
    },
    {
        'leaderboard': 'https://leaderboard.allenai.org/grit-test-restricted',
        'reference': 'https://github.com/allenai/grit_official/blob/main/README.md?plain=1#L95'
    },
    {
        'leaderboard': 'https://leaderboard.allenai.org/grit-test-unrestricted',
        'reference': 'https://github.com/allenai/grit_official/blob/main/README.md?plain=1#L96'
    },
    {
        'leaderboard': 'https://www.kaggle.com/c/dogs-vs-cats/leaderboard',
        'reference': 'https://github.com/amitrajitbose/cat-v-dog-classifier-pytorch/blob/master/README.md?plain=1#L32'
    },
    {
        'leaderboard': 'https://hfl-rc.github.io/cmrc2022/results',
        'reference': 'https://github.com/iflytek/HFL-Anthology/blob/master/README.md?plain=1#L76'
    },
    {
        'leaderboard': 'https://destwang.github.io/CTC2021-explorer',
        'reference': 'https://github.com/iflytek/HFL-Anthology/blob/master/README.md?plain=1#L77'
    },
    {
        'leaderboard': 'https://hfl-rc.github.io/cmrc2019/leaderboard',
        'reference': 'https://github.com/iflytek/HFL-Anthology/blob/master/README.md?plain=1#L79'
    },
    {
        'leaderboard': 'https://hfl-rc.github.io/cmrc2018/leaderboard',
        'reference': 'https://github.com/iflytek/HFL-Anthology/blob/master/README.md?plain=1#L81'
    },
    {
        'leaderboard': 'https://hfl-rc.github.io/cmrc2017/leaderboard',
        'reference': 'https://github.com/iflytek/HFL-Anthology/blob/master/README.md?plain=1#L82'
    },
    {
        'leaderboard': 'https://www.kaggle.com/c/deepfake-detection-challenge/leaderboard',
        'reference': 'https://github.com/polimi-ispl/icpr2020dfdc/blob/master/README.md?plain=1#L18'
    },
    {
        'leaderboard': 'https://github.com/AIFlames/Flames',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/apple/ml-mkqa',
        'reference': ''
    },
    {
        'leaderboard': 'https://www.nuscenes.org/lidar-segmentation',
        'reference': 'https://github.com/wzzheng/TPVFormer/blob/main/README.md?plain=1#L2'
    },
    {
        'leaderboard': 'https://ibug.doc.ic.ac.uk/resources/fg-2020-competition-affective-behavior-analysis',
        'reference': 'https://github.com/wtomin/Multitask-Emotion-Recognition-with-Incomplete-Labels/blob/master/README.md?plain=1#L10'
    },
    {
        'leaderboard': 'https://www.kaggle.com/c/landmark-recognition-2020/leaderboard',
        'reference': 'https://github.com/psinger/kaggle-landmark-recognition-2020-1st-place/blob/main/README.md?plain=1#L4'
    },
    {
        'leaderboard': 'https://github.com/JUNJIE99/MLVU',
        'reference': 'https://github.com/DAMO-NLP-SG/VideoLLaMA2/blob/main/README.md?plain=1#L47'
    },
    {
        'leaderboard': 'https://video-mme.github.io/home_page.html#leaderboard',
        'reference': 'https://github.com/DAMO-NLP-SG/VideoLLaMA2/blob/main/README.md?plain=1#L48'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/homebrewltd/awesome-local-ai/blob/main/README.md?plain=1#L112'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/lmsys/chatbot-arena-leaderboard',
        'reference': 'https://github.com/homebrewltd/awesome-local-ai/blob/main/README.md?plain=1#L113'
    },
    {
        'leaderboard': 'https://tatsu-lab.github.io/alpaca_eval',
        'reference': 'https://github.com/homebrewltd/awesome-local-ai/blob/main/README.md?plain=1#L114'
    },
    {
        'leaderboard': 'https://llm-leaderboard.streamlit.app/',
        'reference': 'https://github.com/homebrewltd/awesome-local-ai/blob/main/README.md?plain=1#L115'
    },
    {
        'leaderboard': 'https://lmarena.ai/?leaderboard',
        'reference': 'https://github.com/homebrewltd/awesome-local-ai/blob/main/README.md?plain=1#L116'
    },
    {
        'leaderboard': 'https://bop.felk.cvut.cz/leaderboards/segmentation-unseen-bop23/core-datasets',
        'reference': 'https://github.com/YoungSean/NIDS-Net/blob/main/README.md?plain=1#L23'
    },
    {
        'leaderboard': 'https://gluebenchmark.com/leaderboard',
        'reference': 'https://github.com/Y1ran/NLP-BERT--ChineseVersion/blob/master/README.md?plain=1#L201'
    },
    {
        'leaderboard': 'https://github.com/JialeCao001/PedSurvey/blob/master/Comparison.md',
        'reference': ''
    },
    {
        'leaderboard': 'https://opendrivelab.com/challenge2023',
        'reference': 'https://github.com/CVPR2023-3D-Occupancy-Prediction/CVPR2023-3D-Occupancy-Prediction/blob/main/README.md?plain=1#L45'
    },
    {
        'leaderboard': 'https://visualcommonsense.com/leaderboard',
        'reference': 'https://github.com/yuweijiang/HGL-pytorch/blob/master/README.md?plain=1#L78'
    },
    {
        'leaderboard': 'https://mathverse-cuhk.github.io/#leaderboard',
        'reference': 'https://github.com/ZrrSkywalker/MathVerse/blob/main/README.md?plain=1#L17'
    },
    {
        'leaderboard': 'http://nlpprogress.com/english/sentiment_analysis.html',
        'reference': 'https://github.com/xwzhong/papernote/blob/master/leaderboard.md?plain=1#L2'
    },
    {
        'leaderboard': 'https://nlp.stanford.edu/projects/snli',
        'reference': 'https://github.com/xwzhong/papernote/blob/master/leaderboard.md?plain=1#L8'
    },
    {
        'leaderboard': 'https://stanfordnlp.github.io/coqa',
        'reference': 'https://github.com/xwzhong/papernote/blob/master/leaderboard.md?plain=1#L11'
    },
    {
        'leaderboard': 'https://rajpurkar.github.io/SQuAD-explorer',
        'reference': 'https://github.com/xwzhong/papernote/blob/master/leaderboard.md?plain=1#L12'
    },
    {
        'leaderboard': 'http://quac.ai',
        'reference': 'https://github.com/xwzhong/papernote/blob/master/leaderboard.md?plain=1#L13'
    },
    {
        'leaderboard': 'https://aclweb.org/aclwiki/Question_Answering_(State_of_the_art)',
        'reference': 'https://github.com/xwzhong/papernote/blob/master/leaderboard.md?plain=1#L16'
    },
    {
        'leaderboard': 'https://gluebenchmark.com/leaderboard',
        'reference': 'https://github.com/xwzhong/papernote/blob/master/leaderboard.md?plain=1#L19'
    },
    {
        'leaderboard': 'https://leaderboard.allenai.org/swag',
        'reference': 'https://github.com/xwzhong/papernote/blob/master/leaderboard.md?plain=1#L20'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/hkust-nlp/deita/blob/main/README.md?plain=1#L312'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota',
        'reference': 'https://github.com/eastmountyxz/AI-Security-Paper/blob/main/README.md?plain=1#L110'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota',
        'reference': 'https://github.com/FangjinhuaWang/PatchmatchNet/blob/main/README.md?plain=1#L113'
    },
    {
        'leaderboard': 'https://www.tanksandtemples.org/leaderboard',
        'reference': 'https://github.com/FangjinhuaWang/PatchmatchNet/blob/main/README.md?plain=1#L113'
    },
    {
        'leaderboard': 'https://github.com/Alab-NII/2wikimultihop',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/svilupp/Julia-LLM-Leaderboard',
        'reference': ''
    },
    {
        'leaderboard': 'https://www.cluebenchmarks.com',
        'reference': 'https://github.com/CLUEbenchmark/PyCLUE/blob/master/README.md?plain=1#L11'
    },
    {
        'leaderboard': 'https://github.com/MikeGu721/XiezhiBenchmark',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/allenai/CommonGen-Eval',
        'reference': ''
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/person-re-identification-on-sysu-30k',
        'reference': 'https://github.com/wanggrun/triplet/blob/main/README.md?plain=1#L16'
    },
    {
        'leaderboard': 'http://www.robustvision.net/leaderboard.php',
        'reference': 'https://github.com/princeton-vl/RAFT-Stereo/blob/main/README.md?plain=1#L77'
    },
    {
        'leaderboard': 'https://github.com/ymcui/Eval-on-NN-of-RC',
        'reference': ''
    },
    {
        'leaderboard': 'https://wilds.stanford.edu/leaderboard',
        'reference': 'https://github.com/facebookresearch/DomainBed/blob/main/README.md?plain=1#L65'
    },
    {
        'leaderboard': 'https://github.com/layumi/AICIty-reID-2020',
        'reference': ''
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/mteb/leaderboard',
        'reference': 'https://github.com/underlines/awesome-ml/blob/master/llm-tools.md?plain=1#L144'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/underlines/awesome-ml/blob/master/llm-tools.md?plain=1#L731'
    },
    {
        'leaderboard': 'https://lmarena.ai/?leaderboard',
        'reference': 'https://github.com/underlines/awesome-ml/blob/master/llm-tools.md?plain=1#L732'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota',
        'reference': 'https://github.com/underlines/awesome-ml/blob/master/llm-tools.md?plain=1#L733'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/mike-ravkine/can-ai-code-results',
        'reference': 'https://github.com/underlines/awesome-ml/blob/master/llm-tools.md?plain=1#L734'
    },
    {
        'leaderboard': 'https://cevalbenchmark.com/static/leaderboard.html',
        'reference': 'https://github.com/underlines/awesome-ml/blob/master/llm-tools.md?plain=1#L735'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/mteb/leaderboard',
        'reference': 'https://github.com/underlines/awesome-ml/blob/master/llm-tools.md?plain=1#L736'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/vectara/leaderboard',
        'reference': 'https://github.com/underlines/awesome-ml/blob/master/llm-tools.md?plain=1#L737'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/bigcode/bigcode-models-leaderboard',
        'reference': 'https://github.com/underlines/awesome-ml/blob/master/llm-tools.md?plain=1#L738'
    },
    {
        'leaderboard': 'https://evalplus.github.io/leaderboard.html',
        'reference': 'https://github.com/underlines/awesome-ml/blob/master/llm-tools.md?plain=1#L739'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/PatronusAI/enterprise_scenarios_leaderboard',
        'reference': 'https://github.com/underlines/awesome-ml/blob/master/llm-tools.md?plain=1#L740'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/NPHardEval/NPHardEval-leaderboard',
        'reference': 'https://github.com/underlines/awesome-ml/blob/master/llm-tools.md?plain=1#L741'
    },
    {
        'leaderboard': 'https://prollm.toqan.ai/leaderboard',
        'reference': 'https://github.com/underlines/awesome-ml/blob/master/llm-tools.md?plain=1#L742'
    },
    {
        'leaderboard': 'https://rank.opencompass.org.cn',
        'reference': 'https://github.com/underlines/awesome-ml/blob/master/llm-tools.md?plain=1#L743'
    },
    {
        'leaderboard': 'https://scale.com/leaderboard',
        'reference': 'https://github.com/underlines/awesome-ml/blob/master/llm-tools.md?plain=1#L745'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/AIR-Bench/leaderboard',
        'reference': 'https://github.com/underlines/awesome-ml/blob/master/llm-tools.md?plain=1#L746'
    },
    {
        'leaderboard': 'https://huggingface.co/collections/clefourrier/leaderboards-and-benchmarks-64f99d2e11e92ca5568a7cce',
        'reference': 'https://github.com/underlines/awesome-ml/blob/master/llm-tools.md?plain=1#L747'
    },
    {
        'leaderboard': 'https://gorilla.cs.berkeley.edu/leaderboard.html',
        'reference': 'https://github.com/underlines/awesome-ml/blob/master/llm-tools.md?plain=1#L748'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/WildVision/vision-arena',
        'reference': 'https://github.com/underlines/awesome-ml/blob/master/llm-tools.md?plain=1#L749'
    },
    {
        'leaderboard': 'https://aider.chat/docs/leaderboards',
        'reference': 'https://github.com/underlines/awesome-ml/blob/master/llm-tools.md?plain=1#L750'
    },
    {
        'leaderboard': 'https://evalplus.github.io/repoqa.html',
        'reference': 'https://github.com/underlines/awesome-ml/blob/master/llm-tools.md?plain=1#L751'
    },
    {
        'leaderboard': 'https://bigcode-bench.github.io',
        'reference': 'https://github.com/underlines/awesome-ml/blob/master/llm-tools.md?plain=1#L752'
    },
    {
        'leaderboard': 'https://www.vellum.ai/llm-leaderboard',
        'reference': 'https://github.com/underlines/awesome-ml/blob/master/llm-tools.md?plain=1#L753'
    },
    {
        'leaderboard': 'https://eqbench.com',
        'reference': 'https://github.com/underlines/awesome-ml/blob/master/llm-tools.md?plain=1#L754'
    },
    {
        'leaderboard': 'https://oobabooga.github.io/benchmark.html',
        'reference': 'https://github.com/underlines/awesome-ml/blob/master/llm-tools.md?plain=1#L755'
    },
    {
        'leaderboard': 'https://livebench.ai',
        'reference': 'https://github.com/underlines/awesome-ml/blob/master/llm-tools.md?plain=1#L756'
    },
    {
        'leaderboard': 'https://llm-aggrefact.github.io',
        'reference': 'https://github.com/Liyan06/MiniCheck/blob/main/README.md?plain=1#L20'
    },
    {
        'leaderboard': 'https://sites.google.com/view/englishasrchallenge/leaderboard',
        'reference': 'https://github.com/AI4Bharat/NPTEL2020-Indian-English-Speech-Dataset/blob/master/README.md?plain=1#L79'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/MM-UPD/MM-UPD_Leaderboard',
        'reference': 'https://github.com/AtsuMiyai/UPD/blob/main/README.md?plain=1#L2'
    },
    {
        'leaderboard': 'https://codalab.lisn.upsaclay.fr/competitions',
        'reference': 'https://github.com/guyfe/LongSumm/blob/master/README.md?plain=1#L205'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/jax-diffusers-event/leaderboard',
        'reference': 'https://github.com/fast-codi/CoDi/blob/master/README.md?plain=1#L60'
    },
    {
        'leaderboard': 'https://dc-bench.github.io',
        'reference': 'https://github.com/justincui03/dc_benchmark/blob/main/README.md?plain=1#L13'
    },
    {
        'leaderboard': 'https://codalab.lisn.upsaclay.fr/competitions/8582',
        'reference': 'https://github.com/czyssrs/ConvFinQA/blob/main/README.md?plain=1#L64'
    },
    {
        'leaderboard': 'https://predibase.com/fine-tuning-index',
        'reference': 'https://github.com/keigohtr/state-of-mlops/blob/main/README.md?plain=1#L289'
    },
    {
        'leaderboard': 'https://github.com/YJiangcm/FollowBench',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/tatsu-lab/alpaca_eval',
        'reference': ''
    },
    {
        'leaderboard': 'https://gorilla.cs.berkeley.edu/leaderboard.html',
        'reference': 'https://github.com/MeetKai/functionary/blob/main/README.md?plain=1#L18'
    },
    {
        'leaderboard': 'https://www.kaggle.com/c/allstate-claims-severity/leaderboard',
        'reference': 'https://github.com/adornes/spark_scala_ml_examples/blob/master/README.md?plain=1#L320'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/gaia-benchmark/leaderboard',
        'reference': 'https://github.com/aymeric-roucher/GAIA/blob/main/README.md?plain=1#L7'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/EvolvingLMMs-Lab/lmms-eval/blob/main/README.md?plain=1#L43'
    },
    {
        'leaderboard': 'https://www.cvlibs.net/datasets/kitti',
        'reference': 'https://github.com/philbort/awesome-self-driving-cars/blob/master/README.md?plain=1#L23'
    },
    {
        'leaderboard': 'https://github.com/YLab-Open/METS-CoV',
        'reference': ''
    },
    {
        'leaderboard': 'https://www.cvlibs.net/datasets/kitti/eval_scene_flow.php?benchmark=stereo',
        'reference': 'https://github.com/XiandaGuo/OpenStereo/blob/v2/README.md?plain=1#L11'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/Intel/powered_by_intel_llm_leaderboard',
        'reference': 'https://github.com/huggingface/optimum-intel/blob/main/README.md?plain=1#L194'
    },
    {
        'leaderboard': 'https://rank.opencompass.org.cn',
        'reference': 'https://github.com/InternLM/InternLM/blob/main/README.md?plain=1#L128'
    },
    {
        'leaderboard': 'https://yuzuai.jp/benchmark',
        'reference': 'https://github.com/yuzu-ai/japanese-llm-ranking/blob/main/readme.md?plain=1#L10'
    },
    {
        'leaderboard': 'https://github.com/wanggrun/SYSU-30k',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/AI-secure/Certified-Robustness-SoK-Oldver',
        'reference': 'https://github.com/AI-secure/VeriGauge/blob/master/readme.md?plain=1#L7'
    },
    {
        'leaderboard': 'https://github.com/TianxingChen/RoboTwin',
        'reference': ''
    },
    {
        'leaderboard': 'https://www.kaggle.com/c/sentiment-analysis-on-movie-reviews/leaderboard',
        'reference': 'https://github.com/rafacarrascosa/samr/blob/develop/README.md?plain=1#L42'
    },
    {
        'leaderboard': 'https://microsoft.github.io/MSMARCO-Document-Ranking-Submissions/leaderboard',
        'reference': 'https://github.com/Albert-Ma/PROP/blob/main/README.md?plain=1#L11'
    },
    {
        'leaderboard': 'https://www.nuscenes.org/object-detection',
        'reference': 'https://github.com/Sense-X/Co-DETR/blob/main/README.md?plain=1#L23'
    },
    {
        'leaderboard': 'https://github.com/incognite-lab/myGym',
        'reference': ''
    },
    {
        'leaderboard': 'https://www.kaggle.com/c/pycon-2015-tutorial/leaderboard',
        'reference': 'https://github.com/justmarkham/kaggle-pycon-2015/blob/master/README.md?plain=1#L9'
    },
    {
        'leaderboard': 'https://yale-lily.github.io/spider',
        'reference': 'https://github.com/taoyds/test-suite-sql-eval/blob/master/README.md?plain=1#L21'
    },
    {
        'leaderboard': 'https://yale-lily.github.io/sparc',
        'reference': 'https://github.com/taoyds/test-suite-sql-eval/blob/master/README.md?plain=1#L21'
    },
    {
        'leaderboard': 'https://yale-lily.github.io/cosql',
        'reference': 'https://github.com/taoyds/test-suite-sql-eval/blob/master/README.md?plain=1#L21'
    },
    {
        'leaderboard': 'https://github.com/Yale-LILY/dart',
        'reference': ''
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/mteb/leaderboard',
        'reference': 'https://github.com/michaelfeil/infinity/blob/main/README.md?plain=1#L303'
    },
    {
        'leaderboard': 'https://spokenwoz.github.io',
        'reference': 'https://github.com/bansky-cl/tods-arxiv-daily-paper/blob/main/README.md?plain=1#L73'
    },
    {
        'leaderboard': 'https://decisionintelligence.github.io/OpenTS',
        'reference': 'https://github.com/decisionintelligence/TFB/blob/master/README.md?plain=1#L18'
    },
    {
        'leaderboard': 'https://henrychur.github.io/MultilingualMedQA',
        'reference': 'https://github.com/MAGIC-AI4Med/MMedLM/blob/main/README.md?plain=1#L16'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/Vchitect/VBench_Leaderboard',
        'reference': 'https://github.com/harpreetsahota204/awesome-cvpr-2024/blob/main/README.md?plain=1#L78'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/code-generation-on-humaneval',
        'reference': 'https://github.com/samholt/L2MAC/blob/master/README.md?plain=1#L143'
    },
    {
        'leaderboard': 'https://waymo.com/open/challenges/2021/real-time-3d-prediction',
        'reference': 'https://github.com/happinesslz/SEED/blob/main/README.md?plain=1#L49'
    },
    {
        'leaderboard': 'https://datahack.analyticsvidhya.com/contest/amexpert-2019-machine-learning-hackathon/#LeaderBoard',
        'reference': 'https://github.com/rajat5ranjan/AV-AmExpert-2019-ML-Hackathon/blob/master/README.md?plain=1#L153'
    },
    {
        'leaderboard': 'https://matbench.materialsproject.org',
        'reference': 'https://github.com/hackingmaterials/automatminer/blob/main/README.md?plain=1#L15'
    },
    {
        'leaderboard': 'http://got-10k.aitestunion.com/leaderboard',
        'reference': 'https://github.com/Giveupfree/SOTDrawRect/blob/main/README.md?plain=1#L211'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/hollobit/GenAI_LLM_timeline/blob/main/README.md?plain=1#L2669'
    },
    {
        'leaderboard': 'https://lamp-benchmark.github.io/leaderboard',
        'reference': 'https://github.com/hollobit/GenAI_LLM_timeline/blob/main/README.md?plain=1#L3609'
    },
    {
        'leaderboard': 'https://github.com/lupantech/ScienceQA',
        'reference': ''
    },
    {
        'leaderboard': 'https://leaderboard.allenai.org/scifact',
        'reference': 'https://github.com/dwadden/multivers/blob/main/README.md?plain=1#L7'
    },
    {
        'leaderboard': 'https://www.datafountain.cn/competitions/533/ranking?sch=1844',
        'reference': 'https://github.com/chaineypung/CCFBDCI-2021-Ultrasonic-Tumor-Segmentation-Rank1st/blob/main/README.md?plain=1#L4'
    },
    {
        'leaderboard': 'https://www.kaggle.com/competitions',
        'reference': 'https://github.com/GovLab/toolsforcollaboration/blob/master/README.md?plain=1#L56'
    },
    {
        'leaderboard': 'https://opus.nlpl.eu/dashboard',
        'reference': 'https://github.com/Helsinki-NLP/Opus-MT/blob/master/README.md?plain=1#L8'
    },
    {
        'leaderboard': 'http://www.cvlibs.net/datasets/kitti/eval_scene_flow.php?benchmark=stereo',
        'reference': 'https://github.com/cogsys-tuebingen/mobilestereonet/blob/main/README.md?plain=1#L50'
    },
    {
        'leaderboard': 'https://ogb.stanford.edu/docs/leader_linkprop/#ogbl-wikikg2',
        'reference': 'https://github.com/facebookresearch/ssl-relation-prediction/blob/main/README.md?plain=1#L125'
    },
    {
        'leaderboard': 'https://ogb.stanford.edu/docs/leader_linkprop/#ogbl-biokg',
        'reference': 'https://github.com/facebookresearch/ssl-relation-prediction/blob/main/README.md?plain=1#L126'
    },
    {
        'leaderboard': 'https://github.com/lupantech/ScienceQA',
        'reference': ''
    },
    {
        'leaderboard': 'https://opendrivelab.com/challenge2024/#mapless_driving',
        'reference': 'https://github.com/OpenDriveLab/OpenLane-V2/blob/master/README.md?plain=1#L25'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/AGC2024/mapless-driving-2024',
        'reference': 'https://github.com/OpenDriveLab/OpenLane-V2/blob/master/README.md?plain=1#L25'
    },
    {
        'leaderboard': 'https://opendrivelab.com/challenge2023/#openlane_topology',
        'reference': 'https://github.com/OpenDriveLab/OpenLane-V2/blob/master/README.md?plain=1#L33'
    },
    {
        'leaderboard': 'https://eval.ai/web/challenges/challenge-page/1925/leaderboard',
        'reference': 'https://github.com/OpenDriveLab/OpenLane-V2/blob/master/README.md?plain=1#L33'
    },
    {
        'leaderboard': 'https://github.com/ml-energy/leaderboard',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/opstower-ai/devops-ai-open-leaderboard',
        'reference': 'https://github.com/opstower-ai/llm-opstower/blob/main/README.md?plain=1#L7'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/EffiBench/effibench-leaderboard',
        'reference': 'https://github.com/huangd1999/EffiBench/blob/main/README.md?plain=1#L4'
    },
    {
        'leaderboard': 'https://paperswithcode.com/sota/3d-semantic-segmentation-on-scribblekitti',
        'reference': 'https://github.com/ouenal/scribblekitti/blob/main/README.md?plain=1#L12'
    },
    {
        'leaderboard': 'https://leaderboard.carla.org',
        'reference': 'https://github.com/zhejz/carla-roach/blob/main/README.md?plain=1#L19'
    },
    {
        'leaderboard': 'https://leaderboard.carla.org/leaderboard',
        'reference': 'https://github.com/shuishida/LangProp/blob/main/README.md?plain=1#L300'
    },
    {
        'leaderboard': 'https://github.com/pddasig/Machine-Learning-Competition-2021',
        'reference': ''
    },
    {
        'leaderboard': 'https://livebench.ai',
        'reference': 'https://github.com/LiveBench/LiveBench/blob/main/README.md?plain=1#L6'
    },
    {
        'leaderboard': 'https://yuzuai.jp/benchmark',
        'reference': 'https://github.com/taishi-i/awesome-japanese-nlp-resources/blob/main/README.md?plain=1#L254'
    },
    {
        'leaderboard': 'https://open-xai.github.io/leaderboard',
        'reference': 'https://github.com/hbaniecki/adversarial-explainable-ai/blob/master/README.md?plain=1#L737'
    },
    {
        'leaderboard': 'https://github.com/SHI-Labs/Agriculture-Vision',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/VSainteuf/pastis-benchmark',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/lupantech/MathVista',
        'reference': ''
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/LanguageBind/Video-Bench',
        'reference': 'https://github.com/PKU-YuanGroup/Video-Bench/blob/main/README.md?plain=1#L48'
    },
    {
        'leaderboard': 'https://evo-eval.github.io/leaderboard.html',
        'reference': 'https://github.com/evo-eval/evoeval/blob/main/README.md?plain=1#L4'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/SUSTech-IDEA/SUS-Chat/blob/main/README.md?plain=1#L90'
    },
    {
        'leaderboard': 'https://novelchallenge.github.io/index.html',
        'reference': 'https://github.com/marzenakrp/nocha/blob/main/README.md?plain=1#L10'
    },
    {
        'leaderboard': 'https://breakend.github.io/RL-Energy-Leaderboard/reinforcement_learning_energy_leaderboard',
        'reference': 'https://github.com/Breakend/experiment-impact-tracker/blob/master/README.md?plain=1#L160'
    },
    {
        'leaderboard': 'https://www.cbica.upenn.edu/BraTS18/lboardValidation.html',
        'reference': 'https://github.com/sacmehta/3D-ESPNet/blob/master/README.md?plain=1#L7'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/allenai/reward-bench',
        'reference': 'https://github.com/allenai/reward-bench/blob/main/README.md?plain=1#L4'
    },
    {
        'leaderboard': 'https://github.com/Visual-Attention-Network/SegNeXt',
        'reference': ''
    },
    {
        'leaderboard': 'http://calvin.cs.uni-freiburg.de',
        'reference': 'https://github.com/mees/calvin/blob/main/README.md?plain=1#L169'
    },
    {
        'leaderboard': 'https://nyu-mll.github.io/quality',
        'reference': 'https://github.com/nyu-mll/quality/blob/main/README.md?plain=1#L29'
    },
    {
        'leaderboard': 'https://github.com/CUHK-ARISE/GAMABench',
        'reference': ''
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/prometheus-eval/BiGGen-Bench-Leaderboard',
        'reference': 'https://github.com/prometheus-eval/prometheus-eval/blob/main/README.md?plain=1#L26'
    },
    {
        'leaderboard': 'https://github.com/yizhen20133868/Awesome-SLU-Survey',
        'reference': ''
    },
    {
        'leaderboard': 'https://lmarena.ai/?leaderboard',
        'reference': 'https://github.com/mikeroyal/Machine-Learning-Guide/blob/main/README.md?plain=1#L479'
    },
    {
        'leaderboard': 'https://github.com/LudwigStumpp/llm-leaderboard',
        'reference': 'https://github.com/mikeroyal/Machine-Learning-Guide/blob/main/README.md?plain=1#L480'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/mikeroyal/Machine-Learning-Guide/blob/main/README.md?plain=1#L481'
    },
    {
        'leaderboard': 'https://eval.ai/web/challenges/challenge-page/1663/leaderboard',
        'reference': 'https://github.com/alexa/alexa-with-dstc10-track2-dataset/blob/main/README.md?plain=1#L15'
    },
    {
        'leaderboard': 'https://github.com/ruixiangcui/AGIEval',
        'reference': ''
    },
    {
        'leaderboard': 'https://www.aicrowd.com/challenges/mediqa-2019-natural-language-inference-nli/leaderboards',
        'reference': 'https://github.com/abachaa/MEDIQA2019/blob/master/README.md?plain=1#L46'
    },
    {
        'leaderboard': 'https://www.aicrowd.com/challenges/mediqa-2019-recognizing-question-entailment-rqe/leaderboards',
        'reference': 'https://github.com/abachaa/MEDIQA2019/blob/master/README.md?plain=1#L47'
    },
    {
        'leaderboard': 'https://www.aicrowd.com/challenges/mediqa-2019-question-answering-qa/leaderboards',
        'reference': 'https://github.com/abachaa/MEDIQA2019/blob/master/README.md?plain=1#L48'
    },
    {
        'leaderboard': 'https://github.com/declare-lab/MELD',
        'reference': ''
    },
    {
        'leaderboard': 'https://www.conceptualcaptions.com/winners-and-data',
        'reference': 'https://github.com/google-research-datasets/Image-Caption-Quality-Dataset/blob/master/README.md?plain=1#L93'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/mteb/leaderboard',
        'reference': 'https://github.com/sudarshan-koirala/llm-resources/blob/main/README.md?plain=1#L79'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/sudarshan-koirala/llm-resources/blob/main/README.md?plain=1#L82'
    },
    {
        'leaderboard': 'https://lmarena.ai/?leaderboard',
        'reference': 'https://github.com/sudarshan-koirala/llm-resources/blob/main/README.md?plain=1#L83'
    },
    {
        'leaderboard': 'https://gorilla.cs.berkeley.edu/leaderboard.html',
        'reference': 'https://github.com/sudarshan-koirala/llm-resources/blob/main/README.md?plain=1#L92'
    },
    {
        'leaderboard': 'https://github.com/declare-lab/MELD',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/vectara/hallucination-leaderboard',
        'reference': 'https://github.com/kimtth/awesome-azure-openai-llm/blob/main/README.md?plain=1#L1500'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/lmsys/chatbot-arena-leaderboard',
        'reference': 'https://github.com/kimtth/awesome-azure-openai-llm/blob/main/README.md?plain=1#L1579'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/kimtth/awesome-azure-openai-llm/blob/main/README.md?plain=1#L1581'
    },
    {
        'leaderboard': 'https://github.com/ray-project/llmperf-leaderboard',
        'reference': 'https://github.com/kimtth/awesome-azure-openai-llm/blob/main/README.md?plain=1#L2244'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/ArtificialAnalysis/LLM-Performance-Leaderboard',
        'reference': 'https://github.com/kimtth/awesome-azure-openai-llm/blob/main/README.md?plain=1#L2245'
    },
    {
        'leaderboard': 'https://gluebenchmark.com/leaderboard',
        'reference': 'https://github.com/kimtth/awesome-azure-openai-llm/blob/main/README.md?plain=1#L2261'
    },
    {
        'leaderboard': 'https://super.gluebenchmark.com/leaderboard',
        'reference': 'https://github.com/kimtth/awesome-azure-openai-llm/blob/main/README.md?plain=1#L2261'
    },
    {
        'leaderboard': 'https://github.com/facebookresearch/CompilerGym',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/RUCAIBox/RecBole-GNN/blob/main/results/general/ml-1m.md',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/RUCAIBox/RecBole-GNN/blob/main/results/sequential/diginetica.md',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/RUCAIBox/RecBole-GNN/blob/main/results/social/lastfm.md',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/abenhamadou/3DTeethSeg22_challenge',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/Arthur151/SOTA-on-monocular-3D-pose-and-shape-estimation',
        'reference': ''
    },
    {
        'leaderboard': 'https://paperswithcode.com/dataset/sun-seg-easy',
        'reference': 'https://github.com/GewelsJI/VPS/blob/main/README.md?plain=1#L130'
    },
    {
        'leaderboard': 'https://paperswithcode.com/dataset/sun-seg-hard',
        'reference': 'https://github.com/GewelsJI/VPS/blob/main/README.md?plain=1#L130'
    },
    {
        'leaderboard': 'https://www.cvlibs.net/datasets/kitti',
        'reference': 'https://github.com/daohu527/awesome-self-driving-car/blob/master/README.md?plain=1#L243'
    },
    {
        'leaderboard': 'https://github.com/JusperLee/SonicSim',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/bethgelab/InDomainGeneralizationBenchmark',
        'reference': ''
    },
    {
        'leaderboard': 'https://matbench.materialsproject.org',
        'reference': 'https://github.com/materialsproject/matbench/blob/main/README.md?plain=1#L11'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/shm007g/LLaMA-Cult-and-More/blob/main/README.md?plain=1#L95'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/Vchitect/VBench_Leaderboard',
        'reference': 'https://github.com/modelscope/data-juicer/blob/main/README.md?plain=1#L42'
    },
    {
        'leaderboard': 'https://github.com/OpenGVLab/Multi-Modality-Arena',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/HowieHwong/TrustLLM',
        'reference': ''
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/mteb/leaderboard',
        'reference': 'https://github.com/huggingface/text-embeddings-inference/blob/main/README.md?plain=1#L87'
    },
    {
        'leaderboard': 'https://mcgill-nlp.github.io/imagecode',
        'reference': 'https://github.com/McGill-NLP/imagecode/blob/main/README.md?plain=1#L82'
    },
    {
        'leaderboard': 'https://www.nuscenes.org/tracking',
        'reference': 'https://github.com/lixiaoyu2000/Rock-Track/blob/main/README.md?plain=1#L12'
    },
    {
        'leaderboard': 'https://www.scrolls-benchmark.com/leaderboard',
        'reference': 'https://github.com/huankoh/long-doc-summarization/blob/main/README.md?plain=1#L72'
    },
    {
        'leaderboard': 'https://github.com/SoccerNet/sn-reid',
        'reference': ''
    },
    {
        'leaderboard': 'https://paperswithcode.com/task/cross-domain-few-shot-object-detection',
        'reference': 'https://github.com/lovelyqian/CDFSOD-benchmark/blob/main/README.md?plain=1#L10'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/ChenYi99/EgoPlan-Bench_Leaderboard',
        'reference': 'https://github.com/ChenYi99/EgoPlan/blob/master/README.md?plain=1#L241'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/HenryHZY/Awesome-Multimodal-LLM/blob/main/README.md?plain=1#L283'
    },
    {
        'leaderboard': 'https://codalab.lisn.upsaclay.fr/competitions/421',
        'reference': 'https://github.com/Junjue-Wang/LoveDA/blob/master/README.md?plain=1#L9'
    },
    {
        'leaderboard': 'https://codalab.lisn.upsaclay.fr/competitions/424',
        'reference': 'https://github.com/Junjue-Wang/LoveDA/blob/master/README.md?plain=1#L10'
    },
    {
        'leaderboard': 'https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/leaderboard',
        'reference': 'https://github.com/veb-101/Data-Science-Projects/blob/master/README.md?plain=1#L22'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/opencompass/open_vlm_leaderboard',
        'reference': 'https://github.com/dylanhogg/awesome-python/blob/main/README.md?plain=1#L749'
    },
    {
        'leaderboard': 'https://torchprotein.ai/benchmark',
        'reference': 'https://github.com/DeepGraphLearning/PEER_Benchmark/blob/main/README.md?plain=1#L135'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard',
        'reference': 'https://github.com/Exorust/LLM-Cookbook/blob/main/README.md?plain=1#L166'
    },
    {
        'leaderboard': 'https://henrychur.github.io/MedS-Bench',
        'reference': 'https://github.com/MAGIC-AI4Med/MedS-Ins/blob/main/README.md?plain=1#L6'
    },
    {
        'leaderboard': 'https://github.com/waico/SKAB',
        'reference': ''
    },
    {
        'leaderboard': 'https://github.com/GAIR-NLP/abel',
        'reference': ''
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/BestWishYsh/ChronoMagic-Bench',
        'reference': 'https://github.com/PKU-YuanGroup/ChronoMagic-Bench/blob/main/README.md?plain=1#L205'
    },
    {
        'leaderboard': 'https://github.com/facebookresearch/anli',
        'reference': 'https://github.com/thunlp/TAADpapers/blob/master/README.md?plain=1#L207'
    },
    {
        'leaderboard': 'https://www.llm-reasoners.net/leaderboard',
        'reference': 'https://github.com/maitrix-org/llm-reasoners/blob/main/README.md?plain=1#L52'
    },
    {
        'leaderboard': 'https://eval.ai/web/challenges/challenge-page/463/leaderboard',
        'reference': 'https://github.com/mmurray/cvdn/blob/master/README.md?plain=1#L64'
    },
    {
        'leaderboard': 'https://opendrivelab.com/challenge2023',
        'reference': 'https://github.com/OpenDriveLab/TopoNet/blob/main/README.md?plain=1#L41'
    },
    {
        'leaderboard': 'https://huggingface.co/spaces/AGC2024/driving-with-language-official',
        'reference': 'https://github.com/OpenDriveLab/DriveLM/blob/main/README.md?plain=1#L88'
    },
]

leaderboard_secondary = [
    {
        'original': 'https://www.superclueai.com',
        'descendants': ['https://github.com/CLUEbenchmark/SuperCLUEgkzw', 'https://www.langyb.com']
    },
    {
        'original': 'https://lmarena.ai/?leaderboard',
        'descendants': ['https://redarena.ai/leaderboard']
    },
    {
        'original': 'https://llm.extractum.io/static/llm-leaderboards',
        'descendants': ['https://eqbench.com', 'https://oobabooga.github.io/benchmark.html', 'https://huggingface.co/spaces/DontPlanToEnd/UGI-Leaderboard']
    },
    {
        'original': 'https://predibase.com/fine-tuning-index',
        'descendants': ['https://predibase.com/fine-tuning-leaderboard']
    },
]

In [18]:
print(f'There are {len(dataset_oriented_leaderboards)} dataset-oriented leaderboards.')
print(f'There are {len(method_oriented_leaderboards)} method-oriented leaderboards.')
print(f'There are {len(metric_oriented_leaderboards)} metric-oriented leaderboards.')
# There are 402 EvalAI + 648 Kaggle + 1 TAOST + 5 Artificial Analysis + rest = 1062 solution-oriented leaderboards.
# There are 5312 PapersWithCode + 78 GitHub + 336 HuggingFace + 49 Independent = 5775 model-oriented leaderboards.

There are 4 dataset-oriented leaderboards.
There are 15 method-oriented leaderboards.
There are 1 metric-oriented leaderboards.


In [19]:
with pd.ExcelFile(path_meta / 'Foundation Model Leaderboards.xlsx') as excel_file:
    df_leaderboard = pd.read_excel(excel_file, sheet_name='Leaderboard')

    df_leaderboard_expanded = []
    for idx, row in df_leaderboard.iterrows():
        task = row['Name']
        if task in pwc_leaderboard_mapping and row['Platforms'] == 'pwc':
            sub_names = pwc_leaderboard_mapping[task]
            for sub_name in sub_names:
                new_row = row.copy()
                new_row['Name'] = sub_name
                df_leaderboard_expanded.append(new_row)
        else:
            df_leaderboard_expanded.append(row)

    df_leaderboard_expanded = pd.DataFrame(df_leaderboard_expanded)
    df_leaderboard_expanded.to_csv(path_meta / 'leaderboard_processed.csv', index=False)

In [20]:
leaderboard_hosted_on_multiple_platforms = 0
leaderboard_without_model_linkage = 0
leaderboard_host_platform_mapping = defaultdict(int)

df_leaderboard_expanded = pd.read_csv(path_meta / 'leaderboard_processed.csv')
df_leaderboard_expanded['Development workflows (non-pwc)'] = df_leaderboard_expanded['Development workflows (non-pwc)'].astype(str).apply(string_to_list)
df_leaderboard_expanded['Display formats (non-pwc)'] = df_leaderboard_expanded['Display formats (non-pwc)'].apply(lambda x: string_to_dict(x, list_format=True))
df_leaderboard_expanded['Publication venues (non-pwc)'] = df_leaderboard_expanded['Publication venues (non-pwc)'].apply(string_to_list)
df_leaderboard_expanded['Release organizations (non-pwc)'] = df_leaderboard_expanded['Release organizations (non-pwc)'].apply(string_to_list)

df_leaderboard_expanded['Display formats'] = [[] for _ in range(len(df_leaderboard_expanded))]
df_leaderboard_expanded['Development workflows'] = [[] for _ in range(len(df_leaderboard_expanded))]
df_leaderboard_expanded['Release organizations'] = [[] for _ in range(len(df_leaderboard_expanded))]

for index, row in df_leaderboard_expanded.iterrows():
    display_formats = set(row['Display formats (non-pwc)'])
    publishers = publisher_synonyms_mapping(row['Release organizations (non-pwc)'])
    workflows = set(row['Development workflows (non-pwc)'])
        
    for platform in row['Platforms'].split(','):
        leaderboard_host_platform_mapping[platform] += 1
            
    if len(row['Platforms'].split(',')) > 1:
        leaderboard_hosted_on_multiple_platforms += 1
        
    if row['Platforms'] == 'pwc':
        publishers.add('Papers With Code')
            
    if 'pwc' in row['Platforms']:
        display_formats = display_formats.union(display_format_pwc)
        workflows = workflows.union(workflow_pwc)
    elif pd.isna(row['Model linkage (non-pwc)']):
        leaderboard_without_model_linkage += 1

    df_leaderboard_expanded.at[index, 'Display formats'] = list(display_formats)
    df_leaderboard_expanded.at[index, 'Development workflows'] = list(workflows)
    df_leaderboard_expanded.at[index, 'Release organizations'] = list(publishers)

df_leaderboard_expanded.to_csv(path_meta / 'leaderboard_processed.csv', index=False)

print(f"Total number of leaderboards: {len(df_leaderboard_expanded)}")
print(f"{round(leaderboard_without_model_linkage/len(df_leaderboard_expanded)*100,2)}% ({leaderboard_without_model_linkage} out of {len(df_leaderboard_expanded)}) leaderboards do not have any model provenence links.")
print(f"{round(leaderboard_hosted_on_multiple_platforms/len(df_leaderboard_expanded)*100,2)}% ({leaderboard_hosted_on_multiple_platforms} out of {len(df_leaderboard_expanded)}) leaderboards are hosted on multiple platforms.")

leaderboard_distribution = df_leaderboard_expanded['Platforms'].value_counts().to_dict()

fig = px.bar(
    x=leaderboard_distribution.keys(),
    y=leaderboard_distribution.values(),
    text_auto=True,
    labels={'x': 'Host Platforms', 'y': 'Number of Leaderboards'},
)
# Update the layout for a tighter look
fig.update_layout(
    autosize=True,
    margin=dict(
        l=10,  # Left margin
        r=10,  # Right margin
        b=10,  # Bottom margin
        t=10,  # Top margin
        pad=4  # Padding between the plot and the margin
    ),
    xaxis=dict(title_font=dict(size=18, family='Arial, bold', color='black'),tickfont=dict(color='black')),
    yaxis=dict(title_font=dict(size=18, family='Arial, bold', color='black'),tickfont=dict(color='black')),
)
pio.write_image(fig, f'{path_meta}/platform-combo-distribution.pdf')
fig.update_layout(
    width=1200,  # Set the width of the figure
    height=800,  # Set the height of the figure
)
fig.show()

for key, value in leaderboard_host_platform_mapping.items():
    print(f"{key.upper()}: {value} ({round(value/len(df_leaderboard_expanded)*100,2)}%)")

Total number of leaderboards: 721
16.64% (120 out of 721) leaderboards do not have any model provenence links.
7.91% (57 out of 721) leaderboards are hosted on multiple platforms.


HF: 120 (16.64%)
GH: 120 (16.64%)
PWC: 446 (61.86%)
IP: 96 (13.31%)


In [21]:
df = pd.read_csv(path_meta / 'leaderboard_processed.csv')
df['Development workflows'] = df['Development workflows'].apply(ast.literal_eval)

# df = df[df['Development workflows'].map(len) > 0]
df_workflow = df.explode('Development workflows')
df_workflow = df_workflow.groupby('Development workflows').size().reset_index(name='Frequency')
df_workflow['Development workflows'] = df_workflow['Development workflows'].apply(lambda x: f'Pattern {x}' if x != 'Unknown' else x)
df_workflow.sort_values(by='Frequency', ascending=False, inplace=True)
df_workflow['Weight'] = 1

print(f"{df_workflow['Development workflows'].iloc[0]} is the most prevalent ({df_workflow['Frequency'].iloc[0]} out of {len(df)}) workflow patterns that accounts for {round(df_workflow['Frequency'].iloc[0]/len(df)*100,2)}%.")
df_pwc = df[df['Platforms'].str.contains('pwc')]
print(f"There are {round(len(df[df['Development workflows'].map(len) > 1])/len(df)*100,2)}% ({len(df[df['Development workflows'].map(len) > 1])} out of {len(df)}) non-PWC leaderboards with multiple workflow patterns.")
print(f'In {df_workflow["Development workflows"].iloc[0]}, {round(len(df_pwc)/df_workflow["Frequency"].iloc[0]*100,2)}% ({len(df_pwc)} out of {df_workflow["Frequency"].iloc[0]}) of the leaderboards are hosted on Papers With Code.')

statistics = {
    'Distribution': 'Leaderboards across Workflow Patterns',
    'Mean': np.mean(df_workflow['Frequency']),
    'Median': np.median(df_workflow['Frequency']),
    'IQR': df_workflow['Frequency'].quantile(0.75) - df_workflow['Frequency'].quantile(0.25),
    'Gini Coefficient': ineqpy.inequality.gini(data=df_workflow[df_workflow['Development workflows'] != 'Unknown'], income='Frequency', weights='Weight'),
}
if os.path.exists(path_meta / 'leaderboard_statistics.csv'):
    df_statistics = pd.read_csv(path_meta / 'leaderboard_statistics.csv')
else:
    df_statistics = pd.DataFrame()
df_statistics = pd.concat([df_statistics, pd.DataFrame([statistics])], ignore_index=True)
df_statistics.drop_duplicates(subset=['Distribution'], keep='last', inplace=True)
df_statistics.sort_values(by='Distribution', inplace=True)
df_statistics.to_csv(path_meta / 'leaderboard_statistics.csv', index=False)

df_workflow = df_workflow[df_workflow['Development workflows'] != 'Unknown']
df_workflow['Ratio'] = round(df_workflow['Frequency'] / len(df) * 100, 2)
fig = px.bar(
    x=df_workflow['Development workflows'], 
    y=df_workflow['Ratio'],
    text_auto=True,
    labels={'x': 'Deployment workflows', 'y': 'Leaderboard Percentage'},
)
fig.update_layout(
    width=1200,  # Set the width of the figure
    height=800,  # Set the height of the figure
)
fig.show()

Pattern 1 is the most prevalent (475 out of 721) workflow patterns that accounts for 65.88%.
There are 11.79% (85 out of 721) non-PWC leaderboards with multiple workflow patterns.
In Pattern 1, 93.89% (446 out of 475) of the leaderboards are hosted on Papers With Code.


In [22]:
df = pd.read_csv(path_meta / 'leaderboard_processed.csv')
print(f"{round(len(df[df['#Empty ranking dataframes (non-pwc)'].notna()])/len(df)*100,2)}% ({len(df[df['#Empty ranking dataframes (non-pwc)'].notna()])} out of {len(df)}) leaderboards have empty ranking dataframes.")
df['#Empty ranking dataframes (non-pwc)'].value_counts().reset_index(name='Frequency')

0.42% (3 out of 721) leaderboards have empty ranking dataframes.


,#Empty ranking dataframes (non-pwc),Frequency
0,25,1
1,Unknown,1
2,2,1


In [23]:
df = pd.read_csv(path_meta / 'leaderboard_processed.csv')

df['Release organizations'] = df['Release organizations'].apply(ast.literal_eval)
df_split = df.explode('Release organizations').groupby('Release organizations').size().reset_index(name='Frequency')
df_split['Weight'] = 1

statistics = {
    'Distribution': 'Leaderboards across Release Organizations',
    'Mean': np.mean(df_split['Frequency']),
    'Median': np.median(df_split['Frequency']),
    'IQR': df_split['Frequency'].quantile(0.75) - df_split['Frequency'].quantile(0.25),
    'Gini Coefficient': ineqpy.inequality.gini(data=df_split, income='Frequency', weights='Weight'),
}
if os.path.exists(path_meta / 'leaderboard_statistics.csv'):
    df_statistics = pd.read_csv(path_meta / 'leaderboard_statistics.csv')
else:
    df_statistics = pd.DataFrame()
df_statistics = pd.concat([df_statistics, pd.DataFrame([statistics])], ignore_index=True)
df_statistics.drop_duplicates(subset=['Distribution'], keep='last', inplace=True)
df_statistics.sort_values(by='Distribution', inplace=True)
df_statistics.to_csv(path_meta / 'leaderboard_statistics.csv', index=False)

df_top = df_split.sort_values(by='Frequency', ascending=False).head(10)
print(f"{df_top['Release organizations'].iloc[0]} stands out as the most prolific release organization among the {len(df_split['Release organizations'].unique())-1} identified organizations, contributing to a notable {round(df_top['Frequency'].iloc[0]/len(df)*100,2)}% ({df_top['Frequency'].iloc[0]} out of {len(df)}) leaderboards.")

df_top['Ratio'] = round(df_top['Frequency'] / len(df) * 100, 2)

fig = go.Figure(go.Bar(
        x=df_top['Frequency'],  # Values for the bar lengths
        y=df_top['Release organizations'],  # Categories for each bar
        orientation='h',  # Sets the bars to be horizontal
        text=df_top['Frequency'],  # Adds the values as text on each bar
        textposition='auto'  # Automatically positions the text on the bars
))
fig.update_layout(
        title=f'Number of Leaderboards across Release Organizations (Top 10)',
        xaxis_title='Leaderboard Number',
        yaxis_title='Publisher Name',
        yaxis_autorange='reversed'  # This line makes the bars go top-down
)
fig.update_layout(
    width=1200,  # Set the width of the figure
    height=800,  # Set the height of the figure
)
fig.show()


Papers With Code stands out as the most prolific release organization among the 253 identified organizations, contributing to a notable 61.44% (443 out of 721) leaderboards.


In [24]:
df = pd.read_csv(path_meta / 'leaderboard_processed.csv')
df['Display formats'] = df['Display formats'].apply(ast.literal_eval)

df_multiple = df[df['Display formats'].map(len) > 1]
print(f"{round(len(df_multiple)/len(df)*100,2)}% ({len(df_multiple)} out of {len(df)}) leaderboards support multiple display formats.")
display_format_rankable = ['Rankable Table', 'Rankable Bar Chart']
df_rankable = keep_rows_by_list_column(df, 'Display formats', display_format_rankable)
print(f"{round(len(df_rankable)/len(df)*100,2)}% ({len(df_rankable)} out of {len(df)}) leaderboards support rankable display formats.")
df_rankable_split = df_rankable.explode('Display formats')['Display formats'].value_counts().reset_index(name='Frequency')
print(f"{df_rankable_split['Display formats'].iloc[0]} is the most popular display format of rankable leaderboards, accounting for {round(df_rankable_split['Frequency'].iloc[0]/len(df_rankable)*100,2)}% ({df_rankable_split['Frequency'].iloc[0]} out of {len(df_rankable)}) rankable leaderboards.")
df_split = df.explode('Display formats')['Display formats'].value_counts().reset_index(name='Frequency')
df_split['Weight'] = 1

statistics = {
    'Distribution': 'Leaderboards across Display Formats',
    'Mean': np.mean(df_split['Frequency']),
    'Median': np.median(df_split['Frequency']),
    'IQR': df_split['Frequency'].quantile(0.75) - df_split['Frequency'].quantile(0.25),
    'Gini Coefficient': ineqpy.inequality.gini(data=df_split, income='Frequency', weights='Weight'),
}
if os.path.exists(path_meta / 'leaderboard_statistics.csv'):
    df_statistics = pd.read_csv(path_meta / 'leaderboard_statistics.csv')
else:
    df_statistics = pd.DataFrame()
df_statistics = pd.concat([df_statistics, pd.DataFrame([statistics])], ignore_index=True)
df_statistics.drop_duplicates(subset=['Distribution'], keep='last', inplace=True)
df_statistics.sort_values(by='Distribution', inplace=True)
df_statistics.to_csv(path_meta / 'leaderboard_statistics.csv', index=False)

df_split['Ratio'] = round(df_split['Frequency'] / len(df) * 100, 2)

fig = px.bar(
    x=df_split['Display formats'],
    y=df_split['Frequency'],
    text_auto=True,
    labels={'x': 'Display format', 'y': 'Number of Leaderboards'},
)
# Update the layout for a tighter look
fig.update_layout(
    autosize=True,
    margin=dict(
        l=10,  # Left margin
        r=10,  # Right margin
        b=10,  # Bottom margin
        t=10,  # Top margin
        pad=4  # Padding between the plot and the margin
    )
)
fig.update_layout(
    width=1200,  # Set the width of the figure
    height=800,  # Set the height of the figure
)
fig.show()

75.45% (544 out of 721) leaderboards support multiple display formats.
88.9% (641 out of 721) leaderboards support rankable display formats.
Rankable Table is the most popular display format of rankable leaderboards, accounting for 99.84% (640 out of 641) rankable leaderboards.


In [25]:
df = pd.read_csv(path_meta / 'leaderboard_processed.csv')
df = df[df['Platforms'] != 'pwc']
df['Publication venues (non-pwc)'] = df['Publication venues (non-pwc)'].apply(ast.literal_eval)
df_pub = df[df['Publication venues (non-pwc)'].apply(len) > 0]
print(f"{round(len(df_pub)/len(df)*100,2)}% ({len(df_pub)} out of {len(df)}) leaderboards are associated with specific publications, including research articles, blog posts, and white papers.")

df_literature = filter_rows_by_list_column(df_pub, 'Publication venues (non-pwc)', non_accepted_publication_type)
print(f"{round(len(df_literature)/len(df_pub)*100,2)}% ({len(df_literature)} out of {len(df_pub)}) publications have been accepted in a specific workshop, conference, magzine, or journal.")
df_split = df_literature.explode('Publication venues (non-pwc)').groupby('Publication venues (non-pwc)').size().reset_index(name='Frequency')
df_split['Weight'] = 1

statistics = {
    'Distribution': 'Leaderboards across Publication Venues',
    'Mean': np.mean(df_split['Frequency']),
    'Median': np.median(df_split['Frequency']),
    'IQR': df_split['Frequency'].quantile(0.75) - df_split['Frequency'].quantile(0.25),
    'Gini Coefficient': ineqpy.inequality.gini(data=df_split, income='Frequency', weights='Weight'),
}
if os.path.exists(path_meta / 'leaderboard_statistics.csv'):
    df_statistics = pd.read_csv(path_meta / 'leaderboard_statistics.csv')
else:
    df_statistics = pd.DataFrame()
df_statistics = pd.concat([df_statistics, pd.DataFrame([statistics])], ignore_index=True)
df_statistics.drop_duplicates(subset=['Distribution'], keep='last', inplace=True)
df_statistics.sort_values(by='Distribution', inplace=True)
df_statistics.to_csv(path_meta / 'leaderboard_statistics.csv', index=False)

df_top = df_split.sort_values(by='Frequency', ascending=False).head(10)
print(f"{df_top['Publication venues (non-pwc)'].iloc[0]} emerges as the conference with the most number of accepted publications, accounting for {round(df_top['Frequency'].iloc[0]/len(df_literature)*100,2)}% ({df_top['Frequency'].iloc[0]} out of {len(df_literature)}) of those published.")

df_top['Ratio'] = round(df_top['Frequency'] / len(df_literature) * 100, 2)

fig = go.Figure(go.Bar(
        x=df_top['Frequency'],  # Values for the bar lengths
        y=df_top['Publication venues (non-pwc)'],  # Categories for each bar
        orientation='h',  # Sets the bars to be horizontal
        text=df_top['Frequency'],  # Adds the values as text on each bar
        textposition='auto'  # Automatically positions the text on the bars
))
fig.update_layout(
        title=f'Number of non-PWC Leaderboards with Accepted Publications across Publication Venues (Top 10)',
        xaxis_title='Leaderboard Number',
        yaxis_title='Publication Name',
        yaxis_autorange='reversed'  # This line makes the bars go top-down
)
fig.update_layout(
    width=1200,  # Set the width of the figure
    height=800,  # Set the height of the figure
)
fig.show()


71.22% (198 out of 278) leaderboards are associated with specific publications, including research articles, blog posts, and white papers.
32.32% (64 out of 198) publications have been accepted in a specific workshop, conference, magzine, or journal.
NeurIPS emerges as the conference with the most number of accepted publications, accounting for 21.88% (14 out of 64) of those published.


In [26]:
df = pd.read_csv(path_meta / 'leaderboard_processed.csv')
print(f"{round((len(df)-len(leaderboard_system_without_documentation))/(len(df))*100, 2)}% ({len(df)-len(leaderboard_system_without_documentation)} out of {len(df)}) leaderboards provide documentation.")
print(f"{round((len(df)-len(leaderboard_system_without_contacts))/(len(df))*100, 2)}% ({len(df)-len(leaderboard_system_without_contacts)} out of {len(df)}) leaderboards provide contact information.")
df['Development workflows'] = df['Development workflows'].apply(ast.literal_eval)
df_submission = keep_rows_by_list_column(df, 'Development workflows', workflow_patterns_with_submission)
print(f"{round(len(df_submission)/(len(df))*100, 2)}% ({len(df_submission)} out of {len(df)}) leaderboards provide submission channels.")
df_pwc = df[df['Platforms'] == 'pwc']
print(f"{round((len(df)-len(leaderboard_system_without_evaluation_harness)-len(df_pwc))/(len(df))*100, 2)}% ({len(df)-len(leaderboard_system_without_evaluation_harness)-len(df_pwc)} out of {len(df)}) leaderboards provide evaluation harness.")
print(f"{round((len(df)-len(leaderboard_system_without_pull_requests)-len(df_pwc))/(len(df))*100, 2)}% ({len(df)-len(leaderboard_system_without_pull_requests)-len(df_pwc)} out of {len(df)}) leaderboards provide pull request channels.")
print(f"{round((len(df)-len(leaderboard_system_without_pull_requests)-len(df_pwc))/(len(df))*100, 2)}% ({len(df)-len(leaderboard_system_without_pull_requests)-len(df_pwc)} out of {len(df)}) leaderboards provide discussion forums.")

99.86% (720 out of 721) leaderboards provide documentation.
99.86% (720 out of 721) leaderboards provide contact information.
100.0% (721 out of 721) leaderboards provide submission channels.
38.42% (277 out of 721) leaderboards provide evaluation harness.
36.75% (265 out of 721) leaderboards provide pull request channels.
36.75% (265 out of 721) leaderboards provide discussion forums.


In [27]:
df = pd.read_csv(path_meta / 'leaderboard_statistics.csv')
df.fillna('$\\times$', inplace=True)
df['Mean'] = df['Mean'].map('{:.2f}'.format)
df['Median'] = df['Median'].map('{:.1f}'.format)
df['IQR'] = df['IQR'].map('{:.0f}'.format)
df['Gini Coefficient'] = df['Gini Coefficient'].map(lambda x: '{:.3f}'.format(x) if isinstance(x,float) else x)
print(df.to_latex(index=False))

\begin{tabular}{lllll}
\toprule
Distribution & Mean & Median & IQR & Gini Coefficient \\
\midrule
Leaderboards across Display Formats & 117.73 & 7.0 & 71 & 0.853 \\
Leaderboards across Publication Venues & 3.14 & 1.5 & 3 & 0.517 \\
Leaderboards across Release Organizations & 4.69 & 1.0 & 2 & 0.718 \\
Leaderboards across Workflow Patterns & 161.80 & 77.0 & 220 & 0.722 \\
\bottomrule
\end{tabular}



In [28]:
total_smell_count = 0

# Load the Excel file
with pd.ExcelFile(path_meta / 'Foundation Model Leaderboards.xlsx') as xls:

    # Initialize a dictionary to store unique counts
    unique_counts = {}

    # Iterate over each sheet
    for sheet_name in xls.sheet_names:
        if sheet_name in ['Leaderboard']:
            continue
        
        df = pd.read_excel(xls, sheet_name=sheet_name)
        
        if sheet_name == 'Self-admitted Technical Debt':
            print(f'There are {len(df)+1} SATD examples.')
            continue
    
        # Iterate over each column
        for column in df.columns:
            unique_count = df[column].dropna().nunique()
        
            if sheet_name not in unique_counts:
                unique_counts[sheet_name] = {}
                
            unique_counts[sheet_name][column] = unique_count
            total_smell_count += unique_count

print(f'There are {total_smell_count} leaderboard smell examples.')

# Display the result
result_df = pd.DataFrame(unique_counts).transpose()
result_df = result_df.fillna('')
result_df = result_df.map(lambda x: int(x) if isinstance(x, float) else x)
result_df

There are 34 SATD examples.
There are 320 leaderboard smell examples.


,Benchmark Metric,Benchmark Protocol,Benchmark Raw Dataset,Evaluation Record/Result,Model (Information),Ranking Dataframe,Submission Channel/Guideline,Benchmark Task,Uncategorized
Confusing Entity,4,10,24,11,7,4,21,,
Deprecated Entity,,,3,1,1,7,3,,
Inaccessible Entity,,,2,3,1,13,7,,
Misdisplayed Entity,5,2,,,,3,,,
Mismatched Entity,3,1,2,2,6,3,,2,
Missing Entity,,1,,7,,15,1,,
Redundant Entity,,,,3,2,4,,,
Unresponsive Entity,,,,,,103,3,,
Others,,,,,,,,,30
